# ***💕영업 성공 여부 분류 경진대회💕***

* [0. Install Library](#0.-Install-Library)
* [1. Read Data](#1.-Read-Data)
* [2. EDA & Data Preprocessing](#2.-EDA-&-Data-Preprocessing)
* [3. Feature Engineering](#3.-Feature-Engineering)
<!-- * [4. Model Tuning (Hyperparameter Optimization)](#4.-Model-Tuning-(Hyperparameter-Optimization))
* [5. Model Ensemble](#5.-Model-Ensemble)
* [6. Deployment](#6.-Deployment) -->

***
***

# 0. Install Library

In [1]:
#!pip install lightgbm
#!pip install pycountry

In [2]:
#!pip install lightgbm==3.3.3

In [3]:
from tqdm import tqdm
import re
import gc
import pandas as pd
import numpy as np
import warnings

from sklearn.cluster import KMeans
from kmodes.kmodes import KModes
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

import lightgbm as lgb

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
print(lgb.__version__)

3.3.3


In [5]:
import os
import random
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42) # Seed 고정

In [6]:
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

***
***

# 1. Read Data

### 데이터 셋 읽어오기

In [7]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)
print('train :',len(df_train), ' / test :',len(df_test))
print('중복 값 :',len(df_train[df_train.duplicated()]))
df_train = df_train.drop_duplicates()
df_train = df_train.reset_index(drop=True)
print('중복 제거 후 :',len(df_train))

train : 59299  / test : 5271
중복 값 : 3519
중복 제거 후 : 55780


모든 컬럼 동일하지만 타겟 값이 다른 값 이상치 판단.

In [8]:
print('중복 값 :',len(df_train[df_train.drop(columns = 'is_converted').duplicated(keep=False)]))
df_train = df_train.drop(df_train[df_train.drop(columns = 'is_converted').duplicated(keep=False)].index)
df_train = df_train.reset_index(drop=True)
print('중복 제거 후 :',len(df_train))

중복 값 : 38
중복 제거 후 : 55742


In [9]:
df_raw = df_train.copy()
df_test_raw = pd.read_csv("submission.csv")

***
***

# 2. EDA & Data Preprocessing

* [1. bant_submit](#1.-bant_submit)
* [2. customer_country](#2.-customer_country)
* [3. business_unit](#3.-business_unit)
* [4. com_reg_ver_win_rate](#4.-com_reg_ver_win_rate)
* [5. customer_idx](#5.-customer_idx)
* [6. customer_type](#6.-customer_type)
* [7. enterprise](#7.-enterprise)
* [8. historical_existing_cnt](#8.-historical_existing_cnt)
* [9. id_strategic_ver](#9.-id_strategic_ver)
* [10. it_strategic_ver](#10.-it_strategic_ver)
* [11. idit_strategic_ver](#11.-idit_strategic_ver)
* [12. customer_job](#12.-customer_job)
* [13. lead_desc_length](#13.-lead_desc_length)
* [14. inquiry_type](#14.-inquiry_type)
* [15. product_category](#15.-product_category)
* [16. product_subcategory](#16.-product_subcategory)
* [17. product_modelname](#17.-product_modelname)
* [18. customer_country.1](#18.-customer_country.1)
* [19. customer_position](#19.-customer_position)
* [20. response_corporate](#20.-response_corporate)
* [21. expected_timeline](#21.-expected_timeline)
* [22. ver_cus](#22.-ver_cus)
* [23. ver_pro](#23.-ver_pro)
* [24. ver_win_rate_x](#24.-ver_win_rate_x)
* [25. ver_win_ratio_per_bu](#25.-ver_win_ratio_per_bu)
* [26. business_area](#26.-business_area)
* [27. business_subarea](#27.-business_subarea)
* [28. lead_owner](#28.-lead_owner)
* [29. is_converted](#29.-is_converted)

***
***

## ***1. bant_submit***
MQL 구성 요소들 중 [1]Budget(예산), [2]Title(고객의 직책/직급), [3]Needs(요구사항), [4]Timeline(희망 납기일) 4가지 항목에 대해서 작성된 값의 비율

In [10]:
df_train['bant_submit'].isnull().sum()

0

In [11]:
df_train['bant_submit'].value_counts()

bant_submit
0.50    16491
1.00    16395
0.75    11383
0.25    10968
0.00      505
Name: count, dtype: int64

***
***

## ***2. customer_country***
고객의 국적

결측값 대체 불가 -> 'unknown'으로 지정  
type str로 수정

In [12]:
df_train['customer_country'] = df_train['customer_country'].apply(lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x)
df_test['customer_country'] = df_test['customer_country'].apply(lambda x: re.sub(r'\s+', ' ', x) if isinstance(x, str) else x)

In [13]:
df_train['customer_country'] = df_train['customer_country'].apply(lambda x: re.sub(r'\s*/\s*', '/', x) if isinstance(x, str) else x)
df_test['customer_country'] = df_test['customer_country'].apply(lambda x: re.sub(r'\s*/\s*', '/', x) if isinstance(x, str) else x)

In [14]:
df_train['customer_country2'] = df_train['customer_country'].copy()
df_test['customer_country2'] = df_test['customer_country'].copy()

In [15]:
df_train['customer_country'].isnull().sum()

974

In [16]:
#customer_country NAN
df_train['customer_country'] = df_train['customer_country'].fillna('Unknown')
df_test['customer_country'] = df_test['customer_country'].fillna('Unknown')

df_train['customer_country'] = df_train['customer_country'].astype(str)
df_test['customer_country'] = df_test['customer_country'].astype(str)

* Email 정리.

In [17]:
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('India', case=True, na=False)),'customer_country'] = 'India'
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('Brazil', case=True, na=False)),'customer_country'] = 'Brazil'
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('Chile', case=True, na=False)),'customer_country'] = 'Chile'
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('Colombia', case=True, na=False)),'customer_country'] = 'Colombia'
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('Italy', case=True, na=False)),'customer_country'] = 'Italy'
df_train.loc[(df_train['customer_country'].str.contains('@', case=True, na=False)) & (df_train['customer_country'].str.contains('Mexico', case=True, na=False)),'customer_country'] = 'Mexico'

In [18]:
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('India', case=True, na=False)),'customer_country'] = 'India'
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('Brazil', case=True, na=False)),'customer_country'] = 'Brazil'
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('Chile', case=True, na=False)),'customer_country'] = 'Chile'
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('Colombia', case=True, na=False)),'customer_country'] = 'Colombia'
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('Italy', case=True, na=False)),'customer_country'] = 'Italy'
df_test.loc[(df_test['customer_country'].str.contains('@', case=True, na=False)) & (df_test['customer_country'].str.contains('Mexico', case=True, na=False)),'customer_country'] = 'Mexico'

In [19]:
df_train.loc[df_train['customer_country'].str.contains('@', case=True, na=False), 'customer_country'] = 'Egypt'
df_test.loc[df_test['customer_country'].str.contains('@', case=True, na=False), 'customer_country'] = 'Egypt'

* 국가명으로 정리.

In [20]:
import pycountry

# 모든 국가 정보 얻기
all_countries = list(pycountry.countries)
country_names_lst = [country.name for country in all_countries]

country_names_lst = [name.replace('Türkiye', 'Turkey') for name in country_names_lst]
country_names_lst = [name.replace('Viet Nam', 'VietNam') for name in country_names_lst]
country_names_lst = [name.replace('Tanzania, United Republic of', 'Tanzania') for name in country_names_lst]
country_names_lst = [name.replace("Korea, Republic of", 'Korea') for name in country_names_lst]
country_names_lst = [name.replace("Czechia", 'Czech') for name in country_names_lst]
country_names_lst = [name.replace('Bolivia, Plurinational State of', 'Bolivia') for name in country_names_lst]
# virgin island 는 통일
country_names_lst.remove('Virgin Islands, British')
country_names_lst.remove('Virgin Islands, U.S.')
country_names_lst.remove('Niger')
country_names_lst.remove('Dominican Republic')
country_names_lst.append('Virgin Islands')
###
country_names_lst = [name.replace("Venezuela, Bolivarian Republic of", 'Venezuela') for name in country_names_lst]
country_names_lst = [name.replace("Iran, Islamic Republic of", 'Iran') for name in country_names_lst]
country_names_lst = [name.replace("Taiwan, Province of China", 'Taiwan') for name in country_names_lst]
country_names_lst = [name.replace("Russian Federation", 'Russia') for name in country_names_lst]
country_names_lst = [name.replace("Côte d'Ivoire", 'Ivory Coast') for name in country_names_lst]
country_names_lst = [name.replace("North Macedonia", 'Macedonia') for name in country_names_lst]
country_names_lst = [name.replace("Brunei Darussalam", 'Brunei') for name in country_names_lst]
country_names_lst = [name.replace("Palestine, State of", 'Palestine') for name in country_names_lst]
country_names_lst = [name.replace("Antigua and Barbuda", 'Antigua') for name in country_names_lst]
country_names_lst = [name.replace("Syrian Arab Republic", 'Syria') for name in country_names_lst]
country_names_lst = [name.replace("Lao People's Democratic Republic", 'Laos') for name in country_names_lst]
country_names_lst = [name.replace("Saint Kitts and Nevis", 'Saint Kitts') for name in country_names_lst]
country_names_lst = [name.replace("Saint Martin (French part)", 'Saint Martin') for name in country_names_lst]
country_names_lst.append('Kosovo')
country_names_lst.append('Swaziland')

In [21]:
for country_name in tqdm(country_names_lst):
    condition = df_train['customer_country'].str.contains(country_name, case=False, na=False)
    df_train.loc[condition, 'customer_country'] = country_name

    condition = df_test['customer_country'].str.contains(country_name, case=False, na=False)
    df_test.loc[condition, 'customer_country'] = country_name

100%|██████████| 248/248 [00:05<00:00, 42.46it/s]


### *주요도시 기반 cleansing*

In [22]:
# 튀르키예
df_train.loc[df_train['customer_country'].str.contains('Türkiye', case=False, na=False), 'customer_country'] = 'Turkey'
df_test.loc[df_test['customer_country'].str.contains('Türkiye', case=False, na=False), 'customer_country'] = 'Turkey'

# 미국 EE. UU. 로 표기
df_train.loc[df_train['customer_country'].str.contains('EE. UU.', case=True, na=False), 'customer_country'] = 'United States'
df_test.loc[df_test['customer_country'].str.contains('EE. UU.', case=True, na=False), 'customer_country'] = 'United States'

# 발렌수엘라 (필리핀 섬)
df_train.loc[df_train['customer_country'].str.contains('Valenzuela', case=True, na=False), 'customer_country'] = 'Philippines'
df_test.loc[df_test['customer_country'].str.contains('Valenzuela', case=True, na=False), 'customer_country'] = 'Philippines'

# 중국 상해
df_train.loc[df_train['customer_country'].str.contains('上海', case=True, na=False), 'customer_country'] = 'China'
df_test.loc[df_test['customer_country'].str.contains('上海', case=True, na=False), 'customer_country'] = 'China'

# 아랍에미리트
df_train.loc[df_train['customer_country'].str.contains('U\.A\.E', case=True, na=False), 'customer_country'] = 'United Arab Emirates'
df_test.loc[df_test['customer_country'].str.contains('U\.A\.E', case=True, na=False), 'customer_country'] = 'United Arab Emirates'

#코트디부아르
df_train.loc[df_train['customer_country'].str.contains("Cote d'Ivoire", case=True, na=False), 'customer_country'] = 'Ivory Coast'
df_test.loc[df_test['customer_country'].str.contains("Cote d'Ivoire", case=True, na=False), 'customer_country'] = 'Ivory Coast'

#세인트루시아
df_train.loc[df_train['customer_country'].str.contains("Soufriere", case=True, na=False), 'customer_country'] = 'Saint Lucia'
df_test.loc[df_test['customer_country'].str.contains("Soufriere", case=True, na=False), 'customer_country'] = 'Saint Lucia'

# 대만
df_train.loc[df_train['customer_country'].str.contains("Taipei", case=True, na=False), 'customer_country'] = 'Taiwan'
df_test.loc[df_test['customer_country'].str.contains("Taipei", case=True, na=False), 'customer_country'] = 'Taiwan'

#세인트키츠
df_train.loc[df_train['customer_country'].str.contains("St Kitts", case=True, na=False), 'customer_country'] = 'Saint Kitts'
df_test.loc[df_test['customer_country'].str.contains("St Kitts", case=True, na=False), 'customer_country'] = 'Saint Kitts'

#세인트 마르턴
df_train.loc[df_train['customer_country'].str.contains("St Maarten", case=True, na=False), 'customer_country'] = 'Saint Martin'
df_test.loc[df_test['customer_country'].str.contains("St Maarten", case=True, na=False), 'customer_country'] = 'Saint Martin'

# Australia
df_train.loc[df_train['customer_country'].str.contains("Daydream Island", case=True, na=False), 'customer_country'] = 'Australia'
df_test.loc[df_test['customer_country'].str.contains("Daydream Island", case=True, na=False), 'customer_country'] = 'Australia'

df_train.loc[df_train['customer_country'].str.contains("Kelvin Grove", case=True, na=False), 'customer_country'] = 'Australia'
df_test.loc[df_test['customer_country'].str.contains("Kelvin Grove", case=True, na=False), 'customer_country'] = 'Australia'

# Fiji
df_train.loc[df_train['customer_country'].str.contains("Suva", case=True, na=False), 'customer_country'] = 'Fiji'
df_test.loc[df_test['customer_country'].str.contains("Suva", case=True, na=False), 'customer_country'] = 'Fiji'

# Hungary
df_train.loc[df_train['customer_country'].str.contains("Budapest", case=True, na=False), 'customer_country'] = 'Hungary'
df_test.loc[df_test['customer_country'].str.contains("Budapest", case=True, na=False), 'customer_country'] = 'Hungary'

# Armenia
df_train.loc[df_train['customer_country'].str.contains("AVAN", case=True, na=False), 'customer_country'] = 'Armenia'
df_test.loc[df_test['customer_country'].str.contains("AVAN", case=True, na=False), 'customer_country'] = 'Armenia'

# Singapore
df_train.loc[df_train['customer_country'].str.contains("Kallang Place", case=True, na=False), 'customer_country'] = 'Singapore'
df_test.loc[df_test['customer_country'].str.contains("Kallang Place", case=True, na=False), 'customer_country'] = 'Singapore'

# Jojia
df_train.loc[df_train['customer_country'].str.contains("rbilisi", case=True, na=False), 'customer_country'] = 'Jojia'
df_test.loc[df_test['customer_country'].str.contains("rbilisi", case=True, na=False), 'customer_country'] = 'Jojia'

# VietNam
df_train.loc[df_train['customer_country'].str.contains('Ha Noi', case=True, na=False), 'customer_country'] = 'VietNam'
df_test.loc[df_test['customer_country'].str.contains('Ha Noi', case=True, na=False), 'customer_country'] = 'VietNam'

# Colombia
df_train.loc[df_train['customer_country'].str.contains('Cartagena', case=True, na=False), 'customer_country'] = 'Colombia'
df_test.loc[df_test['customer_country'].str.contains('Cartagena', case=True, na=False), 'customer_country'] = 'Colombia'

df_train.loc[df_train['customer_country'].str.contains('Bucaramanga', case=True, na=False), 'customer_country'] = 'Colombia'
df_test.loc[df_test['customer_country'].str.contains('Bucaramanga', case=True, na=False), 'customer_country'] = 'Colombia'

# Greece
df_train.loc[df_train['customer_country'].str.contains('Θέση', case=True, na=False), 'customer_country'] = 'Greece'
df_test.loc[df_test['customer_country'].str.contains('Θέση', case=True, na=False), 'customer_country'] = 'Greece'

# UAE
df_train.loc[df_train['customer_country'].str.contains("Dubai", case=True, na=False), 'customer_country'] = 'United Arab Emirates'
df_test.loc[df_test['customer_country'].str.contains("Dubai", case=True, na=False), 'customer_country'] = 'United Arab Emirates'

df_train.loc[df_train['customer_country'].str.contains("DUBAI", case=True, na=False), 'customer_country'] = 'United Arab Emirates'
df_test.loc[df_test['customer_country'].str.contains("DUBAI", case=True, na=False), 'customer_country'] = 'United Arab Emirates'

# Namibia
df_train.loc[df_train['customer_country'].str.contains("Windhoek", case=True, na=False), 'customer_country'] = 'Namibia'
df_test.loc[df_test['customer_country'].str.contains("Windhoek", case=True, na=False), 'customer_country'] = 'Namibia'

# China
df_train.loc[df_train['customer_country'].str.contains("shenzhen", case=True, na=False), 'customer_country'] = 'China'
df_test.loc[df_test['customer_country'].str.contains("shenzhen", case=True, na=False), 'customer_country'] = 'China'

# Japan
df_train.loc[df_train['customer_country'].str.contains("Tokyo", case=True, na=False), 'customer_country'] = 'Japan'
df_test.loc[df_test['customer_country'].str.contains("Tokyo", case=True, na=False), 'customer_country'] = 'Japan'

# Switzerland
df_train.loc[df_train['customer_country'].str.contains("Bergdietikon", case=True, na=False), 'customer_country'] = 'Switzerland'
df_test.loc[df_test['customer_country'].str.contains("Bergdietikon", case=True, na=False), 'customer_country'] = 'Switzerland'

# South Africa
df_train.loc[df_train['customer_country'].str.contains("Midrand", case=True, na=False), 'customer_country'] = 'South Africa'
df_test.loc[df_test['customer_country'].str.contains("Midrand", case=True, na=False), 'customer_country'] = 'South Africa'

# Egypt
df_train.loc[df_train['customer_country'].str.contains("Mohandessein", case=True, na=False), 'customer_country'] = 'Egypt'
df_test.loc[df_test['customer_country'].str.contains("Mohandessein", case=True, na=False), 'customer_country'] = 'Egypt'

df_train.loc[df_train['customer_country'].str.contains("Cairo", case=True, na=False), 'customer_country'] = 'Egypt'
df_test.loc[df_test['customer_country'].str.contains("Cairo", case=True, na=False), 'customer_country'] = 'Egypt'

# Indonesia
df_train.loc[df_train['customer_country'].str.contains("Jakarta", case=True, na=False), 'customer_country'] = 'Indonesia'
df_test.loc[df_test['customer_country'].str.contains("Jakarta", case=True, na=False), 'customer_country'] = 'Indonesia'

# Nigeria
df_train.loc[df_train['customer_country'].str.contains("Ikoyi", case=True, na=False), 'customer_country'] = 'Nigeria'
df_test.loc[df_test['customer_country'].str.contains("Ikoyi", case=True, na=False), 'customer_country'] = 'Nigeria'

df_train.loc[df_train['customer_country'].str.contains("Abuja", case=True, na=False), 'customer_country'] = 'Nigeria'
df_test.loc[df_test['customer_country'].str.contains("Abuja", case=True, na=False), 'customer_country'] = 'Nigeria'

# Luxembourg
df_train.loc[df_train['customer_country'].str.contains("Esch-sur-Alzette", case=True, na=False), 'customer_country'] = 'Luxembourg'
df_test.loc[df_test['customer_country'].str.contains("Esch-sur-Alzette", case=True, na=False), 'customer_country'] = 'Luxembourg'

# Dominica
df_train.loc[df_train['customer_country'].str.contains("Arroyo Hondo Viejo", case=True, na=False), 'customer_country'] = 'Dominica'
df_test.loc[df_test['customer_country'].str.contains("Arroyo Hondo Viejo", case=True, na=False), 'customer_country'] = 'Dominica'

df_train.loc[df_train['customer_country'].str.contains("Santo Domingo", case=True, na=False), 'customer_country'] = 'Dominica'
df_test.loc[df_test['customer_country'].str.contains("Santo Domingo", case=True, na=False), 'customer_country'] = 'Dominica'

# Poland
df_train.loc[df_train['customer_country'].str.contains("Olsztyn", case=True, na=False), 'customer_country'] = 'Poland'
df_test.loc[df_test['customer_country'].str.contains("Olsztyn", case=True, na=False), 'customer_country'] = 'Poland'

df_train.loc[df_train['customer_country'].str.contains("Kielce", case=True, na=False), 'customer_country'] = 'Poland'
df_test.loc[df_test['customer_country'].str.contains("Kielce", case=True, na=False), 'customer_country'] = 'Poland'

df_train.loc[df_train['customer_country'].str.contains("Stalowa Wola", case=True, na=False), 'customer_country'] = 'Poland'
df_test.loc[df_test['customer_country'].str.contains("Stalowa Wola", case=True, na=False), 'customer_country'] = 'Poland'

df_train.loc[df_train['customer_country'].str.contains("Warszawska", case=True, na=False), 'customer_country'] = 'Poland'
df_test.loc[df_test['customer_country'].str.contains("Warszawska", case=True, na=False), 'customer_country'] = 'Poland'

# Norway
df_train.loc[df_train['customer_country'].str.contains("Oslo", case=True, na=False), 'customer_country'] = 'Norway'
df_test.loc[df_test['customer_country'].str.contains("Oslo", case=True, na=False), 'customer_country'] = 'Norway'

# Jamaica
df_train.loc[df_train['customer_country'].str.contains("Portmore", case=True, na=False), 'customer_country'] = 'Jamaica'
df_test.loc[df_test['customer_country'].str.contains("Portmore", case=True, na=False), 'customer_country'] = 'Jamaica'

# Mongolia
df_train.loc[df_train['customer_country'].str.contains("Ulaanbaatar", case=True, na=False), 'customer_country'] = 'Mongolia'
df_test.loc[df_test['customer_country'].str.contains("Ulaanbaatar", case=True, na=False), 'customer_country'] = 'Mongolia'

# Germany
df_train.loc[df_train['customer_country'].str.contains("Berlin", case=True, na=False), 'customer_country'] = 'Germany'
df_test.loc[df_test['customer_country'].str.contains("Berlin", case=True, na=False), 'customer_country'] = 'Germany'

df_train.loc[df_train['customer_country'].str.contains("Hamburg", case=True, na=False), 'customer_country'] = 'Germany'
df_test.loc[df_test['customer_country'].str.contains("Hamburg", case=True, na=False), 'customer_country'] = 'Germany'

df_train.loc[df_train['customer_country'].str.contains("Herdwangen-Schönach", case=True, na=False), 'customer_country'] = 'Germany'
df_test.loc[df_test['customer_country'].str.contains("Herdwangen-Schönach", case=True, na=False), 'customer_country'] = 'Germany'

df_train.loc[df_train['customer_country'].str.contains("Weikersheim", case=True, na=False), 'customer_country'] = 'Germany'
df_test.loc[df_test['customer_country'].str.contains("Weikersheim", case=True, na=False), 'customer_country'] = 'Germany'

# Bahamas
df_train.loc[df_train['customer_country'].str.contains("Nassau", case=True, na=False), 'customer_country'] = 'Bahamas'
df_test.loc[df_test['customer_country'].str.contains("Nassau", case=True, na=False), 'customer_country'] = 'Bahamas'

# Uzbekistan
df_train.loc[df_train['customer_country'].str.contains("Tashkent", case=True, na=False), 'customer_country'] = 'Uzbekistan'
df_test.loc[df_test['customer_country'].str.contains("Tashkent", case=True, na=False), 'customer_country'] = 'Uzbekistan'

# Qatar
df_train.loc[df_train['customer_country'].str.contains("Almirqab", case=True, na=False), 'customer_country'] = 'Qatar'
df_test.loc[df_test['customer_country'].str.contains("Almirqab", case=True, na=False), 'customer_country'] = 'Qatar'

df_train.loc[df_train['customer_country'].str.contains("Doha", case=True, na=False), 'customer_country'] = 'Qatar'
df_test.loc[df_test['customer_country'].str.contains("Doha", case=True, na=False), 'customer_country'] = 'Qatar'

# Czech
df_train.loc[df_train['customer_country'].str.contains("Králův Dvůr Levín", case=True, na=False), 'customer_country'] = 'Czech'
df_test.loc[df_test['customer_country'].str.contains("Králův Dvůr Levín", case=True, na=False), 'customer_country'] = 'Czech'

# Thailand
df_train.loc[df_train['customer_country'].str.contains("Buengkum", case=True, na=False), 'customer_country'] = 'Thailand'
df_test.loc[df_test['customer_country'].str.contains("Buengkum", case=True, na=False), 'customer_country'] = 'Thailand'

# Iceland
df_train.loc[df_train['customer_country'].str.contains("Gardabaer", case=True, na=False), 'customer_country'] = 'Iceland'
df_test.loc[df_test['customer_country'].str.contains("Gardabaer", case=True, na=False), 'customer_country'] = 'Iceland'

# Malaysia
df_train.loc[df_train['customer_country'].str.contains("Shah Alam", case=True, na=False), 'customer_country'] = 'Malaysia'
df_test.loc[df_test['customer_country'].str.contains("Shah Alam", case=True, na=False), 'customer_country'] = 'Malaysia'

df_train.loc[df_train['customer_country'].str.contains("Kuala Lumpur", case=True, na=False), 'customer_country'] = 'Malaysia'
df_test.loc[df_test['customer_country'].str.contains("Kuala Lumpur", case=True, na=False), 'customer_country'] = 'Malaysia'

# Lesotho
df_train.loc[df_train['customer_country'].str.contains("Maseru", case=True, na=False), 'customer_country'] = 'Lesotho'
df_test.loc[df_test['customer_country'].str.contains("Maseru", case=True, na=False), 'customer_country'] = 'Lesotho'

# Sri Lanka
df_train.loc[df_train['customer_country'].str.contains("Colombo 03", case=True, na=False), 'customer_country'] = 'Sri Lanka'
df_test.loc[df_test['customer_country'].str.contains("Colombo 03", case=True, na=False), 'customer_country'] = 'Sri Lanka'

# Austria
df_train.loc[df_train['customer_country'].str.contains("Klagenfurt", case=True, na=False), 'customer_country'] = 'Austria'
df_test.loc[df_test['customer_country'].str.contains("Klagenfurt", case=True, na=False), 'customer_country'] = 'Austria'

# Netherlands
df_train.loc[df_train['customer_country'].str.contains("Den Haag", case=True, na=False), 'customer_country'] = 'Netherlands'
df_test.loc[df_test['customer_country'].str.contains("Den Haag", case=True, na=False), 'customer_country'] = 'Netherlands'

# Uganda
df_train.loc[df_train['customer_country'].str.contains("Kampala", case=True, na=False), 'customer_country'] = 'Uganda'
df_test.loc[df_test['customer_country'].str.contains("Kampala", case=True, na=False), 'customer_country'] = 'Uganda'

# Aruba
df_train.loc[df_train['customer_country'].str.contains("Paradera", case=True, na=False), 'customer_country'] = 'Aruba'
df_test.loc[df_test['customer_country'].str.contains("Paradera", case=True, na=False), 'customer_country'] = 'Aruba'

### *각국 도시 기반 Cleansing*

In [23]:
# Brazil 도시
Brazil_lst = ['Sao Paulo', 'São Paulo', 'João Pessoa', 'Capão da canoa', 'Cuiabá', 'Buzios', 'Aparecida',
             'Manaus', 'Foz de Iguaçu', 'Dourados', 'Recife', 'Rio de Janeiro']
for i in Brazil_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Brazil'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Brazil'
    
# Spain 도시
Spain_lst = ['Madrid', 'MADRID', 'Barrio Viejo De Callosa De Segura', 'CACERES', 'VALENCIA', 'GRAN CANARIAS PLAYA DEL INGLES',
             'ALICANTE'
            ]
for i in Spain_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Spain'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Spain'
    
# Saudi Arabia 도시
Saudi_Arabia_lst = ['Riyadh', 'riyadh', 'Makkah', 'Qatif'
            ]
for i in Saudi_Arabia_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Saudi Arabia'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Saudi Arabia'
    
# Canada 도시
Canada_lst = ['Dubreuilville', 'Port Renfrew', 'Alliston', 'Montreal', 'Etobicoke', 'Hamilton', 'Sandford'
            ]
for i in Canada_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Canada'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Canada'
    
# VietNam 도시
VietNam_lst = ['Hai Duong','Thai Binh'
            ]
for i in VietNam_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'VietNam'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'VietNam'
    
# India 도시
India_lst = ['Mumbai','mumbai','ROURKELA','udupi','bhilai','Ajmer','Satana','Jodhpur City','Bangalore','surat','BANGALORE','Ammanabrolu',
             'Mysore','Muzaffarpur','Lucknow','Bemetara','Buldana','sangli','Kanpur','ahemdabad','Davangere','raipur','Dhanbad',
             'bathinda','Jalgaon','namakkal', 'Guna mp', 'Hosapete','indore','Noida','hyderabad','Ludhiana', 'GUWAHATI','Delhi',
             'Ahmedabad', 'Madurai', 'SOLAN', 'Bengaluru', 'Kanchipuram', 'Sangamner', 'Rajkot', 'Raichur', 'Bhubaneswar', 'Malur',
             'Faridabad', 'Mathura', 'Vasai', 'kanpur', 'Ballia','MADURAI'
            ]
for i in India_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'India'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'India'
    
# Philippines 도시
Philippines_lst = ['Angeles City','Paranaque','Cavite','CITY OF MARIKINA', 'San Juan', 'Quezon City', 'Qc', 'QUEZON CITY',
                   'Bulacan', 'Makati', 'manila', 'Cebu', 'Pasig', 'CALOOCAN CITY', 'dasmarinas city', 'Mandaluyong', 'makati', 'Bataan', 'TAYTAY', 'Las Piñas - Muntinlupa', 'PASIG CITY', 'Manila', 'San Fernando City, La Union',
                   'Parañaque','Rizal','CITY OF IMUS','San Pedro', 'Camarines Sur'
            ]
for i in Philippines_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Philippines'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Philippines'
    
# Jordan 도시
Jordan_lst = ['Amman']

for i in Jordan_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Jordan'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'Jordan'
    
# UK 도시
UK_lst = ['Liss','Manchester','Providencaies','buckley', 'Newport', 'Windermere'
            ]
for i in UK_lst:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'United Kingdom'
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'customer_country'] = 'United Kingdom'

### *미국 따로 처리*

In [24]:
us_cities = [
    "New York City", "New York", "Los Angeles", "California", "Chicago", "Illinois", "Houston", "Texas", "Philadelphia", "Pennsylvania",    "Phoenix", "Arizona", "San Antonio", "Texas", "San Francisco", "California", "Boston", "Massachusetts", "Washington, D.C.", "Dallas", "Texas", "Atlanta", "Georgia", "Miami", "Florida","Seattle", "Washington","Denver", "Colorado","Austin", "Texas","Detroit", "Michigan","Charlotte", "North Carolina","San Diego", "California",    "Minneapolis", "Minnesota","Tampa", "Florida","Orlando", "Florida","Portland", "Oregon","St. Louis", "Missouri","Pittsburgh", "Pennsylvania",
    "Sacramento", "California","Las Vegas", "Nevada","Cincinnati", "Ohio","Kansas City", "Missouri","Cleveland", "Ohio","San Jose", "California","Nashville", "Tennessee","Indianapolis", "Indiana","Columbus", "Ohio"]
us_cities += [
    "Phoenix","CARPINTERIA","BROOKLYN","COLUMBUS", "Arizona","San Antonio", "Texas","San Diego", "California","Dallas", "Texas","San Jose", "California","Austin", "Texas","Jacksonville", "Florida","Fort Worth", "Texas","Columbus", "Ohio","Charlotte", "North Carolina","Indianapolis", "Indiana","San Francisco", "California",    "Seattle", "Washington","Denver", "Colorado","Washington", "Boston", "Massachusetts","El Paso", "Texas","Nashville", "Tennessee","Portland", "Oregon","Oklahoma City", "Oklahoma","Las Vegas", "Nevada","Baltimore", "Maryland","Louisville", "Kentucky", "Milwaukee", "Wisconsin","Albuquerque", "New Mexico","Tucson", "Arizona","Fresno", "California","Sacramento", "California","Kansas City", "Missouri","Long Beach", "California","Mesa", "Arizona","Atlanta", "Georgia","Colorado Springs", "Colorado","Raleigh", "North Carolina", "Miami", "Florida","Oakland", "California","Tulsa", "Oklahoma","Cleveland", "Ohio","Wichita", "Kansas","Arlington", "Texas","New Orleans", "Louisiana","Bakersfield", "California","Tampa", "Florida","Honolulu", "Hawaii","Aurora", "Colorado", "Anaheim", "California","Santa Ana", "California","St. Louis", "Missouri","Riverside", "California","Corpus Christi", "Texas","Lexington", "Kentucky","Pittsburgh", "Pennsylvania","Anchorage", "Alaska","Stockton", "California","Cincinnati", "Ohio","St. Paul", "Minnesota",
    "Toledo", "Ohio","Greensboro", "North Carolina","Newark", "New Jersey","Plano", "Texas","Henderson", "Nevada","Lincoln", "Nebraska","Buffalo", "New York","Fort Wayne", "Indiana","Jersey City", "New Jersey","Chula Vista", "California","Orlando", "Florida", "St. Petersburg", "Florida","Norfolk", "Virginia","Chandler", "Arizona","Laredo", "Texas","Madison", "Wisconsin","Durham", "North Carolina","Lubbock", "Texas","Winston-Salem", "North Carolina","Garland", "Texas","Glendale", "Arizona","Hialeah", "Florida","Reno", "Nevada","Baton Rouge", "Louisiana","Irvine", "California",]
us_cities += [
    "Chesapeake", "Virginia","Irving", "Texas","Scottsdale", "Arizona","North Las Vegas", "Nevada","Fremont", "California","Gilbert", "Arizona","San Bernardino", "California","Boise", "Idaho","Birmingham", "Alabama","Rochester", "New York","Richmond", "Virginia","Spokane", "Washington", "Des Moines", "Iowa","Montgomery", "Alabama","Modesto", "California","Fayetteville", "North Carolina","Tacoma", "Washington","Shreveport", "Louisiana","Fontana", "California","Oxnard", "California","Aurora", "Illinois","Moreno Valley", "California","Akron", "Ohio", "Yonkers", "New York","Columbus", "Georgia","Little Rock", "Arkansas","Augusta", "Georgia","Amarillo", "Texas","Mobile", "Alabama","Huntington Beach", "California","Glendale", "California","Grand Rapids", "Michigan","Salt Lake City", "Utah","Tallahassee", "Florida", "Huntsville", "Alabama","Grand Prairie", "Texas","Overland Park", "Kansas","Knoxville", "Tennessee","Worcester", "Massachusetts","Brownsville", "Texas","Newport News", "Virginia","Santa Clarita", "California","Port St. Lucie", "Florida","Providence", "Rhode Island","Fort Lauderdale", "Florida",
    "Chattanooga", "Tennessee","Tempe", "Arizona","Oceanside", "California","Garden Grove", "California","Rancho Cucamonga", "California","Cape Coral", "Florida","Santa Rosa", "California","Vancouver", "Washington","Sioux Falls", "South Dakota","Peoria", "Arizona","Ontario", "California", "Jackson", "Mississippi","Elk Grove", "California","Springfield", "Missouri","Pembroke Pines", "Florida","Salem", "Oregon","Corona", "California","Eugene", "Oregon","McKinney", "Texas","Fort Collins", "Colorado","Lancaster", "California","Cary", "North Carolina","Palmdale", "California","Hayward", "California","Salinas", "California","Frisc", "Texas",]
us_cities += [
    "Philadelphia", "Pittsburgh","Allentown","Erie","Reading","Scranton","Bethlehem","Lancaster","Harrisburg","Altoona","York","Wilkes-Barre","Chester","Williamsport","Easton","Lebanon","Hazleton","New Castle","Johnstown","McKeesport","Pottstown","Sharon","West Chester", "Norristown","Hermitage","Monroeville","Plum","State College","Easton","Baldwin","Wilkinsburg","Hanover","Bethel Park","Franklin Park","Levittown","Phoenixville","Lansdale","Warminster","Butler","Murrysville","Carlisle","Greensburg","West Mifflin","Chambersburg", "McKees Rocks","Pottsville","Wilkins","King of Prussia","Shaler","McKeesport","Meadville","New Kensington","Stowe","Nanticoke","Uniontown", "Ellwood City","Montgomeryville","Jeannette","South Park","Limerick","Fernway","Haverford","Bristol","Brentwood","Whitehall","Leacock", "Yeadon","Cranberry","Baldwin","Lower Burrell","Bristol","West Norriton","Moon","Bloomsburg","Horsham","East Hempfield","Wayne","Bethel","Monessen","Elizabethtown","Ephrata","Willow Grove","Kingston","Kennett","Coraopolis","Exeter","Northampton", "North Huntingdon","Millcreek","South Williamsport","Springfield","Bristol","Westtown","Warren","South Middleton","Upper Providence","McCandless","Hanover", "South Whitehall","Manheim","Lancaster","East Hempfield","Upper Darby","West Goshen","Newtown","Upper Merion","Whitehall","Whitemarsh","West Lampeter","Lower Macungie","North Huntingdon","Concord","Upper Southampton","Radnor","Middletown","East Bradford","Springfield","West Whiteland","Whitpain", "East Brandywine","West Hempfield","Upper Providence","East Cocalico","Lower Southampton","East Norriton","Whitfield","Lower Merion","Murrysville","Springfield","Newtown","Lower Paxton","West Manchester","East Lampeter","Warminster","Hampden","Lower Providence","Lower Makefield","Spring Garden","Lebanon","North Middleton",
    "Swatara","Palmer","Manheim","Upper Macungie","Springettsbury","West Hanover","South Fayette","East Hempfield","Marple","East Pikeland","Upper Dublin","West Earl","East Marlborough","Lower Southampton","South Middleton","Horsham","Lower Saucon","North Strabane","Warwick","Spring","South Whitehall", "West Vincent","West Hempfield","Hempfield","North Fayette","Upper Makefield","Plumstead","Springfield","Robeson","Whitemarsh","West Manchester","Londonderry","North Lebanon","East Goshen","Middletown","West Brandywine","Susquehanna","Lebanon","Warrington","Sadsbury","Mount Joy","Lower Gwynedd", "Richland","East Pennsboro","West Whiteland","Plumstead","South Park","Warminster","East Vincent","Newberry","Springfield","Tredyffrin","Westtown","North Codorus","Silver Spring","Franklin","Whitehall","Windsor","Cranberry","West Cocalico","Ferguson","North Middleton","Mount Lebanon","East Bradford","Richland","Easttown", "Springfield","East Norriton","Concord","West Bradford","Manheim","Spring Garden","Hanover","West Goshen","Lower Makefield","North Lebanon","Exeter","West Donegal","Economy","Cranberry","Upper Uwchlan","Londonderry","West Nantmeal","North Cornwall","Springfield","East Brandywine","New Hanover","Lebanon","West Pikeland","Upper Providence","Palmer","Sadsbury","South Londonderry"]
us_cities += [
    'San Francisco', ' US', ' USA', 'Boston', 'Chicago', 'U.S.', 'Southfield', 'San Diego', 'New York', 'Los Angeles','Washington DC', 'Houston', 'St. Louis', 'Newton', 'Harrisburg', 'SOUTH SAN FRANCISCO','Hollywood', 'HOUSTON', 'Los Angelos', 'Greenville', 'Clinton', 'Charleston','Batavi', 'Omaha', 'Rockleigh', 'Spirit Lake', 'Draper', 'Hot Springs', 'Schaumburg','Kenner', 'Plainfield', 'Mentor', 'Monterey', 'Stowe', 'Hawthorne', 'Encino', 'Fargo','Elysburg', 'Newberry Springs', 'Ashland', 'Newark', 'Waukegan', 'Ocean Shores', 'Phoenix', 'Sterling', 'Fairfax', 'Garden Grove', 'DRAPER', 'NEWARK', 'Silver Spring',
    'Manhattan', 'utica', 'Mashantucket', 'phoenix', 'Houghton', 'Cambridge', 'Topeka', 'Richland', 'chicago','Dexter', 'Danbury', 'ocean shores', 'Hyannis', 'Kihei', 'Mountville', 'Frederick', 'Naperville', 'Malvern', 'Palo Alto', 'Bellevue', 'Mooresville', 'Norwalk', 'Leesburg', 'Cicero', 'Coffeyville', 'garden grove', 'Orrville', 'Fair Oaks', 'ANAHEIM', 'Kuna', 'Not Hispanic or Latino', 'washington', 'Westfield', 'san antonio', 'Dayton', 'miami', 'Shawnee', 'Slidell','Swink', 'Reston', 'Los banos', 'Salida', 'Marina Del Rey', 'APO', 'PHOENIX', 'Houlton','miami','Durango', 'El paso', 'Beloit', 'Fort Bragg', 'Latham','Snellville','Davison', 'Lewisville', 'Sierra Vista','Hemlock','Soda Springs','Godley','Colonty', 'Post Falls', 'Myrtle Beach', 'Monrovia', 'Douglas', 'Bloomington', 'Zip 98433', 'Archbald', 'Twinsburg', 'Woodhaven','Sunnyvale', 'Chico', 'Nogales', 'Morrow','Nicholasville','Bradenton', 'costa mesa','Holly Springs','Carlsbad','Goodland','Reading','Peachtree Corners','Statesville','Upper Saint Clair','Fenton','Ladson','Saint Maries','5503 major blvd','Lander','Wilkes Barre', 'Altamonte Spg', 'Boulder', 'Bowie', 'Santa Barbara', 'Maple Grove', 'Merrifield', 'Sandusky', 'Hacienda Heights', 'Shepherdstown', 'El Segundo', 'Troy', 'Lafayette', 'Versailles','St Augustine Beach', 'Federal Way', 'Santa Monica', 'Independence', 'Offutt AFB', 'Logan', 'Coral Gables', 'Blue Bell', 'Odessa', 'Bloomingdale', 'Andersen AFB', 'Morristown', 'Lenoir City', 'Bridgeville', 'Marietta']
us_cities += [
    'Federal Way','Federal way', 'Wixom', 'Fort Pierce', 'Boca Raton', 'Mt Laurel Township', 'Stamford', 'Lakewood','Lake Ozark', 'Ann Arbor', 'Uniondale', 'Chuluota', 'CLIFTON', 'East Syracuse', 'Walnut Creek', 'Pasadena', 'Edmond', 'DeKalb', 'Bismarck', 'Torrance','Jefferson', 'Coalgate', 'Schriever Space Force Base', 'Maywood', 'Inglewood', 'South Bend', 'Woodside', 'Smoketown', 'Ellensburg','Agoura Hills', 'Eufaula', 'Randolph', 'Laurel', 'Dublin',"Diamond Point","Bellevue","Ridgefield","Gallipolis","Liverpool",'liverpool',"Eagle Creek","Topeka","Monroe Township","Lake Buena Vista","Otis","Mountainside",'Olympia', 'topeka', 'pasadena', 'edmond', 'dublin', 'laurel', 'stamford','Prattville','inglewood','Bridgeport','bellevue','mountainside','Lewis Center','Alpharetta','Delray Beach','Ipswich','Ashburn','Largo','Farmers Branch','new york','Keshena','Waunakee','Blackville','Haverhill','Skokie','Carson','Cloverdale','Cypress','Vandenberg SFB','Brooklyn','City of Industry','Wellsboro','gardena','Saipan', 'Friendswood','Canton','Auburn','Wildwood','Maryville','Hapeville','La Jolla','The Colony','Fort Belvoir','Mankato','El Dorado Hills','Pismo Beach','White Plains','Fishkill','Stennis Space Center','Mill Valley','Annapolis','Chantilly','Los Gatos','Camden','Lambertville','Hyattsville','Sunrise','Peachtree City','Saint Petersburg','Basin','College Place','Decatur','phila','Columbs','Covington','Gig Harbor','Watertown','St. Cloud','Lac Du Flambeau','ridgeville','South Burlington','Beavercreek','Marysville','La Grande','Bandon','Normal','Weatherford','Moab','Lowell', 'Midlothian', 'Centerville', 'plymouth meeting', 'El Cajon', 'Gulf Shores', 'Biloxi', 'Lindenhurst', 'Jupiter', 'Grand Canyon','Plainsboro', 'East Rutherford', 'Rexburg', 'Oak Brook', 'Bell Gardens','Woodstock', 'Bensalem', 'Yreka', 'Lenexa', 'Summerfield', 'Burbank','Palm Valley', 'Orange', 'Massapequa', 'Lagrange', 'detroit', 'Laughlin','Danville', 'san rafael', 'Severance', 'Poughkeepsie', 'Danville','Eglin AFB', 'houston', 'Palmetto', 'Connellsville', 'Marion', 'port Orange','Wallace', 'athens', 'Lansing', 'Delavan', 'Niles', 'Goleta', 'Olathe','Joliet', 'Eva', 'woodbury', 'Montecito', 'Riverton', 'springdale','Farmington', 'Norcross', 'Eagan', 'Duncan', 'Rodeville', 'Rockville','Castle Rock', 'La Mirada', 'Manassas', 'Cupertino', 'Arkport','Taunton','Camp Hill','Santa Clara','Humboldt','Pooler','Palm Coast','Hackensack','San Leandro','Mission Hills','debary','austin','14700 Caribbean Way', '3 Nasson Avenue',
    'Romulus','Renton','Doral','Kaysville','Meridian','Pompano Beac','Ferndale','St George','Hackensack','Ardmore','Staten Island','Opa-Locka','Great Falls','Champaign','North Babylon','Goldsboro','Kahului','Provo','South bend', 'Mount Juliet','Deptford','Terlingua','Hogansburg','Davenport','Eden Prairie','Morgantown','Oyster Bay','Westwood','Fortuna','Isle Of Palms','Kiawah Island','Eldersburg','Mandeville','Lewiston','Tyerl','Dover','brooklyn','Haltom City','Forney','Kennebunkport','pigeon forge','Cedarville','College Station','Alhambra','Weldon','North Smithfield','Ronkonkoma','barstow','Yuma','Monroe','Cathedral City','Nyack','Glencoe','Andover','Altamont springs','New Bedford','Miramar','Cashmere','Joint Base Lewis McChord','Homestead','Novi','Hunt Valley','Melbourne','Acworth','Nellis AFB','Kent','Kimberly','Mountain Lakes','FT HUACHUCA','Shelbyville','Bothell','montclair','Broken Arrow','Moffett Field','Youngstown','Roseville','Piscataway','Battle Mountain','Roswell','Curtis Bay','laGrange','Sanford','Chilhowie','Amherst','winston','Mahopac','Tinley Park','Kingsville','Manhasset','Abidjan','New Iberia','Holland','Elmsford','Owings mills','Ankeny','Lake Mary','Petersburg','trenton','Oxon Hill','Mableton','Tusayan','Ackerman','Mesquite','Round Rock','North Have','Seagoville','Kingwood','Dyersburg','Oroville','Wheatland','Littleton','League City','Flushing','Dearborn Heights','Speedway','Bell gardens','Lanham','Ft. Myers','Sparta','Stoughton','Charlevoix','Glenndale','Ridgewood','Carthage','Katy','South Shore','Ks','Rehoboth Beach','Bangor','Benbrook','Hastings','Middleburg Heights','Wylie','Edina','Sedona','Missoula','Litleton','lauderhill','Wilton','Flushing','Solana beach','Alameda', '3 Nasson Avenue']

In [25]:
for i in tqdm(us_cities):
    condition = df_train['customer_country'].str.contains(i, case=True, na=False)
    df_train.loc[condition, 'customer_country'] = 'United States'

    condition = df_test['customer_country'].str.contains(i, case=True, na=False)
    df_test.loc[condition, 'customer_country'] = 'United States'

100%|██████████| 1115/1115 [00:18<00:00, 59.67it/s]


In [26]:
df_train.loc[(df_train['customer_country'].str.contains(r'\d', na=False)) & (df_train['response_corporate'] == 'LGEUS'), 'customer_country'] = 'United States'
df_test.loc[(df_test['customer_country'].str.contains(r'\d', na=False)) & (df_test['response_corporate'] == 'LGEUS'), 'customer_country'] = 'United States'

In [27]:
df_train.loc[(df_train['customer_country'].str.contains(r'\d', na=False)) & (df_train['response_corporate'] == 'LGEIS'), 'customer_country'] = 'Italy'
df_test.loc[(df_test['customer_country'].str.contains(r'\d', na=False)) & (df_test['response_corporate'] == 'LGEIS'), 'customer_country'] = 'Italy'

In [28]:
df_train.loc[(df_train['customer_country'].str.contains(r'\d', na=False)) & (df_train['response_corporate'] == 'LGEIL'), 'customer_country'] = 'India'
df_test.loc[(df_test['customer_country'].str.contains(r'\d', na=False)) & (df_test['response_corporate'] == 'LGEIL'), 'customer_country'] = 'India'

In [29]:
df_train.loc[(df_train['customer_country'].str.contains(r'\d', na=False)) & (df_train['response_corporate'] == 'LGECB'), 'customer_country'] = 'Colombia'
df_test.loc[(df_test['customer_country'].str.contains(r'\d', na=False)) & (df_test['response_corporate'] == 'LGECB'), 'customer_country'] = 'Colombia'

In [30]:
# '/' 포함되면 unknown으로 간주.
df_train.loc[df_train['customer_country'].str.contains(r'\/', na=False), 'customer_country'] = 'Unknown'
df_test.loc[df_test['customer_country'].str.contains(r'\/', na=False), 'customer_country'] = 'Unknown'

In [31]:
df_tmp = df_train[df_train['customer_country'] != 'Unknown']
df_tmp = df_tmp.groupby('lead_owner')['customer_country'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None).reset_index()

In [32]:
for i in range(len(df_tmp)):
    lead = df_tmp.loc[i, 'lead_owner']
    co = df_tmp.loc[i, 'customer_country']
    df_train.loc[(df_train['lead_owner'] == lead) & (df_train['customer_country'] == 'Unknown'), 'customer_country'] = co
    df_test.loc[(df_test['lead_owner'] == lead) & (df_test['customer_country'] == 'Unknown'), 'customer_country'] = co

***
***

## ***3. business_unit***
MQL 요청 상품에 대응되는 사업부

In [33]:
df_train['business_unit'].isnull().sum()

0

In [34]:
df_train['business_unit'].value_counts()

business_unit
ID          24849
AS          22646
IT           7959
Solution      287
CM              1
Name: count, dtype: int64

In [35]:
tmp_df = pd.concat([pd.DataFrame(df_train.groupby('business_unit')['is_converted'].mean()), pd.DataFrame(df_train.groupby('business_unit')['is_converted'].count())], axis=1)
tmp_df.columns = ['영업전환_확률', 'Count']
tmp_df

,영업전환_확률,Count
business_unit,,
AS,0.060187,22646
CM,0.000000,1
ID,0.102902,24849
IT,0.085187,7959
Solution,0.010453,287


***
***

## ***4. com_reg_ver_win_rate***
(Vertical Level 1, business unit, region을 기준으로 oppty 비율을 계산)

In [36]:
df_train['com_reg_ver_win_rate'].isnull().sum()

41640

In [37]:
df_train['com_reg_ver_win_rate'].describe()

count    14102.000000
mean         0.086132
std          0.141022
min          0.003788
25%          0.019900
50%          0.049180
75%          0.074949
max          1.000000
Name: com_reg_ver_win_rate, dtype: float64

***
***

## ***5. customer_idx***
고객의 회사명

In [38]:
df_train['customer_idx'].isnull().sum()

0

In [39]:
tmp_df = pd.concat([pd.DataFrame(df_train.groupby('customer_idx')['is_converted'].mean()), pd.DataFrame(df_train.groupby('customer_idx')['is_converted'].count())], axis=1)
tmp_df.columns = ['영업전환_확률', 'Count']

In [40]:
tmp_df[(tmp_df['Count'] > 10) & (tmp_df['영업전환_확률'] > 0.95)].sort_values('영업전환_확률', ascending = False)

,영업전환_확률,Count
customer_idx,,
9324,1.0,14
9857,1.0,11
25096,1.0,2404


In [41]:
tmp_df[(tmp_df['Count'] > 50) & (tmp_df['영업전환_확률'] == 0.00)].sort_values('영업전환_확률', ascending = False)

,영업전환_확률,Count
customer_idx,,
25309,0.0,86
29370,0.0,60
31864,0.0,68
33350,0.0,97
33763,0.0,88
37654,0.0,75


***
***

## ***6. customer_type***
고객 유형

In [42]:
df_train['customer_type'].isnull().sum()

41354

In [43]:
df_train['customer_type'].nunique()

33

In [44]:
df_train.loc[df_train['customer_type'].str.contains('end', case=False, na=False), 'customer_type'] = 'End-Customer'
df_test.loc[df_test['customer_type'].str.contains('end', case=False, na=False), 'customer_type'] = 'End-Customer'

df_train.loc[df_train['customer_type'].str.contains('Specifier', case=False, na=False), 'customer_type'] = 'Specifier/Influencer'
df_test.loc[df_test['customer_type'].str.contains('Specifier', case=False, na=False), 'customer_type'] = 'Specifier/Influencer'

df_train.loc[df_train['customer_type'].str.contains('other', case=False, na=False), 'customer_type'] = 'other'
df_test.loc[df_test['customer_type'].str.contains('other', case=False, na=False), 'customer_type'] = 'other'

# df_train.loc[df_train['customer_type'] == "Etc.", "customer_type"] = 'other'
# df_test.loc[df_test['customer_type'] == "Etc.", "customer_type"] = 'other'

df_train.loc[df_train['customer_type'].str.contains('home', case=False, na=False), 'customer_type'] = 'Home Owner'
df_test.loc[df_test['customer_type'].str.contains('home', case=False, na=False), 'customer_type'] = 'Home Owner'

df_train.loc[df_train['customer_type'].str.contains('Software', case=False, na=False), 'customer_type'] = 'Software/Solution Provider'
df_test.loc[df_test['customer_type'].str.contains('Software', case=False, na=False), 'customer_type'] = 'Software/Solution Provider'

In [45]:
df_train['customer_type'] = df_train['customer_type'].fillna('Unknown')
df_test['customer_type'] = df_test['customer_type'].fillna('Unknown')

In [46]:
df_train['customer_type'].nunique()

27

***
***

## ***7. enterprise***
Global 기업인지, Small/Medium 규모의 기업인지

In [47]:
df_train['enterprise'].isnull().sum()

0

In [48]:
df_train['enterprise'].value_counts()

enterprise
Enterprise    35626
SMB           20116
Name: count, dtype: int64

In [49]:
tmp_df = pd.concat([pd.DataFrame(df_train.groupby('enterprise')['is_converted'].mean()), pd.DataFrame(df_train.groupby('enterprise')['is_converted'].count())], axis=1)
tmp_df.columns = ['영업전환_확률', 'Count']
tmp_df

,영업전환_확률,Count
enterprise,,
Enterprise,0.057851,35626
SMB,0.126268,20116


***
***

## ***8. historical_existing_cnt***
이전에 Converted(영업 전환) 되었던 횟수

In [50]:
df_train['historical_existing_cnt'].isnull().sum()

43380

In [51]:
df_train['historical_existing_cnt'].describe()

count    12362.000000
mean        20.189937
std         45.482021
min          0.000000
25%          1.000000
50%          4.000000
75%         19.000000
max       1394.000000
Name: historical_existing_cnt, dtype: float64

***
***

## ***9. id_strategic_ver***
(도메인 지식) 특정 사업부(Business Unit이 ID일 때), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여

> 특정 사업부 -> ID / 특정 사업 영역 -> ['corporate / office', 'hotel & accommodation']

In [52]:
df_train['id_strategic_ver'].isnull().sum()

52409

In [53]:
df_train['id_strategic_ver'].value_counts()

id_strategic_ver
1.0    3333
Name: count, dtype: int64

In [54]:
df_train[df_train['id_strategic_ver'].notna()]['business_area'].unique()

array(['corporate / office', 'hotel & accommodation'], dtype=object)

결측치 0으로 대체

In [55]:
df_train['id_strategic_ver'] = df_train['id_strategic_ver'].fillna(0)
df_test['id_strategic_ver'] = df_test['id_strategic_ver'].fillna(0)

***
***

## ***10. it_strategic_ver***
(도메인 지식) 특정 사업부(Business Unit이 IT일 때), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여

> 특정 사업부 -> IT / 특정 사업 영역 -> ['corporate / office', 'hotel & accommodation']

In [56]:
df_train['it_strategic_ver'].isnull().sum()

54634

In [57]:
df_train['it_strategic_ver'].value_counts()

it_strategic_ver
1.0    1108
Name: count, dtype: int64

In [58]:
df_train[df_train['it_strategic_ver'].notna()]['business_area'].unique()

array(['corporate / office', 'hotel & accommodation'], dtype=object)

결측치 0으로 대체

In [59]:
df_train['it_strategic_ver'] = df_train['it_strategic_ver'].fillna(0)
df_test['it_strategic_ver'] = df_test['it_strategic_ver'].fillna(0)

***
***

## ***11. idit_strategic_ver***
Id_strategic_ver이나 it_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현

In [60]:
df_train['idit_strategic_ver'].isnull().sum()

51301

In [61]:
df_train['idit_strategic_ver'].value_counts()

idit_strategic_ver
1.0    4441
Name: count, dtype: int64

결측치 0으로 대체

In [62]:
df_train['idit_strategic_ver'] = df_train['idit_strategic_ver'].fillna(0)
df_test['idit_strategic_ver'] = df_test['idit_strategic_ver'].fillna(0)

***
***

## ***12. customer_job***
고객의 직업군

In [63]:
df_train['customer_job'].isnull().sum()

16776

In [64]:
df_train['customer_job'].value_counts()

customer_job
engineering                       6021
other                             4500
administrative                    3282
education                         2260
sales                             2138
                                  ... 
consultant,cabinet fabricator        1
appliance specialist                 1
av estimator                         1
purchasers                           1
exhibition / convention center       1
Name: count, Length: 560, dtype: int64

In [65]:
df_train['customer_job'].nunique()

560

결측치 Unknown으로 대체

In [66]:
#customer_job NAN
df_train['customer_job'] = df_train['customer_job'].fillna('Unknown')
df_test['customer_job'] = df_test['customer_job'].fillna('Unknown')

일부 오탈자 및 비슷한 직업 묶기

In [67]:
# engineering
engineering_lst = df_train.loc[df_train['customer_job'].str.contains('engine', case=False, na=False), 'customer_job'].unique()

for i in engineering_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'engineering'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'engineering'
    
# administrative
administrative_lst = df_train.loc[df_train['customer_job'].str.contains('admin', case=False, na=False), 'customer_job'].unique()

for i in administrative_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'administrative'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'administrative'
    
#education
education_lst = list(df_train.loc[df_train['customer_job'].str.contains('edu', case=False, na=False), 'customer_job'].unique())

for i in education_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'education'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'education'
    
# administrative
administrative_lst = df_train.loc[df_train['customer_job'].str.contains('admin', case=False, na=False), 'customer_job'].unique()

for i in administrative_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'administrative'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'administrative'

#healthcare services
healthcare_services_lst = list(df_train.loc[df_train['customer_job'].str.contains('health', case=False, na=False), 'customer_job'].unique())

for i in healthcare_services_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'healthcare services'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'healthcare services'
    
# sales
sales_lst = df_train.loc[df_train['customer_job'].str.contains('sale', case=False, na=False), 'customer_job'].unique()

for i in sales_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'sales'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'sales'
    
# operations
operations_lst = df_train.loc[df_train['customer_job'].str.contains('sale', case=False, na=False), 'customer_job'].unique()

for i in operations_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'operations'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'operations'

# information technology
information_technology_lst = df_train.loc[df_train['customer_job'].str.contains('inform', case=False, na=False), 'customer_job'].unique()

for i in information_technology_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'information technology'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'information technology'
    
# purchasing
purchasing_lst = df_train.loc[df_train['customer_job'].str.contains('purchas', case=False, na=False), 'customer_job'].unique()

for i in purchasing_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'purchasing'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'purchasing'
    
# business development
business_development_lst = df_train.loc[df_train['customer_job'].str.contains('develop', case=False, na=False), 'customer_job'].unique()

for i in business_development_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'business development'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'business development'
    
# consulting
consulting_lst = df_train.loc[df_train['customer_job'].str.contains('consult', case=False, na=False), 'customer_job'].unique()

for i in consulting_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'consulting'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'consulting'

# art and design
art_and_design_lst = list(df_train.loc[df_train['customer_job'].str.contains('design', case=False, na=False), 'customer_job'].unique())
art_and_design_lst.extend(['graphic/color art', 'artist, lead on equipment selection', 'art installation', '3d/vfx art', 'arte y diseño'])

for i in art_and_design_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'art and design'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'art and design'
    
# marketing
marketing_lst = df_train.loc[df_train['customer_job'].str.contains('market', case=False, na=False), 'customer_job'].unique()

for i in marketing_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'marketing'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'marketing'
    
# program and project management
program_and_project_management_lst = df_train.loc[df_train['customer_job'].str.contains('program', case=False, na=False), 'customer_job'].unique()

for i in program_and_project_management_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'program and project management'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'program and project management'
    
program_and_project_management_lst_1 = df_train.loc[df_train['customer_job'].str.contains('project', case=False, na=False), 'customer_job'].unique()

for i in program_and_project_management_lst_1:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'program and project management'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'program and project management'
    
# media and communication
media_and_communication_lst = df_train.loc[df_train['customer_job'].str.contains('communication', case=False, na=False), 'customer_job'].unique()

for i in media_and_communication_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'media and communication'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'media and communication'
    
media_and_communication_lst_1 = df_train.loc[df_train['customer_job'].str.contains('media', case=False, na=False), 'customer_job'].unique()

for i in media_and_communication_lst_1:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'media and communication'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'media and communication'
    
# product management
product_management_lst = df_train.loc[df_train['customer_job'].str.contains('product_management', case=False, na=False), 'customer_job'].unique()

for i in product_management_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'product management'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'product management'
    
# finance
finance_lst = df_train.loc[df_train['customer_job'].str.contains('market', case=False, na=False), 'customer_job'].unique()

for i in finance_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'finance'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'finance'
    
# other
other_lst = df_train.loc[df_train['customer_job'].str.contains('other', case=False, na=False), 'customer_job'].unique()

for i in other_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'other'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'other'
    
# accounting
accounting_lst = df_train.loc[df_train['customer_job'].str.contains('account', case=False, na=False), 'customer_job'].unique()

for i in accounting_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'accounting'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'accounting'
    
# support
support_lst = df_train.loc[df_train['customer_job'].str.contains('account', case=False, na=False), 'customer_job'].unique()

for i in support_lst:
    df_train.loc[df_train['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'support'
    df_test.loc[df_test['customer_job'].str.contains(i, case=False, na=False), 'customer_job'] = 'support'

561 -> 384

In [68]:
df_train['customer_job'].nunique()

384

***
***

## ***13. lead_desc_length***
고객이 작성한 Lead Descriptoin 텍스트 총 길이

In [69]:
df_train['lead_desc_length'].isnull().sum()

0

In [70]:
df_train['lead_desc_length'].describe()

count    55742.000000
mean        80.824872
std        133.883205
min          1.000000
25%          8.000000
50%         31.000000
75%         94.000000
max       1264.000000
Name: lead_desc_length, dtype: float64

***
***

## ***14. inquiry_type***
고객의 문의 유형

In [71]:
df_train['inquiry_type'].isnull().sum()

891

In [72]:
df_train['inquiry_type'].value_counts().head(5)

inquiry_type
Quotation or purchase consultation    22904
Quotation or Purchase Consultation    17194
Sales Inquiry                          8844
Product Information                    1100
Other                                   901
Name: count, dtype: int64

In [73]:
df_train['inquiry_type'].nunique()

71

일부 오탈자 및 비슷한 직업 묶기

In [74]:
# Quotation or Purchase Consultation
df_train.loc[df_train['inquiry_type'].str.contains("quotation", case=False, na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'
df_test.loc[df_test['inquiry_type'].str.contains("quotation", case=False, na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'
df_train.loc[df_train['inquiry_type'].str.contains("purchase", case=False, na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'
df_test.loc[df_test['inquiry_type'].str.contains("purchase", case=False, na=False), 'inquiry_type'] = 'Quotation or Purchase Consultation'

# Sales Inquiry
df_train.loc[df_train['inquiry_type'].str.contains("sale", case=False, na=False), 'inquiry_type'] = 'Sales Inquiry'
df_test.loc[df_test['inquiry_type'].str.contains("sale", case=False, na=False), 'inquiry_type'] = 'Sales Inquiry'

# Usage or Technical Consultation
df_train.loc[df_train['inquiry_type'].str.contains("Technical Consultation", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'
df_test.loc[df_test['inquiry_type'].str.contains("Technical Consultation", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'
df_train.loc[df_train['inquiry_type'].str.contains("usage", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'
df_test.loc[df_test['inquiry_type'].str.contains("usage", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'
df_train.loc[df_train['inquiry_type'].str.contains("Technical", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'
df_test.loc[df_test['inquiry_type'].str.contains("Technical", case=False, na=False), 'inquiry_type'] = 'Usage or Technical Consultation'

# Other
df_train.loc[df_train['inquiry_type'].str.contains("other", case=False, na=False), 'inquiry_type'] = 'Other'
df_test.loc[df_test['inquiry_type'].str.contains("other", case=False, na=False), 'inquiry_type'] = 'Other'
df_train.loc[df_train['inquiry_type'].str.contains("etc", case=False, na=False), 'inquiry_type'] = 'Other'
df_test.loc[df_test['inquiry_type'].str.contains("etc", case=False, na=False), 'inquiry_type'] = 'Other'
#Product Information
df_train.loc[df_train['inquiry_type'].str.contains("product", case=False, na=False), 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'].str.contains("product", case=False, na=False), 'inquiry_type'] = 'Product Information'


# Request for Partnership
df_train.loc[df_train['inquiry_type'].str.contains("partnership", case=False, na=False), 'inquiry_type'] = 'Request for Partnership'
df_test.loc[df_test['inquiry_type'].str.contains("partnership", case=False, na=False), 'inquiry_type'] = 'Request for Partnership'


# Services
df_train.loc[df_train['inquiry_type'].str.contains("service", case=False, na=False), 'inquiry_type'] = 'Services'
df_test.loc[df_test['inquiry_type'].str.contains("service", case=False, na=False), 'inquiry_type'] = 'Services'


# Trainings
df_train.loc[df_train['inquiry_type'].str.contains("train", case=False, na=False), 'inquiry_type'] = 'Trainings'
df_test.loc[df_test['inquiry_type'].str.contains("train", case=False, na=False), 'inquiry_type'] = 'Trainings'

71 -> 44

In [75]:
df_train['inquiry_type'].nunique()

44

***
***

## ***15. product_category***
요청 제품 카테고리

In [76]:
df_train['raw_product_category'] = df_train['product_category'].copy()
df_test['raw_product_category'] = df_test['product_category'].copy()

In [77]:
df_train['product_category'].isnull().sum()

17146

In [78]:
df_train['product_category'].nunique()

357

In [79]:
product_dict ={
'interactive digital board' : 'Commercial Display / Interactive Signage', 'vrf' : 'HVAC, ESS / VRF', 'multi-split' : 'HVAC, ESS / Multi-Split', 'video wall signage' : 'Commercial Display / Video Wall Signage', 'etc.' : 'etc.', 'led signage' : 'Commercial Display / LED Signage', 'interactive signage' : 'Commercial Display / Interactive Signage', 'single-split' : 'HVAC, ESS / Single-Split', 'rac' : 'HVAC, ESS / AC', 'oled signage' : 'Commercial Display / OLED Signage', 'hotel tv' : 'Commercial Display / Hotel TV', 'chiller' : 'HVAC, ESS / Chiller', 'standard signage' : 'Commercial Display / Standard Signage', 'medical display' : 'IT PRODUCTS / Medical Display', 'lg one:quick series' : 'Commercial Display / One:Quick Series', 'monitor': 'IT PRODUCTS / Monitor', 'one:quick series' : 'Commercial Display / One:Quick Series', 'heating' : 'HVAC, ESS / Heating', 'high brightness signage' : 'Commercial Display / High Brightness Signage', 'ventilation' : 'HVAC, ESS / Ventilation', 'teto ou cassete inverter' : 'HVAC, ESS / inverter', 'control' : 'HVAC, ESS / Control', 'multi inverter' : 'HVAC, ESS / inverter', 'ar condicionado residencial' : 'HVAC, ESS / AC', 'high brightness' : 'Commercial Display / High Brightness Signage', 'software solution' : 'Commercial Display / Software Solution', 'accessories' : 'Commercial Display / Accessories', 'special signage' : 'Commercial Display / Special Signage', 'hospital tv' : 'Commercial Display / Hospital TV', 'webos' : 'Commercial Display / WebOS', 'pc' : 'IT PRODUCTS', 'pro:centric' : 'Commercial Display / Pro:Centric', 'video wall' : 'Commercial Display / Video Wall Signage', 'projector' : 'IT PRODUCTS / Projector', 'all lg vrf systems' : 'HVAC, ESS / VRF', 'commercial display' : 'Commercial Display', 'residential air conditioner' :  'HVAC, ESS / AC', 'ur640' : 'Commercial Display', 'outros' : 'others', 'signage care solution' : 'Commercial Display / Signage Care Solution', 'multi v 5 air' : 'HVAC, ESS / ou', 'smart tv signage' : 'Commercial Display', 'technical support' : 'technical support', 'ur640s' : 'Commercial Display', 'cloud device' : 'IT PRODUCTS / Cloud Device', 'medical displays' : 'IT PRODUCTS / Medical Display', 'laptop' : 'IT PRODUCTS / Laptop', 'a thermodynamic water heater' : 'HVAC, ESS / Heating', 'uhd signage' : 'Commercial Display', 'monitor signage,monior/monitor tv' : 'Commercial Display', 'idb' : 'Commercial Display / Interactive Signage', 'virtual production' : 'virtual production', 'ogrzewanie (pompy ciepła)' : 'HVAC, ESS / Chiller', 'commercial tv' : 'Commercial Display / commercial tv', 'videowall_rmk' : 'Commercial Display / Video Wall Signage', 'digital signage' : 'Commercial Display / Digital Signage', '43us660h0sd.awz' : 'Commercial Display / Hotel TV', 'ledallinone' : 'Commercial Display / LED Signage', 'solar,ess' : 'solar & ess', 'services' : 'services', 'commercial tv,tv' : 'Commercial Display / commercial tv', 'monitor & pc' : 'IT PRODUCTS', 'aire acondicionado residencial' : 'HVAC, ESS / AC', 'onequick series' : 'Commercial Display / One:Quick Series', 'others' : 'others', 'led 顯示屏' : 'Commercial Display / LED Signage', 'education createboard' : 'Commercial Display / Interactive Signage', '28mq780' : 'IT PRODUCTS / Monitor', '32lq621cbsb.awz' : 'Commercial Display', 'other' : 'others', 'monior/monitor tv,tv' : 'Commercial Display', 'multi v water 5' : 'HVAC, ESS / ou', 'isıtma' : 'HVAC, ESS / Heating', 'tv' : 'Commercial Display', 'standalone' : 'Commercial Display', 'lainnya' : 'others', 'id' : 'id', 'calefacción' : 'HVAC, ESS / Heating', 'otros' : 'others', 'solar,system ac' : 'solar & HVAC, ESS / AC', 'sales inquiry' : '판매문의', 'fhd series' : 'Commercial Display', 'lg one:quick' : 'Commercial Display / One:Quick Series', 'washing machine,dryer' : 'Home Appliances / Washing Machine & Dryer', 'solar,aircare' : 'solar & Home Appliances / Aircare', '互動式顯示屏' : 'Commercial Display / Interactive signage', 'تكييف وتبريد' : 'HVAC, ESS / AC', 'tv signage' : 'Commercial Display', 'chiller,aircare' : 'HVAC, ESS / Chiller & Home Appliances / Aircare', 'system ac,aircare' : 'HVAC, ESS / AC & Home Appliances / Aircare', 'allinone_rmk' : 'all in one', 'led' : 'Commercial Display / LED Signage', 'multi v5 vrf' : 'HVAC, ESS / VRF', 'signage' : 'Commercial Display', 'điều hòa trung tâm multi' : 'HVAC, ESS', 'solar,chiller' : 'solar & HVAC, ESS / Chiller', 'vrf,multi-split,single-split' : 'HVAC, ESS / VRF & Multi-Split & Single-Split', 'monitor signage,commercial tv' : 'Commercial Display / commercial tv', 'leadallin' : 'Commercial Display / LED Signage', 'oled 顯示屏' : 'Commercial Display / OLED Signage', 'soğutucu' : 'HVAC, ESS / AC', 'robots' : 'Robot', 'htv' : 'Commercial Display', 'vrf,multi-split,single-split,chiller' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller', 'vrf,multi-split' : 'HVAC, ESS / VRF & Multi-Split', 'computer monitors' : 'IT PRODUCTS / Monitor', 'vrf,single-split' : 'HVAC, ESS / VRF & Single-Split', 'system ac,chiller' : 'HVAC, ESS / AC & Chiller', 'single split' : 'HVAC, ESS / Single-Split', 'מזגנים למקום מגורים' : 'HVAC, ESS / AC', '標準顯示屏' : 'Commercial Display', 'monitor signage,commercial tv,monior/monitor tv' : 'Commercial Display / commercial tv & Commercial Display', 'monitor signage,commercial tv,solar,ess,monior/monitor tv,pc,projector,robot,system ac,ems,rac,chill' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / Projector & HVAC, ESS / AC & HVAC, ESS / Chiller & ems & HVAC, ESS / AC & Robot', 'commercial tv,projector' : 'Commercial Display / commercial tv & IT PRODUCTS / Projector', 'corpouh5f' : 'Commercial Display', 'commercial tv,monior/monitor tv' : 'Commercial Display / commercial tv & Commercial Display', 'error' : 'error', 'تكييفات' : 'HVAC, ESS / AC', 'standard' : 'Commercial Display / Standard Signage', 'مبرد (تشيلر)' : 'HVAC, ESS / AC', 'نظام التدفق المتغيرvrf' : 'HVAC, ESS / VRF', 'vb.' : 'Home Appliances / Vacuum Cleaner', 'multi-split (plusieurs pièces)' : 'HVAC, ESS / Multi-Split', 'lg customer care program' : 'lg customer care program', 'aquecimento' : 'HVAC, ESS / Heating', 'laec015' : 'Commercial Display / LED Signage', 'commercial tv,audio/video' : 'Commercial Display / commercial tv & Commercial Display / AV', 'monitor signage,tv' : 'Commercial Display', 'medical monitors' : 'IT PRODUCTS / Medical Display', 'vrf,multi-split,single-split,chiller,heating' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller & Heating', 'ultra stretch signage' : 'Commercial Display', 'autre' : 'others', 'solar' : 'solar', 'corpuh5f-' : 'Commercial Display', 'monitor signage,pc' : 'Commercial Display & IT PRODUCTS', 'digital signage or commercial tvs' : 'Commercial Display / Digital Signage & Commercial Display / commercial tv', 'monior/monitor tv,pc' : 'Commercial Display & IT PRODUCTS / pc', '特別顯示屏' : 'IT PRODUCTS / Monitor', 'חימום' : 'HVAC, ESS / Heating', 'vrf,chiller' : 'HVAC, ESS / VRF & Chiller', 'ฯลฯ' : 'none', 'system air conditioner' : 'HVAC, ESS / AC', 'hoteleria_us670h' : 'Commercial Display', '55uq801c0sb.bwz' : 'Commercial Display', 'chiller,water care' : 'HVAC, ESS / Chiller & Home Appliances / Water Care', 'klimatyzacja multi-split' : 'HVAC, ESS / Multi-Split', '43uq751c0sf.bwz' : 'Commercial Display', 'scroll compressor' : 'Compressor, Motor / Scroll Compressor', '50uq801c0sb.bwz' : 'Commercial Display', 'vrf,multi-split,single-split,heating' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Heating', 'on or several reversible ac' : 'HVAC, ESS / AC', '55us660h0sd.bwz' : 'Commercial Display / Hotel TV', 'transparent oled' : 'Commercial Display / OLED Signage', 'monitor signage,commercial tv,monior/monitor tv,projector,tv' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / Projector', 'commercial tv,water care' : 'Commercial Display / commercial tv & Home Appliances / Water Care', 'one quick:flex' : 'Commercial Display / One:Quick Series', 'vrf,multi-split,single-split,chiller,etc.' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller', 'tv,commercial tv' : 'Commercial Display / commercial tv',
 'signage care solutions' : 'Commercial Display / Signage Care Solution', 'system ac,chiller,aircare' : 'HVAC, ESS & HVAC, ESS / Chiller & Home Appliances / Aircare', 'tv,refrigerator,washing machine' : 'Commercial Display & Home Appliances / Refrigerator & Washing Machine', 'climatiseur résidentiel' : 'HVAC, ESS', 'kimatyzacja vrf' : 'HVAC, ESS / VRF', 'เครื่องปรับอากาศเผื่อที่อยู่อาศัย' : 'HVAC, ESS & AC', 'monitor signage,audio/video' : 'Commercial Display & Commercial Display / AV', 'window facing display' : 'Commercial Display', 'lg home bliss air solution' : 'HVAC, ESS', 'ahu' : 'HVAC, ESS / VRF', 'split tunggal' : 'HVAC, ESS / Single-Split', 'ultra stretch series' : 'Commercial Display', 'system ac,rac' : 'HVAC, ESS / AC ', 'monitor signage,commercial tv,solar,ess,monior/monitor tv,pc' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS & solar & HVAC, ESS', 'bu50nst' : 'IT PRODUCTS / Projector', 'energy storage system' : 'HVAC, ESS', 'ac rumah' : 'HVAC, ESS / AC', 'commercial tv,robot' : 'Commercial Display / commercial tv & Robot', 'multi v' : 'HVAC, ESS / ou', 'tv,audio/video' : 'Commercial Display & Commercial Display / AV', 'solar,projector' : 'solar & IT PRODUCTS / Projector', 'multi split' : 'HVAC, ESS / Multi-Split', '酒店電視' : 'Commercial Display / Hotel TV', 'solar,monior/monitor tv' : 'solar & Commercial Display', 'system ac,refrigerator' : 'HVAC, ESS / AC & Home Appliances / Refrigerator', 'solar,refrigerator' : 'solar & Home Appliances / Refrigerator', 'monitor signage,system ac' : 'Commercial Display & HVAC, ESS / AC', 'system air conditioner,solar' : 'HVAC, ESS / AC & solar', 'solar,tv' : 'solar & Commercial Display', 'aio' : 'all in one', 'điều hòa cục bộ' : 'HVAC, ESS / AC', 'điều hòa gia dụng' : 'HVAC, ESS / AC', '55vm5e-a' : 'Commercial Display', 'laec15' : 'Commercial Display / LED Signage', '醫院電視' : 'Commercial Display / Hospital TV', 'ctv' : 'Commercial Display', '軟體' : 'IT PRODUCTS / software_', '55vm5j-h' : 'Commercial Display / Video Wall Signage', 'hospitality' : 'IT PRODUCTS / Medical Display', '49vl5f' : 'Commercial Display', 'monitorindustrial_rmk' : 'IT PRODUCTS / Monitor', 'lg magnit' : 'Commercial Display / LED Signage', 'retaildigital' : 'Commercial Display', 'one:quick' : 'Commercial Display / One:Quick Series', 'led cinema' : 'Commercial Display / LED Signage', '86uh5f' : 'Commercial Display', '55tc3d' : 'Commercial Display / Interactive signage', '43us660h (na)' : 'Commercial Display / Hotel TV', 'pro centric hotel' : 'Commercial Display / Hotel TV', 'comercial tv' : 'Commercial Display', 'video wall + aio' : 'Commercial Display / Video Wall Signage', '高亮度顯示屏' : 'Commercial Display / High Brightness Signage', 'led 70m2' : 'Commercial Display / LED Signage', 'single cac' : 'HVAC, ESS / AC', 'vrf - multi v s' : 'HVAC, ESS / VRF', 'procentric' : 'Commercial Display / Pro:Centric', 'monitor signage,monior/monitor tv,vacuum cleaner,tv,home beauty,commercial tv,pc,refrigerator,styler' : 'Commercial Display / commercial tv & Commercial Display & Home Appliances / Vacuum Cleaner & Home Appliances / Home Beauty & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances / Styler', 'refrigerator,built-in/cooking' : 'Home Appliances / Refrigerator & Built-in, Cooking', 'ems,audio/video' : 'ems & Commercial Display / AV', 'projector,ems,mobile,audio/video' : 'IT PRODUCTS / Projector & ems & mobile & Commercial Display / AV', 'smart tv' : 'Commercial Display', 'tv 55"' : 'Commercial Display', 'surgical monitor' : 'IT PRODUCTS / Medical Display', 'gsca046' : 'Commercial Display / LED Signage', 'gscd100' : 'Commercial Display / LED Signage', '32 pol' : 'Commercial Display', 'systèmes de débit à réfrigérant variable (drv)' : 'HVAC, ESS / AC', 'tv 43 pol' : 'Commercial Display', '43 pol' : 'Commercial Display', 'commercial tv,solar,ess,monior/monitor tv,pc,projector,robot,system ac,ems,rac,chiller,refrigerator,' : 'Commercial Display / commercial tv & HVAC, ESS / AC & solar & ess & IT PRODUCTS / Projector & Robot & Commercial Display & ems & HVAC, ESS / Chiller & Home Appliances / Refrigerator', 'tv,refrigerator' : 'Commercial Display & Home Appliances / Refrigerator', 'a definir' : 'none', 'điều hòa trung tâm chiller' : 'HVAC, ESS / Chiller', 'lsca039' : 'Commercial Display / LED Signage', 'essential series' : 'Commercial Display / LED Signage', 'parts' : 'Commercial Display / Accessories', 'vrf,heating' : 'HVAC, ESS / VRF & HVAC, ESS / Heating', 'điều hòa trung tâm vrf' : 'HVAC, ESS / VRF', 'inne' : 'others', '98uh5e' : 'Commercial Display', 'khác' : 'others', 'lg salang air solution for dream homes' : 'HVAC, ESS / AC', 'rac single cac' : 'HVAC, ESS / AC', 'آخر' : 'none', 'multi-split,single-split' : 'HVAC, ESS / Multi-Split & HVAC, ESS / Single-Split', 'vrf,multi-split,etc.' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split', 'vrf,multi-split,heating' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split & HVAC, ESS / Heating', 'vrf,single-split,chiller' : 'HVAC, ESS / VRF & HVAC, ESS / Single-Split & HVAC, ESS / Chiller', 'commercial tv,solar,ess,projector,system ac,tv,washing machine,home beauty,audio/video' : 'Commercial Display / commercial tv & Commercial Display & solar & ess & IT PRODUCTS / Projector & HVAC, ESS / AC & Home Appliances / Washing Machine & Home Appliances / Home Beauty & Commercial Display / AV', 'solar,robot' : 'solar & Robot', 'pantalla led outdoor' : 'Commercial Display / LED Signage', 'monitor signage,commercial tv,monior/monitor tv,pc,projector,tv,audio/video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & IT PRODUCTS / Projector & Commercial Display / AV', 'not specified' : 'none', 'lg led bloc' : 'Commercial Display / LED Signage', 'solar,energy storage system' : 'solar & ess', 'system ac,solar' : '냉HVAC, ESS / AC & solar', 'solar,system ac,water care' : 'solar & HVAC, ESS / AC & Home Appliances / Water Care', '55svh7f-a' : 'Commercial Display', 'tr3' : 'Commercial Display', 'uh' : 'Commercial Display', 'laec015-gn.awz' : 'Commercial Display / LED Signage', '49vl5g-m' : 'Commercial Display / Video Wall Signage', 'chiller,dryer' : 'HVAC, ESS / Chiller & Home Appliances / Dryer', 'monitor signage,monior/monitor tv,system ac,vacuum cleaner,tv,home beauty,commercial tv,mobile,audio' : 'Commercial Display / commercial tv & Commercial Display & HVAC, ESS / AC & Home Appliances / Vacuum Cleaner & Home Appliances / Home Beauty & mobile & Commercial Display / AV', 'videwall' : 'Commercial Display / Video Wall Signage', 'solar,vacuum cleaner' : 'solar & Home Appliances / Vacuum Cleaner', 'solar,monior/monitor tv,pc,tv,refrigerator,washing machine,dryer,home beauty' : 'solar & Commercial Display & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances & Home Appliances / Washing Machine & Home Appliances / Dryer & Home Appliances / Home Beauty', 'monitor signage,solar,robot,water care' : 'Commercial Display & solar & Robot & Home Appliances / Water Care', 'mobile,audio/video' : 'mobile & Commercial Display / AV',
 'system ac,refrigerator,washing machine,dryer' : 'HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Dryer', 'solar,system ac,aircare' : 'solar & HVAC, ESS / AC & Home Appliances / Aircare', 'projector,system ac,water care' : 'IT PRODUCTS / Projector & HVAC, ESS / AC & Home Appliances / Water Care', 'monior/monitor tv,tv,commercial tv,pc,refrigerator,solar,rac,washing machine,mobile,ess,audio/video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & Home Appliances / Refrigerator & solar & HVAC, ESS / AC & Home Appliances / Washing Machine & mobile & ess & Commercial Display / AV', 'ess,chiller' : 'ess & HVAC, ESS / Chiller', 'monitor signage,monior/monitor tv,pc,tv' : 'Commercial Display & IT PRODUCTS / pc', 'solar,water care' : 'solar & Home Appliances / Water Care', 'monitor signage,commercial tv,monior/monitor tv,tv' : 'Commercial Display / commercial tv & Commercial Display', 'monitor signage,commercial tv,audio/video' : 'Commercial Display / commercial tv & Commercial Display & Commercial Display / AV', 'solar,built-in/cooking' : 'solar & Home Appliances / Built-in, Cooking', 'monitor signage,monior/monitor tv,commercial tv' : 'Commercial Display / commercial tv & Commercial Display', 'robot,system ac' : 'Robot & HVAC, ESS / AC', 'medical- surgical' : 'IT PRODUCTS / Medical Display', 'radiology displays' : 'IT PRODUCTS / Medical Display', 'high inch 86 / 98 or 110' : 'Commercial Display', 'dryer,chiller' : 'Home Appliances / Dryer & HVAC, ESS / Chiller', 'solar,dryer' : 'solar & Home Appliances / Dryer', 'chiller/enfriadoras' : 'HVAC, ESS / Chiller & HVAC, ESS / AC', 'refrigerator' : 'Home Appliances / Refrigerator', 'led aio 136' : 'Commercial Display / LED Signage', '110 + video wall' : 'Commercial Display / Video Wall Signage', 'videowall signage' : 'Commercial Display / Video Wall Signage', 'one:quick flex' : 'Commercial Display / One:Quick Series', 'collaboration displays' : 'Commercial Display', 'meeting & screen sharedirect view leddirect view led' : 'Commercial Display / LED Signage', 'one quick works' : 'Commercial Display / One:Quick Series', 'pendingin' : 'HVAC, ESS / AC', 'lg paradise air solution' : 'HVAC, ESS', 'פיצול מרובה' : 'HVAC, ESS / Multi-Split', 'אחר' : 'others', 'vrf,multi-split,chiller' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split & HVAC, ESS / Chiller', 'system ac,solar,washing machine' : 'HVAC, ESS / AC & solar & Home Appliances / Washing Machine', 'solar,ess,ems' : 'solar & ess & ems', 'tv,mobile' : 'Commercial Display & mobile', 'aircare,water care' : 'Home Appliances / Aircare & Home Appliances / Water Care', 'monior/monitor tv,chiller' : 'Commercial Display & HVAC, ESS / Chiller', 'system ac,tv' : 'HVAC, ESS / AC & Commercial Display', 'monitor signage,commercial tv,solar,ess' : 'Commercial Display / commercial tv & Commercial Display & solar & ess', 'monior/monitor tv,system ac,tv,refrigerator,washing machine,dryer,built-in/cooking' : 'Commercial Display & HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Dryer & Home Appliances / Built-in, Cooking', 'monitor signage,solar' : 'Commercial Display & solar', 'pc,washing machine' : 'IT PRODUCTS / pc & Home Appliances / Washing Machine', 'monitor signage,solar,monior/monitor tv,pc,projector,robot,system ac,tv,refrigerator,washing machine' : 'Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & Robot & HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine', 'system ac,tv,refrigerator,washing machine,built-in/cooking,audio/video' : 'HVAC, ESS / AC & Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Built-in, Cooking & Commercial Display / AV', 'system air conditioner,energy storage system' : 'HVAC, ESS / AC & ess', '50us660h0sd.bwz' : 'Commercial Display / LED Signage', 'tv 60"' : 'Commercial Display', 'medical monitor' : 'IT PRODUCTS / Medical Display', 'aircare,mobile' : 'Home Appliances / Aircare & mobile', 'solar,ess,system ac' : 'solar & ess & HVAC, ESS / AC', 'tv,pc' : 'Commercial Display & IT PRODUCTS / pc', 'monitor signage,commercial tv,solar,pc,projector,system ac,ems,rac,chiller,refrigerator,washing mach' : 'Commercial Display / commercial tv & Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & HVAC, ESS / AC & ems & HVAC, ESS / Chiller & Home Appliances / Refrigerator & Home Appliances / Washing Machine', 'chiller,refrigerator' : 'HVAC, ESS / Chiller & Home Appliances / Refrigerator', 'monitor signage,commercial tv,solar,projector,robot,chiller,refrigerator,built-in/cooking,water care' : 'Commercial Display / commercial tv & Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & Robot & HVAC, ESS / Chiller & Home Appliances / Refrigerator & Home Appliances / Built-in, Cooking & Home Appliances / Water Care', 'commercial tv,solar' : 'Commercial Display / commercial tv & solar', 'monior/monitor tv,projector,audio/video' : 'Commercial Display & IT PRODUCTS / Projector & Commercial Display / AV', 'unitario' : 'none', 'refrigerator,chiller' : 'Home Appliances / Refrigerator & HVAC, ESS / Chiller', 'chiller,tv' : 'HVAC, ESS / Chiller & Commercial Display', 'projector,ems' : 'IT PRODUCTS / Projector & ems', 'cac' : 'HVAC, ESS', 'single package' : 'HVAC, ESS / Single-Split', 'monitor signage,mobile' : 'Commercial Display & mobile', 'robot,vacuum cleaner' : 'Robot & Home Appliances / Vacuum Cleaner', 'monitor signage,commercial tv,monior/monitor tv,audio/video' : 'Commercial Display / commercial tv & Commercial Display & Commercial Display / AV', 'aircare,built-in/cooking' : 'Home Appliances / Aircare & Home Appliances / Built-in, Cooking', '49uh / 49xf' : 'Commercial Display', '43uq751c0sb.bwz' : 'Commercial Display / Hotel TV', '32 / 43 pol' : 'Commercial Display', 'monior/monitor tv,audio/video' : 'Commercial Display & Commercial Display / AV', 'system ac,home beauty' : 'HVAC, ESS / AC & Home Appliances / Home Beauty', 'information display,monitor' : 'Commercial Display', 'taa lcd lfd displays' : 'Commercial Display', 'rac/cac' : 'HVAC, ESS / AC', 'حلول التدفئة' : 'HVAC, ESS / Heating', 'aio | one quick' : 'Commercial Display / One:Quick Series', 'monior/monitor tv,refrigerator' : 'Commercial Display & Home Appliances / Refrigerator', 'monior/monitor tv,system ac,tv,pc,refrigerator,water care,solar,washing machine,mobile,chiller,built' : 'Commercial Display & HVAC, ESS / AC & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances / Water Care & solar & Home Appliances / Washing Machine & mobile & HVAC, ESS / Chiller & Home Appliances / Built-in, Cooking', 'projector,audio/video' : 'IT PRODUCTS / Projector & Commercial Display / AV', 'monitor signage,commercial tv,monior/monitor tv,pc,tv,home beauty,audio/video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & Home Appliances / Home Beauty & Commercial Display / AV', '49vl5g-m.awzm' : 'Commercial Display / Video Wall Signage', 'pc,robot,system ac,chiller,tv,refrigerator,washing machine,vacuum cleaner,styler,dryer,mobile,audio/' : 'IT PRODUCTS / pc & Robot & HVAC, ESS / AC & HVAC, ESS / Chiller & Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Vacuum Cleaner & Home Appliances / Dryer & Home Appliances / Styler & mobile & HVAC, ESS / AC', 'refrigerator,washing machine,built-in/cooking' : 'Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Built-in, Cooking', 'monitor signage,monior/monitor tv,tv,audio/video' : 'Commercial Display & Commercial Display / AV', 'tv,refrigerator,washing machine,vacuum cleaner,audio/video' : 'Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Vacuum Cleaner & HVAC, ESS / AC', 'pc,tv' : 'IT PRODUCTS / pc & Commercial Display', 'system ac' : 'HVAC, ESS / AC', 'sac' : 'HVAC, ESS / AC', 'gscd046' : 'Commercial Display / LED Signage', '43uh5f-h.awzm' : 'Commercial Display', 'monior/monitor tv,refrigerator,audio/video' : 'Commercial Display & Home Appliances / Refrigerator & Commercial Display / AV'
 }

In [80]:
product_dict ={
'interactive digital board' : 'Commercial Display / Interactive Signage', 'vrf' : 'HVAC, ESS / VRF', 'multi-split' : 'HVAC, ESS / Multi-Split', 'video wall signage' : 'Commercial Display / Video Wall Signage', 'etc.' : 'etc.', 'led signage' : 'Commercial Display / LED Signage', 'interactive signage' : 'Commercial Display / Interactive Signage', 'single-split' : 'HVAC, ESS / Single-Split', 'rac' : 'HVAC, ESS / AC', 'oled signage' : 'Commercial Display / OLED Signage', 'hotel tv' : 'Commercial Display / Hotel TV', 'chiller' : 'HVAC, ESS / Chiller', 'standard signage' : 'Commercial Display / Standard Signage', 'medical display' : 'IT PRODUCTS / Medical Display', 'lg one:quick series' : 'Commercial Display / One:Quick Series', 'monitor': 'IT PRODUCTS / Monitor', 'one:quick series' : 'Commercial Display / One:Quick Series', 'heating' : 'HVAC, ESS / Heating', 'high brightness signage' : 'Commercial Display / High Brightness Signage', 'ventilation' : 'HVAC, ESS / Ventilation', 'teto ou cassete inverter' : 'HVAC, ESS / inverter', 'control' : 'HVAC, ESS / Control', 'multi inverter' : 'HVAC, ESS / inverter', 'ar condicionado residencial' : 'HVAC, ESS / AC', 'high brightness' : 'Commercial Display / High Brightness Signage', 'software solution' : 'Commercial Display / Software Solution', 'accessories' : 'Commercial Display / Accessories', 'special signage' : 'Commercial Display / Special Signage', 'hospital tv' : 'Commercial Display / Hospital TV', 'webos' : 'Commercial Display / WebOS', 'pc' : 'IT PRODUCTS', 'pro:centric' : 'Commercial Display / Pro:Centric', 'video wall' : 'Commercial Display / Video Wall Signage', 'projector' : 'IT PRODUCTS / Projector', 'all lg vrf systems' : 'HVAC, ESS / VRF', 'commercial display' : 'Commercial Display', 'residential air conditioner' :  'HVAC, ESS / AC', 'ur640' : 'Commercial Display', 'outros' : 'others', 'signage care solution' : 'Commercial Display / Signage Care Solution', 'multi v 5 air' : 'HVAC, ESS / ou', 'smart tv signage' : 'Commercial Display', 'technical support' : 'technical support', 'ur640s' : 'Commercial Display', 'cloud device' : 'IT PRODUCTS / Cloud Device', 'medical displays' : 'IT PRODUCTS / Medical Display', 'laptop' : 'IT PRODUCTS / Laptop', 'a thermodynamic water heater' : 'HVAC, ESS / Heating', 'uhd signage' : 'Commercial Display', 'monitor signage,monior / monitor tv' : 'Commercial Display', 'idb' : 'Commercial Display / Interactive Signage', 'virtual production' : 'virtual production', 'ogrzewanie (pompy ciepła)' : 'HVAC, ESS / Chiller', 'commercial tv' : 'Commercial Display / commercial tv', 'videowall_rmk' : 'Commercial Display / Video Wall Signage', 'digital signage' : 'Commercial Display / Digital Signage', '43us660h0sd.awz' : 'Commercial Display / Hotel TV', 'ledallinone' : 'Commercial Display / LED Signage', 'solar,ess' : 'solar & ess', 'services' : 'services', 'commercial tv,tv' : 'Commercial Display / commercial tv', 'monitor & pc' : 'IT PRODUCTS', 'aire acondicionado residencial' : 'HVAC, ESS / AC', 'onequick series' : 'Commercial Display / One:Quick Series', 'others' : 'others', 'led 顯示屏' : 'Commercial Display / LED Signage', 'education createboard' : 'Commercial Display / Interactive Signage', '28mq780' : 'IT PRODUCTS / Monitor', '32lq621cbsb.awz' : 'Commercial Display', 'other' : 'others', 'monior / monitor tv,tv' : 'Commercial Display', 'multi v water 5' : 'HVAC, ESS / ou', 'isıtma' : 'HVAC, ESS / Heating', 'tv' : 'Commercial Display', 'standalone' : 'Commercial Display', 'lainnya' : 'others', 'id' : 'id', 'calefacción' : 'HVAC, ESS / Heating', 'otros' : 'others', 'solar,system ac' : 'solar & HVAC, ESS / AC', 'sales inquiry' : '판매문의', 'fhd series' : 'Commercial Display', 'lg one:quick' : 'Commercial Display / One:Quick Series', 'washing machine,dryer' : 'Home Appliances / Washing Machine & Dryer', 'solar,aircare' : 'solar & Home Appliances / Aircare', '互動式顯示屏' : 'Commercial Display / Interactive signage', 'تكييف وتبريد' : 'HVAC, ESS / AC', 'tv signage' : 'Commercial Display', 'chiller,aircare' : 'HVAC, ESS / Chiller & Home Appliances / Aircare', 'system ac,aircare' : 'HVAC, ESS / AC & Home Appliances / Aircare', 'allinone_rmk' : 'all in one', 'led' : 'Commercial Display / LED Signage', 'multi v5 vrf' : 'HVAC, ESS / VRF', 'signage' : 'Commercial Display', 'điều hòa trung tâm multi' : 'HVAC, ESS', 'solar,chiller' : 'solar & HVAC, ESS / Chiller', 'vrf,multi-split,single-split' : 'HVAC, ESS / VRF & Multi-Split & Single-Split', 'monitor signage,commercial tv' : 'Commercial Display / commercial tv', 'leadallin' : 'Commercial Display / LED Signage', 'oled 顯示屏' : 'Commercial Display / OLED Signage', 'soğutucu' : 'HVAC, ESS / AC', 'robots' : 'Robot', 'htv' : 'Commercial Display', 'vrf,multi-split,single-split,chiller' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller', 'vrf,multi-split' : 'HVAC, ESS / VRF & Multi-Split', 'computer monitors' : 'IT PRODUCTS / Monitor', 'vrf,single-split' : 'HVAC, ESS / VRF & Single-Split', 'system ac,chiller' : 'HVAC, ESS / AC & Chiller', 'single split' : 'HVAC, ESS / Single-Split', 'מזגנים למקום מגורים' : 'HVAC, ESS / AC', '標準顯示屏' : 'Commercial Display', 'monitor signage,commercial tv,monior / monitor tv' : 'Commercial Display / commercial tv & Commercial Display', 'monitor signage,commercial tv,solar,ess,monior / monitor tv,pc,projector,robot,system ac,ems,rac,chill' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / Projector & HVAC, ESS / AC & HVAC, ESS / Chiller & ems & HVAC, ESS / AC & Robot', 'commercial tv,projector' : 'Commercial Display / commercial tv & IT PRODUCTS / Projector', 'corpouh5f' : 'Commercial Display', 'commercial tv,monior / monitor tv' : 'Commercial Display / commercial tv & Commercial Display', 'error' : 'error', 'تكييفات' : 'HVAC, ESS / AC', 'standard' : 'Commercial Display / Standard Signage', 'مبرد (تشيلر)' : 'HVAC, ESS / AC', 'نظام التدفق المتغيرvrf' : 'HVAC, ESS / VRF', 'vb.' : 'Home Appliances / Vacuum Cleaner', 'multi-split (plusieurs pièces)' : 'HVAC, ESS / Multi-Split', 'lg customer care program' : 'lg customer care program', 'aquecimento' : 'HVAC, ESS / Heating', 'laec015' : 'Commercial Display / LED Signage', 'commercial tv,audio / video' : 'Commercial Display / commercial tv & Commercial Display / AV', 'monitor signage,tv' : 'Commercial Display', 'medical monitors' : 'IT PRODUCTS / Medical Display', 'vrf,multi-split,single-split,chiller,heating' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller & Heating', 'ultra stretch signage' : 'Commercial Display', 'autre' : 'others', 'solar' : 'solar', 'corpuh5f-' : 'Commercial Display', 'monitor signage,pc' : 'Commercial Display & IT PRODUCTS', 'digital signage or commercial tvs' : 'Commercial Display / Digital Signage & Commercial Display / commercial tv', 'monior / monitor tv,pc' : 'Commercial Display & IT PRODUCTS / pc', '特別顯示屏' : 'IT PRODUCTS / Monitor', 'חימום' : 'HVAC, ESS / Heating', 'vrf,chiller' : 'HVAC, ESS / VRF & Chiller', 'ฯลฯ' : 'none', 'system air conditioner' : 'HVAC, ESS / AC', 'hoteleria_us670h' : 'Commercial Display', '55uq801c0sb.bwz' : 'Commercial Display', 'chiller,water care' : 'HVAC, ESS / Chiller & Home Appliances / Water Care', 'klimatyzacja multi-split' : 'HVAC, ESS / Multi-Split', '43uq751c0sf.bwz' : 'Commercial Display', 'scroll compressor' : 'Compressor, Motor / Scroll Compressor', '50uq801c0sb.bwz' : 'Commercial Display', 'vrf,multi-split,single-split,heating' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Heating', 'on or several reversible ac' : 'HVAC, ESS / AC', '55us660h0sd.bwz' : 'Commercial Display / Hotel TV', 'transparent oled' : 'Commercial Display / OLED Signage', 'monitor signage,commercial tv,monior / monitor tv,projector,tv' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / Projector', 'commercial tv,water care' : 'Commercial Display / commercial tv & Home Appliances / Water Care', 'one quick:flex' : 'Commercial Display / One:Quick Series', 'vrf,multi-split,single-split,chiller,etc.' : 'HVAC, ESS / VRF & Multi-Split & Single-Split & Chiller', 'tv,commercial tv' : 'Commercial Display / commercial tv',
 'signage care solutions' : 'Commercial Display / Signage Care Solution', 'system ac,chiller,aircare' : 'HVAC, ESS & HVAC, ESS / Chiller & Home Appliances / Aircare', 'tv,refrigerator,washing machine' : 'Commercial Display & Home Appliances / Refrigerator & Washing Machine', 'climatiseur résidentiel' : 'HVAC, ESS', 'kimatyzacja vrf' : 'HVAC, ESS / VRF', 'เครื่องปรับอากาศเผื่อที่อยู่อาศัย' : 'HVAC, ESS & AC', 'monitor signage,audio / video' : 'Commercial Display & Commercial Display / AV', 'window facing display' : 'Commercial Display', 'lg home bliss air solution' : 'HVAC, ESS', 'ahu' : 'HVAC, ESS / VRF', 'split tunggal' : 'HVAC, ESS / Single-Split', 'ultra stretch series' : 'Commercial Display', 'system ac,rac' : 'HVAC, ESS / AC ', 'monitor signage,commercial tv,solar,ess,monior / monitor tv,pc' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS & solar & HVAC, ESS', 'bu50nst' : 'IT PRODUCTS / Projector', 'energy storage system' : 'HVAC, ESS', 'ac rumah' : 'HVAC, ESS / AC', 'commercial tv,robot' : 'Commercial Display / commercial tv & Robot', 'multi v' : 'HVAC, ESS / ou', 'tv,audio / video' : 'Commercial Display & Commercial Display / AV', 'solar,projector' : 'solar & IT PRODUCTS / Projector', 'multi split' : 'HVAC, ESS / Multi-Split', '酒店電視' : 'Commercial Display / Hotel TV', 'solar,monior / monitor tv' : 'solar & Commercial Display', 'system ac,refrigerator' : 'HVAC, ESS / AC & Home Appliances / Refrigerator', 'solar,refrigerator' : 'solar & Home Appliances / Refrigerator', 'monitor signage,system ac' : 'Commercial Display & HVAC, ESS / AC', 'system air conditioner,solar' : 'HVAC, ESS / AC & solar', 'solar,tv' : 'solar & Commercial Display', 'aio' : 'all in one', 'điều hòa cục bộ' : 'HVAC, ESS / AC', 'điều hòa gia dụng' : 'HVAC, ESS / AC', '55vm5e-a' : 'Commercial Display', 'laec15' : 'Commercial Display / LED Signage', '醫院電視' : 'Commercial Display / Hospital TV', 'ctv' : 'Commercial Display', '軟體' : 'IT PRODUCTS / software_', '55vm5j-h' : 'Commercial Display / Video Wall Signage', 'hospitality' : 'IT PRODUCTS / Medical Display', '49vl5f' : 'Commercial Display', 'monitorindustrial_rmk' : 'IT PRODUCTS / Monitor', 'lg magnit' : 'Commercial Display / LED Signage', 'retaildigital' : 'Commercial Display', 'one:quick' : 'Commercial Display / One:Quick Series', 'led cinema' : 'Commercial Display / LED Signage', '86uh5f' : 'Commercial Display', '55tc3d' : 'Commercial Display / Interactive signage', '43us660h (na)' : 'Commercial Display / Hotel TV', 'pro centric hotel' : 'Commercial Display / Hotel TV', 'comercial tv' : 'Commercial Display', 'video wall + aio' : 'Commercial Display / Video Wall Signage', '高亮度顯示屏' : 'Commercial Display / High Brightness Signage', 'led 70m2' : 'Commercial Display / LED Signage', 'single cac' : 'HVAC, ESS / AC', 'vrf - multi v s' : 'HVAC, ESS / VRF', 'procentric' : 'Commercial Display / Pro:Centric', 'monitor signage,monior / monitor tv,vacuum cleaner,tv,home beauty,commercial tv,pc,refrigerator,styler' : 'Commercial Display / commercial tv & Commercial Display & Home Appliances / Vacuum Cleaner & Home Appliances / Home Beauty & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances / Styler', 'refrigerator,built-in / cooking' : 'Home Appliances / Refrigerator & Built-in, Cooking', 'ems,audio / video' : 'ems & Commercial Display / AV', 'projector,ems,mobile,audio / video' : 'IT PRODUCTS / Projector & ems & mobile & Commercial Display / AV', 'smart tv' : 'Commercial Display', 'tv 55"' : 'Commercial Display', 'surgical monitor' : 'IT PRODUCTS / Medical Display', 'gsca046' : 'Commercial Display / LED Signage', 'gscd100' : 'Commercial Display / LED Signage', '32 pol' : 'Commercial Display', 'systèmes de débit à réfrigérant variable (drv)' : 'HVAC, ESS / AC', 'tv 43 pol' : 'Commercial Display', '43 pol' : 'Commercial Display', 'commercial tv,solar,ess,monior / monitor tv,pc,projector,robot,system ac,ems,rac,chiller,refrigerator,' : 'Commercial Display / commercial tv & HVAC, ESS / AC & solar & ess & IT PRODUCTS / Projector & Robot & Commercial Display & ems & HVAC, ESS / Chiller & Home Appliances / Refrigerator', 'tv,refrigerator' : 'Commercial Display & Home Appliances / Refrigerator', 'a definir' : 'none', 'điều hòa trung tâm chiller' : 'HVAC, ESS / Chiller', 'lsca039' : 'Commercial Display / LED Signage', 'essential series' : 'Commercial Display / LED Signage', 'parts' : 'Commercial Display / Accessories', 'vrf,heating' : 'HVAC, ESS / VRF & HVAC, ESS / Heating', 'điều hòa trung tâm vrf' : 'HVAC, ESS / VRF', 'inne' : 'others', '98uh5e' : 'Commercial Display', 'khác' : 'others', 'lg salang air solution for dream homes' : 'HVAC, ESS / AC', 'rac single cac' : 'HVAC, ESS / AC', 'آخر' : 'none', 'multi-split,single-split' : 'HVAC, ESS / Multi-Split & HVAC, ESS / Single-Split', 'vrf,multi-split,etc.' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split', 'vrf,multi-split,heating' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split & HVAC, ESS / Heating', 'vrf,single-split,chiller' : 'HVAC, ESS / VRF & HVAC, ESS / Single-Split & HVAC, ESS / Chiller', 'commercial tv,solar,ess,projector,system ac,tv,washing machine,home beauty,audio / video' : 'Commercial Display / commercial tv & Commercial Display & solar & ess & IT PRODUCTS / Projector & HVAC, ESS / AC & Home Appliances / Washing Machine & Home Appliances / Home Beauty & Commercial Display / AV', 'solar,robot' : 'solar & Robot', 'pantalla led outdoor' : 'Commercial Display / LED Signage', 'monitor signage,commercial tv,monior / monitor tv,pc,projector,tv,audio / video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & IT PRODUCTS / Projector & Commercial Display / AV', 'not specified' : 'none', 'lg led bloc' : 'Commercial Display / LED Signage', 'solar,energy storage system' : 'solar & ess', 'system ac,solar' : '냉HVAC, ESS / AC & solar', 'solar,system ac,water care' : 'solar & HVAC, ESS / AC & Home Appliances / Water Care', '55svh7f-a' : 'Commercial Display', 'tr3' : 'Commercial Display', 'uh' : 'Commercial Display', 'laec015-gn.awz' : 'Commercial Display / LED Signage', '49vl5g-m' : 'Commercial Display / Video Wall Signage', 'chiller,dryer' : 'HVAC, ESS / Chiller & Home Appliances / Dryer', 'monitor signage,monior / monitor tv,system ac,vacuum cleaner,tv,home beauty,commercial tv,mobile,audio' : 'Commercial Display / commercial tv & Commercial Display & HVAC, ESS / AC & Home Appliances / Vacuum Cleaner & Home Appliances / Home Beauty & mobile & Commercial Display / AV', 'videwall' : 'Commercial Display / Video Wall Signage', 'solar,vacuum cleaner' : 'solar & Home Appliances / Vacuum Cleaner', 'solar,monior / monitor tv,pc,tv,refrigerator,washing machine,dryer,home beauty' : 'solar & Commercial Display & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances & Home Appliances / Washing Machine & Home Appliances / Dryer & Home Appliances / Home Beauty', 'monitor signage,solar,robot,water care' : 'Commercial Display & solar & Robot & Home Appliances / Water Care', 'mobile,audio / video' : 'mobile & Commercial Display / AV',
 'system ac,refrigerator,washing machine,dryer' : 'HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Dryer', 'solar,system ac,aircare' : 'solar & HVAC, ESS / AC & Home Appliances / Aircare', 'projector,system ac,water care' : 'IT PRODUCTS / Projector & HVAC, ESS / AC & Home Appliances / Water Care', 'monior / monitor tv,tv,commercial tv,pc,refrigerator,solar,rac,washing machine,mobile,ess,audio / video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & Home Appliances / Refrigerator & solar & HVAC, ESS / AC & Home Appliances / Washing Machine & mobile & ess & Commercial Display / AV', 'ess,chiller' : 'ess & HVAC, ESS / Chiller', 'monitor signage,monior / monitor tv,pc,tv' : 'Commercial Display & IT PRODUCTS / pc', 'solar,water care' : 'solar & Home Appliances / Water Care', 'monitor signage,commercial tv,monior / monitor tv,tv' : 'Commercial Display / commercial tv & Commercial Display', 'monitor signage,commercial tv,audio / video' : 'Commercial Display / commercial tv & Commercial Display & Commercial Display / AV', 'solar,built-in / cooking' : 'solar & Home Appliances / Built-in, Cooking', 'monitor signage,monior / monitor tv,commercial tv' : 'Commercial Display / commercial tv & Commercial Display', 'robot,system ac' : 'Robot & HVAC, ESS / AC', 'medical- surgical' : 'IT PRODUCTS / Medical Display', 'radiology displays' : 'IT PRODUCTS / Medical Display', 'high inch 86  /  98 or 110' : 'Commercial Display', 'dryer,chiller' : 'Home Appliances / Dryer & HVAC, ESS / Chiller', 'solar,dryer' : 'solar & Home Appliances / Dryer', 'chiller / enfriadoras' : 'HVAC, ESS / Chiller & HVAC, ESS / AC', 'refrigerator' : 'Home Appliances / Refrigerator', 'led aio 136' : 'Commercial Display / LED Signage', '110 + video wall' : 'Commercial Display / Video Wall Signage', 'videowall signage' : 'Commercial Display / Video Wall Signage', 'one:quick flex' : 'Commercial Display / One:Quick Series', 'collaboration displays' : 'Commercial Display', 'meeting & screen sharedirect view leddirect view led' : 'Commercial Display / LED Signage', 'one quick works' : 'Commercial Display / One:Quick Series', 'pendingin' : 'HVAC, ESS / AC', 'lg paradise air solution' : 'HVAC, ESS', 'פיצול מרובה' : 'HVAC, ESS / Multi-Split', 'אחר' : 'others', 'vrf,multi-split,chiller' : 'HVAC, ESS / VRF & HVAC, ESS / Multi-Split & HVAC, ESS / Chiller', 'system ac,solar,washing machine' : 'HVAC, ESS / AC & solar & Home Appliances / Washing Machine', 'solar,ess,ems' : 'solar & ess & ems', 'tv,mobile' : 'Commercial Display & mobile', 'aircare,water care' : 'Home Appliances / Aircare & Home Appliances / Water Care', 'monior / monitor tv,chiller' : 'Commercial Display & HVAC, ESS / Chiller', 'system ac,tv' : 'HVAC, ESS / AC & Commercial Display', 'monitor signage,commercial tv,solar,ess' : 'Commercial Display / commercial tv & Commercial Display & solar & ess', 'monior / monitor tv,system ac,tv,refrigerator,washing machine,dryer,built-in / cooking' : 'Commercial Display & HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Dryer & Home Appliances / Built-in, Cooking', 'monitor signage,solar' : 'Commercial Display & solar', 'pc,washing machine' : 'IT PRODUCTS / pc & Home Appliances / Washing Machine', 'monitor signage,solar,monior / monitor tv,pc,projector,robot,system ac,tv,refrigerator,washing machine' : 'Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & Robot & HVAC, ESS / AC & Home Appliances / Refrigerator & Home Appliances / Washing Machine', 'system ac,tv,refrigerator,washing machine,built-in / cooking,audio / video' : 'HVAC, ESS / AC & Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Built-in, Cooking & Commercial Display / AV', 'system air conditioner,energy storage system' : 'HVAC, ESS / AC & ess', '50us660h0sd.bwz' : 'Commercial Display / LED Signage', 'tv 60"' : 'Commercial Display', 'medical monitor' : 'IT PRODUCTS / Medical Display', 'aircare,mobile' : 'Home Appliances / Aircare & mobile', 'solar,ess,system ac' : 'solar & ess & HVAC, ESS / AC', 'tv,pc' : 'Commercial Display & IT PRODUCTS / pc', 'monitor signage,commercial tv,solar,pc,projector,system ac,ems,rac,chiller,refrigerator,washing mach' : 'Commercial Display / commercial tv & Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & HVAC, ESS / AC & ems & HVAC, ESS / Chiller & Home Appliances / Refrigerator & Home Appliances / Washing Machine', 'chiller,refrigerator' : 'HVAC, ESS / Chiller & Home Appliances / Refrigerator', 'monitor signage,commercial tv,solar,projector,robot,chiller,refrigerator,built-in / cooking,water care' : 'Commercial Display / commercial tv & Commercial Display & solar & IT PRODUCTS / pc & IT PRODUCTS / Projector & Robot & HVAC, ESS / Chiller & Home Appliances / Refrigerator & Home Appliances / Built-in, Cooking & Home Appliances / Water Care', 'commercial tv,solar' : 'Commercial Display / commercial tv & solar', 'monior / monitor tv,projector,audio / video' : 'Commercial Display & IT PRODUCTS / Projector & Commercial Display / AV', 'unitario' : 'none', 'refrigerator,chiller' : 'Home Appliances / Refrigerator & HVAC, ESS / Chiller', 'chiller,tv' : 'HVAC, ESS / Chiller & Commercial Display', 'projector,ems' : 'IT PRODUCTS / Projector & ems', 'cac' : 'HVAC, ESS', 'single package' : 'HVAC, ESS / Single-Split', 'monitor signage,mobile' : 'Commercial Display & mobile', 'robot,vacuum cleaner' : 'Robot & Home Appliances / Vacuum Cleaner', 'monitor signage,commercial tv,monior / monitor tv,audio / video' : 'Commercial Display / commercial tv & Commercial Display & Commercial Display / AV', 'aircare,built-in / cooking' : 'Home Appliances / Aircare & Home Appliances / Built-in, Cooking', '49uh  /  49xf' : 'Commercial Display', '43uq751c0sb.bwz' : 'Commercial Display / Hotel TV', '32  /  43 pol' : 'Commercial Display', 'monior / monitor tv,audio / video' : 'Commercial Display & Commercial Display / AV', 'system ac,home beauty' : 'HVAC, ESS / AC & Home Appliances / Home Beauty', 'information display,monitor' : 'Commercial Display', 'taa lcd lfd displays' : 'Commercial Display', 'rac / cac' : 'HVAC, ESS / AC', 'حلول التدفئة' : 'HVAC, ESS / Heating', 'aio | one quick' : 'Commercial Display / One:Quick Series', 'monior / monitor tv,refrigerator' : 'Commercial Display & Home Appliances / Refrigerator', 'monior / monitor tv,system ac,tv,pc,refrigerator,water care,solar,washing machine,mobile,chiller,built' : 'Commercial Display & HVAC, ESS / AC & IT PRODUCTS / pc & Home Appliances / Refrigerator & Home Appliances / Water Care & solar & Home Appliances / Washing Machine & mobile & HVAC, ESS / Chiller & Home Appliances / Built-in, Cooking', 'projector,audio / video' : 'IT PRODUCTS / Projector & Commercial Display / AV', 'monitor signage,commercial tv,monior / monitor tv,pc,tv,home beauty,audio / video' : 'Commercial Display / commercial tv & Commercial Display & IT PRODUCTS / pc & Home Appliances / Home Beauty & Commercial Display / AV', '49vl5g-m.awzm' : 'Commercial Display / Video Wall Signage', 'pc,robot,system ac,chiller,tv,refrigerator,washing machine,vacuum cleaner,styler,dryer,mobile,audio / ' : 'IT PRODUCTS / pc & Robot & HVAC, ESS / AC & HVAC, ESS / Chiller & Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Vacuum Cleaner & Home Appliances / Dryer & Home Appliances / Styler & mobile & HVAC, ESS / AC', 'refrigerator,washing machine,built-in / cooking' : 'Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Built-in, Cooking', 'monitor signage,monior / monitor tv,tv,audio / video' : 'Commercial Display & Commercial Display / AV', 'tv,refrigerator,washing machine,vacuum cleaner,audio / video' : 'Commercial Display & Home Appliances / Refrigerator & Home Appliances / Washing Machine & Home Appliances / Vacuum Cleaner & HVAC, ESS / AC', 'pc,tv' : 'IT PRODUCTS / pc & Commercial Display', 'system ac' : 'HVAC, ESS / AC', 'sac' : 'HVAC, ESS / AC', 'gscd046' : 'Commercial Display / LED Signage', '43uh5f-h.awzm' : 'Commercial Display', 'monior / monitor tv,refrigerator,audio / video' : 'Commercial Display & Home Appliances / Refrigerator & Commercial Display / AV'
 }

In [81]:
df_train['product_category'] = df_train['product_category'].map(product_dict)
df_test['product_category'] = df_test['product_category'].map(product_dict)

In [82]:
df_train['product_category'].isnull().sum()

17255

### ***modelname 존재 -> category null 값 / 결측치 대체***

In [83]:
product_dict ={
    'LG SuperSign CMS' : 'Commercial Display / Software Solution',    '55CT5WJ,43HT3WJ' : 'Commercial Display / Interactive signage',    '55CT5WJ,43HT3WJ,SC-00DA' : 'Commercial Display / Interactive signage & Commercial Display / One:Quick Series',    'SC-00DA' : 'Commercial Display / One:Quick Series',    '43HT3WJ' : 'Commercial Display / Interactive signage',    '27BN85U' : 'IT PRODUCTS / Monitor',    'CL600' : 'IT PRODUCTS / Cloud Device',    '55CT5WJ' : 'Commercial Display / Interactive signage',    '32BP95E' : 'IT PRODUCTS / Monitor',    '31HN713D' : 'IT PRODUCTS / Medical Display',    '21HQ513D' : 'IT PRODUCTS / Medical Display',    'UltraFine Ergo(32UN880)' : 'IT PRODUCTS / Monitor',    'Ergo Dual(27QP88D)' : 'IT PRODUCTS / Monitor',    '34WN780, 28MQ780' : 'IT PRODUCTS / Monitor',    '32UN880' : 'IT PRODUCTS / Monitor',    '38CL950' : 'IT PRODUCTS / Cloud Device',    '65EP5G OLED Pro' : 'Commercial Display / OLED Signage',    '32HL512D' : 'IT PRODUCTS / Medical Display',    '19HK312C' : 'IT PRODUCTS / Medical Display',    '38CL950P' : 'IT PRODUCTS / Cloud Device',    'BU60' : 'IT PRODUCTS / Projector',    'AI/Machine Learning | Antennas, Transmitters and Towers | Audience Measurement | Cameras and Lenses' : '',    'UltraWide Ergo(34WN780)' : 'IT PRODUCTS / Monitor',    '28MQ780' : 'IT PRODUCTS / Monitor',    'DualUp(28MQ780)' : 'IT PRODUCTS / Monitor',    '34WN780' : 'IT PRODUCTS / Monitor',    '27HJ713C' : 'IT PRODUCTS / Medical Display',    'BU50N' : 'IT PRODUCTS / Projector',    '17Z90P' : 'IT PRODUCTS / Laptop',    '24CN670N-6N' : 'IT PRODUCTS / Cloud Device'}

In [84]:
product_dict_key = product_dict.keys()
product_dict_value = product_dict.values()

for i, val in tqdm(enumerate(product_dict_key)):
    df_train.loc[df_train['product_modelname'] == val, 'product_category'] = product_dict[val]
    df_test.loc[df_test['product_modelname'] == val, 'product_category'] = product_dict[val]

30it [00:00, 610.09it/s]


### ***subcategory 존재 -> category null 값 / 결측치 대체***

In [85]:
product_dict ={
    'All Monitors' : 'IT PRODUCTS / Monitor',    'Diagnostic Monitors' : 'IT PRODUCTS / Medical Display',    'Thin Clients' : 'IT PRODUCTS',    'Digital X-ray Detectors' : 'IT PRODUCTS / Medical Display',    'Zero Clients' : 'IT PRODUCTS / Cloud Device',    'All Medical Display' : 'IT PRODUCTS / Medical Display',    'UHD 4K Monitors' : 'IT PRODUCTS / Monitor',    'Clinical Review Monitors' : 'IT PRODUCTS / Medical Display',    'Surgical Monitors' : 'IT PRODUCTS / Medical Display',    '其他' : 'others',    'All Monitors & PCs' : 'IT PRODUCTS / Monitor',    'Laptops' : 'IT PRODUCTS / Laptop',    'IPS Monitors' : 'IT PRODUCTS / Monitor',    'จอภาพสำหรับการตรวจสอบทางคลินิก' : 'IT PRODUCTS / Medical Display',    'All Medical Displays' : 'IT PRODUCTS / Medical Display',    'ProBeam' : 'IT PRODUCTS / Projector',    'Other' : 'others',    'All Cloud Devices' : 'IT PRODUCTS / Cloud Device',    'Digital Signage or Commercial TVs' : 'Commercial Display / Digital Signage',    'Category' : 'Category',    'All Projectors' : 'IT PRODUCTS / Projector',    'Monitors with USB-C connection' : 'IT PRODUCTS / Monitor',    'Monitors for graphics applications & video editing' : 'IT PRODUCTS / Monitor',    'Channel Partner done FTS Details in System' : 'Channel Partner done FTS Details in System',    'Others' : 'others',    'Ergonomic monitors' : 'IT PRODUCTS / Monitor',    'UltraWide™ Monitors' : 'IT PRODUCTS / Monitor',    'Todo Medical Display' : 'IT PRODUCTS / Medical Display',    'TV Monitors' : 'IT PRODUCTS / Monitor',    'จอภาพเพื่อการวินิจฉัย' : 'Medical Display'}

In [86]:
product_dict_key = product_dict.keys()
product_dict_value = product_dict.values()

for i, val in tqdm(enumerate(product_dict_key)):
    df_train.loc[df_train['product_subcategory'] == val, 'product_category'] = product_dict[val]
    df_test.loc[df_test['product_subcategory'] == val, 'product_category'] = product_dict[val]

30it [00:00, 617.56it/s]


결측치에 business_unit 정보 포함시켜주기

In [87]:
df_train['product_category'] = df_train['product_category'].fillna('Unknown')
df_test['product_category'] = df_test['product_category'].fillna('Unknown')

In [88]:
df_train.loc[(df_train['product_category']=='Unknown')& (df_train['business_unit']=='ID'),'product_category']='ID_unknown'
df_train.loc[(df_train['product_category']=='Unknown')& (df_train['business_unit']=='IT'),'product_category']='IT_unknown'
df_train.loc[(df_train['product_category']=='Unknown')& (df_train['business_unit']=='AS'),'product_category']='AS_unknown'

df_test.loc[(df_test['product_category']=='Unknown')& (df_test['business_unit']=='ID'),'product_category']='ID_unknown'
df_test.loc[(df_test['product_category']=='Unknown')& (df_test['business_unit']=='IT'),'product_category']='IT_unknown'
df_test.loc[(df_test['product_category']=='Unknown')& (df_test['business_unit']=='AS'),'product_category']='AS_unknown'

***
***

## ***16. product_subcategory***
요청 제품 하위 카테고리

In [89]:
df_train['product_subcategory'].isnull().sum()

46740

In [90]:
df_train['product_subcategory'].nunique()

330

In [91]:
df_train['product_subcategory'].fillna(df_train['product_category'], inplace=True)
df_test['product_subcategory'].fillna(df_test['product_category'], inplace=True)

***
***

## ***17. product_modelname***
요청 제품 모델명

In [92]:
df_train['product_modelname'].isnull().sum()

46715

In [93]:
df_train['product_modelname'].nunique()

665

In [94]:
df_train['product_modelname'].fillna(df_train['product_subcategory'], inplace=True)
df_test['product_modelname'].fillna(df_test['product_subcategory'], inplace=True)

***
***

## ***18. customer_country.1***
담당 자사 법인명 기반의 지역 정보(대륙)

In [95]:
tmp_df = df_raw[~df_raw['customer_country'].isnull()]

In [96]:
(tmp_df['customer_country'] == tmp_df['customer_country.1']).all()

True

아예 같은 컬럼이기 때문에 컬럼 삭제

In [97]:
df_train = df_train.drop(columns='customer_country.1')
df_test = df_test.drop(columns='customer_country.1')

***
***

## ***19. customer_position***
고객의 회사 직책

In [98]:
df_train['customer_position'].isnull().sum()

0

In [99]:
df_train['customer_position'].nunique()

117

In [100]:
# manager
df_train.loc[df_train['customer_position'].str.contains("manage", case=False, na=False), 'customer_position'] = 'manager'
df_test.loc[df_test['customer_position'].str.contains("manage", case=False, na=False), 'customer_position'] = 'manager'

# ceo/founder
df_train.loc[df_train['customer_position'].str.contains("ceo", case=False, na=False), 'customer_position'] = 'ceo/founder'
df_test.loc[df_test['customer_position'].str.contains("ceo", case=False, na=False), 'customer_position'] = 'ceo/founder'
df_train.loc[df_train['customer_position'].str.contains("founder", case=False, na=False), 'customer_position'] = 'ceo/founder'
df_test.loc[df_test['customer_position'].str.contains("founder", case=False, na=False), 'customer_position'] = 'ceo/founder'

# other
df_train.loc[df_train['customer_position'].str.contains("other", case=False, na=False), 'customer_position'] = 'other'
df_test.loc[df_test['customer_position'].str.contains("other", case=False, na=False), 'customer_position'] = 'other'

# director
df_train.loc[df_train['customer_position'].str.contains("director", case=False, na=False), 'customer_position'] = 'director'
df_test.loc[df_test['customer_position'].str.contains("director", case=False, na=False), 'customer_position'] = 'director'

# associate/analyst
df_train.loc[df_train['customer_position'].str.contains("associate", case=False, na=False), 'customer_position'] = 'associate/analyst'
df_test.loc[df_test['customer_position'].str.contains("associate", case=False, na=False), 'customer_position'] = 'associate/analyst'
df_train.loc[df_train['customer_position'].str.contains("analyst", case=False, na=False), 'customer_position'] = 'associate/analyst'
df_test.loc[df_test['customer_position'].str.contains("analyst", case=False, na=False), 'customer_position'] = 'associate/analyst'

# partner
df_train.loc[df_train['customer_position'].str.contains("partner", case=False, na=False), 'customer_position'] = 'partner'
df_test.loc[df_test['customer_position'].str.contains("partner", case=False, na=False), 'customer_position'] = 'partner'

# entrylevel
df_train.loc[df_train['customer_position'].str.contains("entrylevel", case=False, na=False), 'customer_position'] = 'entry level'
df_test.loc[df_test['customer_position'].str.contains("entrylevel", case=False, na=False), 'customer_position'] = 'entry level'

# trainee
df_train.loc[df_train['customer_position'].str.contains("train", case=False, na=False), 'customer_position'] = 'trainee'
df_test.loc[df_test['customer_position'].str.contains("train", case=False, na=False), 'customer_position'] = 'trainee'

# c-level executive
df_train.loc[df_train['customer_position'].str.contains("executive", case=False, na=False), 'customer_position'] = 'c-level executive'
df_test.loc[df_test['customer_position'].str.contains("executive", case=False, na=False), 'customer_position'] = 'c-level executive'

# vice president
df_train.loc[df_train['customer_position'].str.contains("vicepresident", case=False, na=False), 'customer_position'] = 'vice president'
df_test.loc[df_test['customer_position'].str.contains("vicepresident", case=False, na=False), 'customer_position'] = 'vice president'

# intern
df_train.loc[df_train['customer_position'].str.contains("intern", case=False, na=False), 'customer_position'] = 'intern'
df_test.loc[df_test['customer_position'].str.contains("intern", case=False, na=False), 'customer_position'] = 'intern'

# end-user
df_train.loc[df_train['customer_position'].str.contains("end-user", case=False, na=False), 'customer_position'] = 'end-user'
df_test.loc[df_test['customer_position'].str.contains("end-user", case=False, na=False), 'customer_position'] = 'end-user'

# consultant
df_train.loc[df_train['customer_position'].str.contains("consult", case=False, na=False), 'customer_position'] = 'consultant'
df_test.loc[df_test['customer_position'].str.contains("consult", case=False, na=False), 'customer_position'] = 'consultant'

# manufacturer
df_train.loc[df_train['customer_position'].str.contains("manufacturer", case=False, na=False), 'customer_position'] = 'manufacturer'
df_test.loc[df_test['customer_position'].str.contains("manufacturer", case=False, na=False), 'customer_position'] = 'manufacturer'

# assistant professor
df_train.loc[df_train['customer_position'].str.contains("assistant", case=False, na=False), 'customer_position'] = 'assistant professor'
df_test.loc[df_test['customer_position'].str.contains("assistant", case=False, na=False), 'customer_position'] = 'assistant professor'


117->90

In [101]:
df_train['customer_position'].nunique()

90

***
***

## ***20. response_corporate***
담당 자사 법인명

In [102]:
df_train['response_corporate'].isnull().sum()

0

In [103]:
df_train['response_corporate'].unique()

array(['LGEPH', 'LGEIL', 'LGEAF', 'LGESJ', 'LGESL', 'LGESP', 'LGEGF',
       'LGESA', 'LGEUS', 'LGECB', 'LGEMS', 'LGEEG', 'LGEEF', 'LGEAP',
       'LGEIN', 'LGEUK', 'LGEKR', 'LGEPS', 'LGECI', 'LGECL', 'LGETK',
       'LGELF', 'LGEPT', 'LGEPR', 'LGEDG', 'LGERO', 'LGEMK', 'LGEPL',
       'LGECZ', 'LGEES', 'LGEAR', 'LGEHK', 'LGEML', 'LGEJP', 'LGEHS',
       'LGEAS', 'LGEYK', 'LGEIS', 'LGEBN', 'LGEFS', 'LGESW', 'LGEMC',
       'LGEAG', 'LGEEB', 'LGETH', 'LGEVH', 'LGECH', 'LGELA', 'LGETT',
       'LGERA', 'LGEUR', 'LGEIR', 'LGEBT'], dtype=object)

***
***

## ***21. expected_timeline***
고객의 요청한 처리 일정

In [104]:
df_train['expected_timeline'].isnull().sum()

28260

In [105]:
df_train['expected_timeline'].nunique()

449

In [106]:
#less than 3 months
df_train.loc[df_train['expected_timeline'] == "less_than_3_months", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "less than 3 months. customer not answered . to call back", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "duplicate lead - il220100042906. less than 3 months", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "less than 3 months- outdoor led requiment", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "3 months", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43", 'expected_timeline'] = "less than 3 months"
df_train.loc[df_train['expected_timeline'] == "4/8 months", 'expected_timeline'] = "less than 3 months"

#3 months ~ 6 months
df_train.loc[df_train['expected_timeline'] == "less than 6 months", 'expected_timeline'] = "3 months ~ 6 months"
df_train.loc[df_train['expected_timeline'] == "3_months_~_6_months", 'expected_timeline'] = "3 months ~ 6 months"
df_train.loc[df_train['expected_timeline'] == "more then 3 months", 'expected_timeline'] = "3 months ~ 6 months"
df_train.loc[df_train['expected_timeline'] == "less then 6 months", 'expected_timeline'] = "3 months ~ 6 months"
df_train.loc[df_train['expected_timeline'] == "less than 5 months", 'expected_timeline'] = "3 months ~ 6 months"

#6 months ~ 9 months
df_train.loc[df_train['expected_timeline'] == "6_months_~_9_months", 'expected_timeline'] = "6 months ~ 9 months"

#9 months ~ 1 year
df_train.loc[df_train['expected_timeline'] == "9_months_~_1_year", 'expected_timeline'] = "9 months ~ 1 year"
df_train.loc[df_train['expected_timeline'] == "9 months - 1 year", 'expected_timeline'] = "9 months ~ 1 year"

#more than a year
df_train.loc[df_train['expected_timeline'] == "more_than_a_year", 'expected_timeline'] = "more than a year"
df_train.loc[df_train['expected_timeline'] == "more_than_a_year", 'expected_timeline'] = "more than a year"

#df_test
df_test.loc[df_test['expected_timeline'] == "less_than_3_months", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "less than 3 months. customer not answered . to call back", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "duplicate lead - il220100042906. less than 3 months", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "less than 3 months- outdoor led requiment", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "3 months", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "less than 3 months ,meeting with the customer for the more details and tentative boq will ne 32 and 43", 'expected_timeline'] = "less than 3 months"
df_test.loc[df_test['expected_timeline'] == "4/8 months", 'expected_timeline'] = "less than 3 months"

df_test.loc[df_test['expected_timeline'] == "less than 6 months", 'expected_timeline'] = "3 months ~ 6 months"
df_test.loc[df_test['expected_timeline'] == "3_months_~_6_months", 'expected_timeline'] = "3 months ~ 6 months"
df_test.loc[df_test['expected_timeline'] == "more then 3 months", 'expected_timeline'] = "3 months ~ 6 months"
df_test.loc[df_test['expected_timeline'] == "less then 6 months", 'expected_timeline'] = "3 months ~ 6 months"
df_test.loc[df_test['expected_timeline'] == "less than 5 months", 'expected_timeline'] = "3 months ~ 6 months"

df_test.loc[df_test['expected_timeline'] == "6_months_~_9_months", 'expected_timeline'] = "6 months ~ 9 months"

df_test.loc[df_test['expected_timeline'] == "9_months_~_1_year", 'expected_timeline'] = "9 months ~ 1 year"
df_test.loc[df_test['expected_timeline'] == "9 months - 1 year", 'expected_timeline'] = "9 months ~ 1 year"

df_test.loc[df_test['expected_timeline'] == "more_than_a_year", 'expected_timeline'] = "more than a year"
df_test.loc[df_test['expected_timeline'] == "more_than_a_year", 'expected_timeline'] = "more than a year"

df_train.loc[df_train['expected_timeline'] == "being followed up", 'expected_timeline'] = "being followed up"
df_train.loc[df_train['expected_timeline'] == "being followed up.", 'expected_timeline'] = "being followed up"

In [107]:
#처리완료가 아닌 것 및 결측치 처리
timeline = ['less than 3 months', '3 months ~ 6 months', '6 months ~ 9 months', '9 months ~ 1 year', 'more than a year']
df_train['expected_timeline'] = df_train['expected_timeline'].fillna("incomplete")
timeline.append("incomplete")
df_train.loc[~df_train['expected_timeline'].isin(timeline), "expected_timeline"] = "etc"

df_test['expected_timeline'] = df_test['expected_timeline'].fillna("incomplete")
df_test.loc[~df_test['expected_timeline'].isin(timeline), "expected_timeline"] = "etc"

***
***

## ***22. ver_cus***
특정 Vertical Level 1(사업영역) 이면서 Customer_type(고객 유형)이 소비자(End-user)인 경우에 대한 가중치

In [108]:
df_train['ver_cus'].isnull().sum()

0

In [109]:
df_train['ver_cus'].value_counts()

ver_cus
0    53378
1     2364
Name: count, dtype: int64

In [110]:
df_train[df_train['ver_cus']==1]['business_area'].unique()

array(['corporate / office', 'education', 'hotel & accommodation',
       'retail'], dtype=object)

In [111]:
df_train[df_train['ver_cus']==1]['customer_type'].unique()

array(['End-Customer'], dtype=object)

***
***

## ***23. ver_pro***
특정 Vertical Level 1(사업영역) 이면서 특정 Product Category(제품 유형)인 경우에 대한 가중치

In [112]:
df_train['ver_pro'].isnull().sum()

0

In [113]:
df_train['ver_pro'].value_counts()

ver_pro
0    52809
1     2933
Name: count, dtype: int64

In [114]:
df_train[df_train['ver_pro']==1]['business_area'].unique()

array(['corporate / office', 'retail', 'hotel & accommodation'],
      dtype=object)

In [115]:
df_train[df_train['ver_pro']==1]['product_category'].unique()

array(['Commercial Display / Standard Signage',
       'Commercial Display / High Brightness Signage',
       'Commercial Display / Interactive Signage',
       'Commercial Display / Video Wall Signage',
       'Commercial Display / LED Signage',
       'Commercial Display / Signage Care Solution',
       'Commercial Display / OLED Signage',
       'Commercial Display / Special Signage', 'Commercial Display',
       'Commercial Display / Digital Signage', 'Unknown',
       'Commercial Display / Hotel TV'], dtype=object)

***
***

## ***24. ver_win_rate_x***
전체 Lead 중에서 Vertical을 기준으로 Vertical 수 비율과 Vertical 별 Lead 수 대비 영업 전환 성공 비율 값을 곱한 값

In [116]:
df_train['ver_win_rate_x'].isnull().sum()

37975

In [117]:
df_train['ver_win_rate_x'].value_counts()

ver_win_rate_x
0.003079    3985
0.001183    2918
0.000717    1958
0.000543    1873
0.000298    1786
0.000572    1678
0.000060    1019
0.000215    1013
0.000097     635
0.000026     502
0.000013     292
0.000002     108
Name: count, dtype: int64

In [118]:
df_train.groupby('ver_win_rate_x')[['business_unit','business_area']].value_counts()

ver_win_rate_x  business_unit  business_area                 
0.000002        AS             power plant / renewable energy      58
                ID             power plant / renewable energy      42
                IT             power plant / renewable energy       8
0.000013        ID             transportation                     209
                AS             transportation                      51
                IT             transportation                      28
                Solution       transportation                       4
0.000026        ID             public facility                    265
                AS             public facility                    192
                IT             public facility                     37
                Solution       public facility                      8
0.000060        IT             hospital & health care             739
                AS             hospital & health care             150
                ID          

***
***

## ***25. ver_win_ratio_per_bu***
특정 Vertical Level1의 Business Unit 별 샘플 수 대비 영업 전환된 샘플 수의 비율을 계산

In [119]:
df_train['ver_win_ratio_per_bu'].isnull().sum()

40868

In [120]:
df_train['ver_win_ratio_per_bu'].value_counts()

ver_win_ratio_per_bu
0.064566    2397
0.049840    1963
0.020121    1342
0.048630    1325
0.026846    1237
0.064070    1037
0.071345     936
0.011583     771
0.022634     699
0.060924     530
0.079412     411
0.036913     397
0.035484     378
0.031579     265
0.053571     209
0.051471     206
0.028777     192
0.022727     170
0.128571     150
0.131148     130
0.227273      58
0.285714      42
0.034483      29
Name: count, dtype: int64

In [121]:
df_train.groupby('ver_win_ratio_per_bu')[['business_unit','business_area']].value_counts()

ver_win_ratio_per_bu  business_unit  business_area                 
0.011583              AS             retail                             771
0.020121              AS             residential (home)                1342
0.022634              AS             special purpose                    699
0.022727              AS             government department              170
0.026846              AS             corporate / office                1237
0.028777              AS             public facility                    192
0.031579              ID             public facility                    265
0.034483              Solution       corporate / office                  29
0.035484              ID             residential (home)                 378
0.036913              AS             factory                            397
0.048630              ID             education                         1325
0.049840              ID             retail                            1963
0.051471            

***
***

## ***26. business_area***
고객의 사업 영역

In [122]:
df_train['business_area'].isnull().sum()

37975

In [123]:
df_train['business_area'].value_counts()

business_area
corporate / office                3985
retail                            2918
hotel & accommodation             1958
special purpose                   1873
residential (home)                1786
education                         1678
hospital & health care            1019
factory                           1013
government department              635
public facility                    502
transportation                     292
power plant / renewable energy     108
Name: count, dtype: int64

In [124]:
#Business_Area 결측치 처리
df_train['business_area'] = df_train['business_area'].fillna("Not entered")
df_test['business_area'] = df_test['business_area'].fillna("Not entered")

***
***

## ***27. business_subarea***
고객의 세부 사업 영역

In [125]:
df_train['business_subarea'].isnull().sum()

50578

In [126]:
df_train['business_subarea'].value_counts()

business_subarea
Others                           1579
Other Stores                      268
Hospital                          245
Manufacturing Factory / Plant     190
Construction                      182
                                 ... 
Duty Free Shop                      2
Dormitory                           1
Casino Resort                       1
Holdings                            1
Travel Agency                       1
Name: count, Length: 86, dtype: int64

In [127]:
#Business_Subarea 결측치 처리
df_train['business_subarea'] = df_train['business_subarea'].fillna("Not entered")
df_test['business_subarea'] = df_test['business_subarea'].fillna("Not entered")

In [128]:
#Business_Subarea 건강관련 값 통일
health = ['Hospital','General Hospital','Clinic','Healthcare']
df_train.loc[df_train['business_subarea'].isin(health), 'business_subarea'] = "Health"
df_test.loc[df_test['business_subarea'].isin(health), 'business_subarea'] = "Health"

In [129]:
tmp_df = pd.concat([pd.DataFrame(df_train.groupby('business_subarea')['is_converted'].mean()), pd.DataFrame(df_train.groupby('business_subarea')['is_converted'].count())], axis=1)
tmp_df.columns = ['영업전환_확률', 'Count']
tmp_df.sort_values('영업전환_확률', ascending = False)

,영업전환_확률,Count
business_subarea,,
Casino Resort,1.000000,1
CVS (Convenience Store),1.000000,2
F&B(Food and Beverage),1.000000,2
Cruise,0.659574,47
Hotel,0.657143,35
...,...,...
Gas Station,0.000000,4
Holdings,0.000000,1
Aerospace,0.000000,5


***
***

## ***28. lead_owner***
영업 담당자 이름

In [130]:
df_train['lead_owner'].isnull().sum()

0

In [131]:
# # lead_owner가 2개 이하일 경우 과적합 우려 있어 특정 값 대체.

# tmp_lst = df_train.groupby('lead_owner')['is_converted'].count().loc[lambda x: x <= 3].index
# df_train.loc[df_train['lead_owner'].isin(tmp_lst), 'lead_owner'] = 'Unknown'
# df_test.loc[df_test['lead_owner'].isin(tmp_lst), 'lead_owner'] = 'Unknown'

***
***

## ***29. is_converted***
영업 성공 여부. True일 시 성공.

In [132]:
df_train['is_converted'].isnull().sum()

0

In [133]:
df_train['is_converted'].value_counts()

is_converted
False    51141
True      4601
Name: count, dtype: int64

***
***
***

# 3. Feature Engineering

***

## ***product_category_2***
동일 조건 여러개 category

In [134]:
df_train['product_category_2'] = df_train['product_category'].copy()
df_test['product_category_2'] = df_test['product_category'].copy()

In [135]:
product_dict = {
    '110UM5J' : 'Commercial Display / Digital Signage',    '15LS766F (NA)' : 'Commercial Display / commercial tv',    '28MQ780' : 'IT PRODUCTS / Monitor',    '32LT340C' : 'Commercial Display / Digital Signage',    '43HT3WJ' : 'Commercial Display / Interactive signage',    '43UH5F-H' : 'Commercial Display / Digital Signage',    '43US660H' : 'Commercial Display / Hotel TV',    '49UH7F-H' : 'Commercial Display / Digital Signage',    '49VL5PF' : 'Commercial Display / Video Wall Signage',    '55CT5WJ' : 'Commercial Display / One:Quick Series',    '55EF5F-L' : 'Commercial Display / OLED Signage',    '55EF5F-P' : 'Commercial Display / OLED Signage',    '55EF5G-P' : 'Commercial Display / OLED Signage',    '55TR3BG-B' : 'Commercial Display / Interactive signage',    '55UH5F-H': 'Commercial Display / Digital Signage',    '55UH7F-B' : 'Commercial Display / Digital Signage',    '55UR640S' : 'Commercial Display / commercial tv',    '55VM5J-H' : 'Commercial Display / Video Wall Signage',    '55VSM5J' : 'Commercial Display / Video Wall Signage',    '55XE4F-M' : 'Commercial Display / High Brightness Signage',    '55XS2E' : 'Commercial Display / Digital Signage',
    '65EV5E' : 'Commercial Display / OLED Signage',    '65UH5F-H' : 'Commercial Display / Standard Signage',    '75TC3D' : 'Commercial Display / Interactive signage',    '75UR640S (EU/CIS)' : 'Commercial Display / Digital Signage',    '85TR3BF' : 'Commercial Display / Interactive signage',    '86BH5F' : 'Commercial Display / Digital Signage',    '86BH5F-B' : 'Commercial Display / Digital Signage',    '86TR3E' : 'Commercial Display / Interactive signage',    '86UT640S' : 'Commercial Display / commercial tv',    '98UH5F-H' : 'Commercial Display / commercial tv',    'Ergo Dual(27QP88D)' : 'IT PRODUCTS / Monitor',    'GSCC066' : 'Commercial Display / LED Signage',    'KT-T32E' : 'Commercial Display / Digital Signage',    'LAD033F' : 'Commercial Display / LED Signage',    'LAEB015' : 'Commercial Display / LED Signage',    'LAS039DB9-V' : 'Commercial Display / LED Signage',    'LAT140' : 'Commercial Display / LED Signage',    'LSAA012' : 'Commercial Display / LED Signage',    'LSAB009' : 'Commercial Display / LED Signage',    'LSAB012' : 'Commercial Display / LED Signage',    'SC-00DA' : 'Commercial Display / One:Quick Series',    'UltraFine Ergo(32UN880)' : 'IT PRODUCTS / Monitor',    'UltraWide Ergo(34WN780)' : 'IT PRODUCTS / Monitor'}

In [136]:
product_dict_key = product_dict.keys()
product_dict_value = product_dict.values()

In [137]:
for i, val in tqdm(enumerate(product_dict_key)):
    df_train.loc[df_train['product_modelname'] == val, 'product_category_2'] = product_dict[val]
    df_test.loc[df_test['product_modelname'] == val, 'product_category_2'] = product_dict[val]

44it [00:00, 300.18it/s]


In [138]:
product_dict = {
    '49" 500 nits FHD Slim Bezel Video Wall' : 'Commercial Display / Video Wall Signage',    'All Medical Display' : 'IT PRODUCTS / Medical Display',    'All Medical Displays' : 'IT PRODUCTS / Medical Display',    'All Monitors & PCs' : 'IT PRODUCTS / Monitor & IT PRODUCTS / pc',    'All Projectors' : 'IT PRODUCTS / Projector',    'Built-in Touch Display' : 'Commercial Display / Interactive signage',    'Clinical Review Monitors' : 'IT PRODUCTS / Medical Display',    'Diagnostic Monitors' : 'IT PRODUCTS / Medical Display',    'Diagnostic Monitors\u200b' : 'IT PRODUCTS / Medical Display',    'Digital X-ray Detectors' : 'IT PRODUCTS / Medical Display',    'Essential Series' : 'Commercial Display / LED Signage',    'Flat OLED Signage' : 'Commercial Display / OLED Signage',    'Interactive Digital Board' : 'Commercial Display / Interactive signage',    'LED Cinema' : 'Commercial Display / LED Signage',    'LT340C Series' : 'Commercial Display / commercial tv',    'LU640H Series' : 'Commercial Display / Hotel TV',    'Laptops' : 'IT PRODUCTS / Laptop',    'OLED Pro Monitor' : 'IT PRODUCTS / Monitor',    'One:Quick Flex' : 'Commercial Display / One:Quick Series',    'One:Quick Works' : 'Commercial Display / One:Quick Series',    'One:Quick works' : 'Commercial Display / One:Quick Series',    'Other' : 'others',    'Others' : 'others',    'SE3KE Series' : 'Commercial Display / Digital Signage',    'SH7DD 系列' : 'Commercial Display / Digital Signage',    'SM3G Series' : 'Commercial Display / Digital Signage',    'SM5KE Series' : 'Commercial Display / Digital Signage',    'Series' : 'others',    'Smart Touch Screen TV' : 'Commercial Display / commercial tv',    'Surgical Monitors' : 'IT PRODUCTS / Medical Display',    'TC3D Series' : 'Commercial Display / Interactive signage',
    'TR3BF Series' : 'Commercial Display / Interactive signage',    'TR3BG Series' : 'Commercial Display / Interactive signage',    'Thin Clients' : 'IT PRODUCTS / Cloud Device',    'Touch Overlay Kit' : 'Commercial Display / Digital Signage',    'UH5F Series' : 'UH5F Series',    'UH5F-H Series' : 'Commercial Display / Digital Signage',    'UH7F Series' : 'Commercial Display / Digital Signage',    'UH7F-H Series' : 'Commercial Display / Digital Signage',    'UHD 4K Monitors' : 'IT PRODUCTS / Monitor',    'UHD Large Screen Signage Display' : 'Commercial Display / Digital Signage',    'UHD TV Signage' : 'Commercial Display / Digital Signage',    'UL3G Series' : 'Commercial Display / Digital Signage',    'UM3DG Series' : 'Commercial Display / Digital Signage',    'UM5J Series' : 'Commercial Display / Digital Signage',    'UR640S Series' : 'Commercial Display / Digital Signage',    'UT640S' : 'Commercial Display / Digital Signage',    'UT640S Series' : 'Commercial Display / Digital Signage',    'Ultra Narrow Bezel Video Wall' : 'Commercial Display / Video Wall Signage',    'Ultra Slim Series' : 'Commercial Display / LED Signage',    'Ultra Stretch Series' : 'Commercial Display / Digital Signage',    'Ultra Stretch Signage' : 'Commercial Display / Digital Signage',    'VH7E Series' : 'Commercial Display / Digital Signage',    'Video Wall' : 'Commercial Display / Video Wall Signage',    'Wallpaper OLED Signage' : 'Commercial Display / OLED Signage',    'Window Facing Display' : 'Commercial Display / Digital Signage'}

In [139]:
product_dict_key = product_dict.keys()
product_dict_value = product_dict.values()

In [140]:
for i, val in tqdm(enumerate(product_dict_key)):
    df_train.loc[df_train['product_subcategory'] == val, 'product_category_2'] = product_dict[val]
    df_test.loc[df_test['product_subcategory'] == val, 'product_category_2'] = product_dict[val]

56it [00:00, 299.59it/s]


***

## ***write_budget***
MQL 구성 요소 중 Budget 작성여부

In [141]:
#MQL 구성 요소 중 Budget 작성여부
df_train.loc[(df_train['bant_submit'] == 1.0) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 1

df_train.loc[(df_train['bant_submit'] == 0.0) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 0

df_train.loc[(df_train['bant_submit'] == 0.25) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.25) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 0
df_train.loc[(df_train['bant_submit'] == 0.25) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 0
df_train.loc[(df_train['bant_submit'] == 0.25) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 0

df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 0
df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 0
df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 0
df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.5) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 1

df_train.loc[(df_train['bant_submit'] == 0.75) & (df_train['expected_timeline'].isna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.75) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.75) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] != "none") & (df_train['inquiry_type'].isna()),'write_budget'] = 1
df_train.loc[(df_train['bant_submit'] == 0.75) & (df_train['expected_timeline'].notna())  & (df_train['customer_position'] == "none") & (df_train['inquiry_type'].notna()),'write_budget'] = 0

#MQL 구성 요소 중 Budget 작성여부
df_test.loc[(df_test['bant_submit'] == 1.0) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 1

df_test.loc[(df_test['bant_submit'] == 0.0) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 0

df_test.loc[(df_test['bant_submit'] == 0.25) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.25) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 0
df_test.loc[(df_test['bant_submit'] == 0.25) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 0
df_test.loc[(df_test['bant_submit'] == 0.25) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 0

df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 0
df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 0
df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 0
df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.5) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 1

df_test.loc[(df_test['bant_submit'] == 0.75) & (df_test['expected_timeline'].isna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.75) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.75) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] != "none") & (df_test['inquiry_type'].isna()),'write_budget'] = 1
df_test.loc[(df_test['bant_submit'] == 0.75) & (df_test['expected_timeline'].notna())  & (df_test['customer_position'] == "none") & (df_test['inquiry_type'].notna()),'write_budget'] = 0

In [142]:
df_train['write_budget'] = df_train['write_budget'].fillna(0)
df_test['write_budget'] = df_test['write_budget'].fillna(0)

***

## ***bant_submit_zegob***
bant_submit 제곱

In [143]:
df_train['bant_submit_zegob'] = df_train['bant_submit'] * df_train['bant_submit']
df_test['bant_submit_zegob'] = df_test['bant_submit'] * df_test['bant_submit']

***

## ***raw_country***
기존 나라 컬럼 그대로 사용

In [144]:
df_train['raw_country'] = df_train['customer_country2'].copy()
df_test['raw_country'] = df_test['customer_country2'].copy()

In [145]:
df_train['raw_country'] = df_train['raw_country'].apply(lambda x: re.sub(r'\s*/\s*', '/', x) if isinstance(x, str) else x)
df_test['raw_country'] = df_test['raw_country'].apply(lambda x: re.sub(r'\s*/\s*', '/', x) if isinstance(x, str) else x)

In [146]:
df_train['raw_country'] = df_train['raw_country'].astype(str).apply(lambda x : x.strip().lower())
df_test['raw_country'] = df_test['raw_country'].astype(str).apply(lambda x : x.strip().lower())

***

## ***City***
도시정보

In [147]:
split = df_train['customer_country2'].str.split('/', expand=True)
condition_1 = split[1].notna() & split[2].notna()
df_train.loc[condition_1, 'Field'] = None
df_train.loc[condition_1, 'City'] = split[1]
df_train.loc[condition_1, 'Country'] = split[2]
# 두 번째 섹션이 비어 있음
condition_2 = split[0].notna() & split[2].notna()
df_train.loc[condition_2, 'Field'] = split[0]
df_train.loc[condition_2, 'City'] = None
df_train.loc[condition_2, 'Country'] = split[2]
# 세 번째 섹션이 비어 있음
condition_3 = split[0].notna() & split[1].notna()
df_train.loc[condition_3, 'Field'] = split[0]
df_train.loc[condition_3, 'City'] = split[1]
df_train.loc[condition_3, 'Country'] = None
# 첫 번째와 두 번째 섹션이 비어 있음
condition_4 = split[2].notna() & split[0].isna() & split[1].isna()
df_train.loc[condition_4, 'Field'] = None
df_train.loc[condition_4, 'City'] = None
df_train.loc[condition_4, 'Country'] = split[2]
# 첫 번째와 세 번째 섹션이 비어 있고 두 번째 섹션에만 값이 있음
condition_5 = split[1].notna() & split[0].isna() & split[2].isna()
df_train.loc[condition_5, 'Field'] = None
df_train.loc[condition_5, 'City'] = split[1]
df_train.loc[condition_5, 'Country'] = None
# 두 번째와 세 번째 섹션이 비어 있고 첫 번째 섹션에만 값이 있음
condition_6 = split[0].notna() & split[1].isna() & split[2].isna()
df_train.loc[condition_6, 'Field'] = split[0]
df_train.loc[condition_6, 'City'] = None
df_train.loc[condition_6, 'Country'] = None
# 모든 섹션이 비어 있음
condition_7 = split[0].isna() & split[1].isna() & split[2].isna()
df_train.loc[condition_7, ['Field', 'City', 'Country']] = None
# 모든 섹션이 채워져 있는 경우 처리 추가
condition_all_filled = split[0].notna() & split[1].notna() & split[2].notna()
df_train.loc[condition_all_filled, 'Field'] = split[0]
df_train.loc[condition_all_filled, 'City'] = split[1]
df_train.loc[condition_all_filled, 'Country'] = split[2]
# 이미 처리한 조건들에 대한 인덱스를 제외하고 나머지 데이터 확인을 위한 조건 재설정
remaining_conditions = ~(condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_all_filled)
# 처리되지 않은 나머지 데이터 확인 (있을 경우)
remaining_df_train = df_train[remaining_conditions]
remaining_sample = remaining_df_train[['customer_country2', 'Field', 'City', 'Country']] if not remaining_df_train.empty else "All cases have been handled."
split = df_test['customer_country2'].str.split('/', expand=True)
condition_1 = split[1].notna() & split[2].notna()
df_test.loc[condition_1, 'Field'] = None
df_test.loc[condition_1, 'City'] = split[1]
df_test.loc[condition_1, 'Country'] = split[2]
condition_2 = split[0].notna() & split[2].notna()
df_test.loc[condition_2, 'Field'] = split[0]
df_test.loc[condition_2, 'City'] = None
df_test.loc[condition_2, 'Country'] = split[2]
condition_3 = split[0].notna() & split[1].notna()
df_test.loc[condition_3, 'Field'] = split[0]
df_test.loc[condition_3, 'City'] = split[1]
df_test.loc[condition_3, 'Country'] = None
condition_4 = split[2].notna() & split[0].isna() & split[1].isna()
df_test.loc[condition_4, 'Field'] = None
df_test.loc[condition_4, 'City'] = None
df_test.loc[condition_4, 'Country'] = split[2]
condition_5 = split[1].notna() & split[0].isna() & split[2].isna()
df_test.loc[condition_5, 'Field'] = None
df_test.loc[condition_5, 'City'] = split[1]
df_test.loc[condition_5, 'Country'] = None
condition_6 = split[0].notna() & split[1].isna() & split[2].isna()
df_test.loc[condition_6, 'Field'] = split[0]
df_test.loc[condition_6, 'City'] = None
df_test.loc[condition_6, 'Country'] = None
condition_7 = split[0].isna() & split[1].isna() & split[2].isna()
df_test.loc[condition_7, ['Field', 'City', 'Country']] = None
condition_all_filled = split[0].notna() & split[1].notna() & split[2].notna()
df_test.loc[condition_all_filled, 'Field'] = split[0]
df_test.loc[condition_all_filled, 'City'] = split[1]
df_test.loc[condition_all_filled, 'Country'] = split[2]
remaining_conditions = ~(condition_1 | condition_2 | condition_3 | condition_4 | condition_5 | condition_6 | condition_7 | condition_all_filled)
remaining_df_test = df_test[remaining_conditions]
remaining_sample = remaining_df_test[['customer_country2', 'Field', 'City', 'Country']] if not remaining_df_test.empty else "All cases have been handled."

In [148]:
df_train.drop(['customer_country2'],axis=1,inplace=True)
df_test.drop(['customer_country2'],axis=1,inplace=True)

In [149]:
df_train['City'] = df_train['City'].astype(str).apply(lambda x : x.strip().lower())
df_test['City'] = df_test['City'].astype(str).apply(lambda x : x.strip().lower())

In [150]:
# 영국 도시
df_train.loc[df_train['City'].str.contains('paulo', case=True, na=False), 'City'] = 'sanpaulo'
df_test.loc[df_test['City'].str.contains('paulo', case=True, na=False), 'City'] = 'sanpaulo'

df_train.loc[df_train['City'].str.contains('mumbai', case=True, na=False), 'City'] = 'mumbai'
df_test.loc[df_test['City'].str.contains('mumbai', case=True, na=False), 'City'] = 'mumbai'

df_train.loc[df_train['City'].str.contains('hyderabad', case=True, na=False), 'City'] = 'hyderabad'
df_test.loc[df_test['City'].str.contains('hyderabad', case=True, na=False), 'City'] = 'hyderabad'

df_train.loc[df_train['City'].str.contains('lima', case=True, na=False), 'City'] = 'lima'
df_test.loc[df_test['City'].str.contains('lima', case=True, na=False), 'City'] = 'lima'

df_train.loc[df_train['City'].str.contains('bangalore', case=True, na=False), 'City'] = 'bangalore'
df_test.loc[df_test['City'].str.contains('bangalore', case=True, na=False), 'City'] = 'bangalore'

df_train.loc[df_train['City'].str.contains('santiago', case=True, na=False), 'City'] = 'santiago'
df_test.loc[df_test['City'].str.contains('santiago', case=True, na=False), 'City'] = 'santiago'

df_train.loc[df_train['City'].str.contains('mexico', case=True, na=False), 'City'] = 'mexico'
df_test.loc[df_test['City'].str.contains('mexico', case=True, na=False), 'City'] = 'mexico'

df_train.loc[df_train['City'].str.contains('dubai', case=True, na=False), 'City'] = 'dubai'
df_test.loc[df_test['City'].str.contains('dubai', case=True, na=False), 'City'] = 'dubai'

df_train.loc[df_train['City'].str.contains('pune', case=True, na=False), 'City'] = 'pune'
df_test.loc[df_test['City'].str.contains('pune', case=True, na=False), 'City'] = 'pune'

df_train.loc[df_train['City'].str.contains('delhi', case=True, na=False), 'City'] = 'delhi'
df_test.loc[df_test['City'].str.contains('delhi', case=True, na=False), 'City'] = 'delhi'

df_train.loc[df_train['City'].str.contains('chennai', case=True, na=False), 'City'] = 'chennai'
df_test.loc[df_test['City'].str.contains('chennai', case=True, na=False), 'City'] = 'chennai'

df_train.loc[df_train['City'].str.contains('bogot', case=True, na=False), 'City'] = 'bogota'
df_test.loc[df_test['City'].str.contains('bogot', case=True, na=False), 'City'] = 'bogota'

df_train.loc[df_train['City'].str.contains('janeiro', case=True, na=False), 'City'] = 'janeiro'
df_test.loc[df_test['City'].str.contains('janeiro', case=True, na=False), 'City'] = 'janeiro'

df_train.loc[df_train['City'].str.contains('kolkata', case=True, na=False), 'City'] = 'kolkata'
df_test.loc[df_test['City'].str.contains('kolkata', case=True, na=False), 'City'] = 'kolkata'

df_train.loc[df_train['City'].str.contains('ahmedabad', case=True, na=False), 'City'] = 'ahmedabad'
df_test.loc[df_test['City'].str.contains('ahmedabad', case=True, na=False), 'City'] = 'ahmedabad'

df_train.loc[df_train['City'].str.contains('patna', case=True, na=False), 'City'] = 'patna'
df_test.loc[df_test['City'].str.contains('patna', case=True, na=False), 'City'] = 'patna'

df_train.loc[df_train['City'].str.contains('cdmx', case=True, na=False), 'City'] = 'cdmx'
df_test.loc[df_test['City'].str.contains('cdmx', case=True, na=False), 'City'] = 'cdmx'

df_train.loc[df_train['City'].str.contains('riyadh', case=True, na=False), 'City'] = 'riyadh'
df_test.loc[df_test['City'].str.contains('riyadh', case=True, na=False), 'City'] = 'riyadh'

df_train.loc[df_train['City'].str.contains('surat', case=True, na=False), 'City'] = 'surat'
df_test.loc[df_test['City'].str.contains('surat', case=True, na=False), 'City'] = 'surat'

df_train.loc[df_train['City'].str.contains('gujarat', case=True, na=False), 'City'] = 'gujarat'
df_test.loc[df_test['City'].str.contains('gujarat', case=True, na=False), 'City'] = 'gujarat'

df_train.loc[df_train['City'].str.contains('quezon city', case=True, na=False), 'City'] = 'quezon city'
df_test.loc[df_test['City'].str.contains('quezon city', case=True, na=False), 'City'] = 'quezon city'

df_train.loc[df_train['City'].str.contains('oran', case=True, na=False), 'oran'] = 'oran'
df_test.loc[df_test['City'].str.contains('oran', case=True, na=False), 'oran'] = 'oran'

df_train.loc[df_train['City'].str.contains('ohio', case=True, na=False), 'ohio'] = 'ohio'
df_test.loc[df_test['City'].str.contains('ohio', case=True, na=False), 'ohio'] = 'ohio'

df_train.loc[df_train['City'].str.contains('valencia', case=True, na=False), 'valencia'] = 'valencia'
df_test.loc[df_test['City'].str.contains('valencia', case=True, na=False), 'valencia'] = 'valencia'

In [151]:
df_train.loc[df_train['City']=='.','City'] =''
df_test.loc[df_test['City']=='.','City'] =''

df_train.loc[df_train['City']==',','City'] =''
df_test.loc[df_test['City']==',','City'] =''

data1 = df_train['City'].value_counts().reset_index()
small_city_list1 = list(data1[data1['count']<=2]['City'].unique())
small_city = small_city_list1

df_train.loc[df_train['City'].isin(small_city),'City'] = df_train['customer_country']
df_test.loc[df_test['City'].isin(small_city),'City'] = df_test['customer_country']

df_train['City'] = df_train['City'].astype(str).apply(lambda x : x.strip().lower())
df_test['City'] = df_test['City'].astype(str).apply(lambda x : x.strip().lower())

df_train.loc[df_train['City']=='','City']=np.nan
df_test.loc[df_test['City']=='','City']=np.nan

df_train.loc[df_train['City']=='unknown','City']=np.nan
df_test.loc[df_test['City']=='unknown','City']=np.nan

df_train['City'] = df_train['City'].fillna(df_train['customer_country'])
df_test['City'] = df_test['City'].fillna(df_test['customer_country'])

df_train['City'] = df_train['City'].astype(str).apply(lambda x : x.strip().lower())
df_test['City'] = df_test['City'].astype(str).apply(lambda x : x.strip().lower())

In [152]:
df_train.loc[df_train['Field'].str.contains('Cloud', case=True, na=False), 'Field'] ='IT/CLOUD'
df_train.loc[df_train['Field'].str.contains('Commercial Display', case=True, na=False), 'Field'] = 'Commercial Display'
df_train.loc[df_train['Field'].str.contains('Education', case=True, na=False), 'Field'] = 'Education'
df_train.loc[df_train['Field'].str.contains('Cruise', case=True, na=False), 'Field'] = 'Cruise'
df_train.loc[df_train['Field'].str.contains('Medical', case=True, na=False), 'Field'] = 'Medical'
df_train.loc[df_train['Field'].str.contains('Healthcare', case=True, na=False), 'Field'] = 'Healthcare'
df_train.loc[df_train['Field'].str.contains('Information Technology', case=True, na=False), 'Field'] = 'Information Technology'
df_train.loc[df_train['Field'].str.contains('Hotel', case=True, na=False), 'Field'] = 'Hotel'
df_train.loc[df_train['Field'].str.contains('Finance & Insurance', case=True, na=False), 'Field'] = 'Finance & Insurance'
df_train.loc[df_train['Field'].str.contains('Retail', case=True, na=False), 'Field'] = 'Retail'
df_train.loc[df_train['Field'].str.contains('Transport & Logistics', case=True, na=False), 'Field'] = 'Transport & Logistics'
#it products / cloud device

df_train.loc[df_train['Field']=='IT/CLOUD','product_category']='it products / cloud device'
df_train.loc[(df_train['Field']=='Commercial Display') & (df_train['product_category']=='ID_unknown'),'product_category']='Commercial Display'
df_train.loc[(df_train['Field']=='Commercial Display') & (df_train['product_category']=='IT_unknown'),'product_category']='Commercial Display'
df_train.loc[(df_train['Field']=='Cruise') & (df_train['business_subarea']=='Not entered'),'business_subarea']='Cruise'
df_train.loc[(df_train['Field']=='Cruise') & (df_train['business_area']=='hotel & accommodation'),'business_subarea']='hotel & accommodation'
df_train.loc[(df_train['Field']=='Medical') ,'business_subarea']='Health'
df_train.loc[(df_train['Field']=='Medical') ,'business_area']='hospital & health care'

df_train.loc[(df_train['Field']=='Healthcare') ,'business_subarea']='Health'
df_train.loc[(df_train['Field']=='Healthcare') ,'business_area']='hospital & health care'

df_train.loc[(df_train['Field']=='Finance & Insurance') ,'business_subarea']='Finance'
df_train.loc[(df_train['Field']=='Finance & Insurance') ,'business_area']='corporate / office'

df_train.loc[(df_train['Field']=='Retail') ,'business_subarea']='Other Stores'
df_train.loc[(df_train['Field']=='Retail') ,'business_area']='retail'

#df_train.loc[(df_train['Field']=='Education') ,'product_category']='commercial display'
df_train.loc[(df_train['Field']=='Education') ,'business_area']='education'

In [153]:
df_test.loc[df_test['Field'].str.contains('Cloud', case=True, na=False), 'Field'] ='IT/CLOUD'
df_test.loc[df_test['Field'].str.contains('Commercial Display', case=True, na=False), 'Field'] = 'Commercial Display'
df_test.loc[df_test['Field'].str.contains('Education', case=True, na=False), 'Field'] = 'Education'
df_test.loc[df_test['Field'].str.contains('Cruise', case=True, na=False), 'Field'] = 'Cruise'
df_test.loc[df_test['Field'].str.contains('Medical', case=True, na=False), 'Field'] = 'Medical'
df_test.loc[df_test['Field'].str.contains('Healthcare', case=True, na=False), 'Field'] = 'Healthcare'
df_test.loc[df_test['Field'].str.contains('Information Technology', case=True, na=False), 'Field'] = 'Information Technology'
df_test.loc[df_test['Field'].str.contains('Hotel', case=True, na=False), 'Field'] = 'Hotel'
df_test.loc[df_test['Field'].str.contains('Finance & Insurance', case=True, na=False), 'Field'] = 'Finance & Insurance'
df_test.loc[df_test['Field'].str.contains('Retail', case=True, na=False), 'Field'] = 'Retail'
df_test.loc[df_test['Field'].str.contains('Transport & Logistics', case=True, na=False), 'Field'] = 'Transport & Logistics'
#it products / cloud device

df_test.loc[df_test['Field']=='IT/CLOUD','product_category']='it products / cloud device'
df_test.loc[(df_test['Field']=='Commercial Display') & (df_test['product_category']=='ID_unknown'),'product_category']='Commercial Display'
df_test.loc[(df_test['Field']=='Commercial Display') & (df_test['product_category']=='IT_unknown'),'product_category']='Commercial Display'
df_test.loc[(df_test['Field']=='Cruise') & (df_test['business_subarea']=='Not entered'),'business_subarea']='Cruise'
df_test.loc[(df_test['Field']=='Cruise') & (df_test['business_area']=='hotel & accommodation'),'business_subarea']='hotel & accommodation'
df_test.loc[(df_test['Field']=='Medical') ,'business_subarea']='Health'
df_test.loc[(df_test['Field']=='Medical') ,'business_area']='hospital & health care'

df_test.loc[(df_test['Field']=='Healthcare') ,'business_subarea']='Health'
df_test.loc[(df_test['Field']=='Healthcare') ,'business_area']='hospital & health care'

df_test.loc[(df_test['Field']=='Finance & Insurance') ,'business_subarea']='Finance'
df_test.loc[(df_test['Field']=='Finance & Insurance') ,'business_area']='corporate / office'

df_test.loc[(df_test['Field']=='Retail') ,'business_subarea']='Other Stores'
df_test.loc[(df_test['Field']=='Retail') ,'business_area']='retail'

#df_test.loc[(df_test['Field']=='Education') ,'product_category']='commercial display'
df_test.loc[(df_test['Field']=='Education') ,'business_area']='education'

In [154]:
df_train = df_train.drop(columns = ['Field','Country'])
df_test = df_test.drop(columns = ['Field','Country'])

***

## ***customer_continent***
나라 대륙 정보

In [155]:
middle_east = ['India','Afghanistan', 'Iran', 'Iraq', 'Israel', 'Jordan', 'Kuwait', 'Lebanon', 'Oman', 'Qatar', 'Saudi Arabia', 'Syria', 'Turkey', 'United Arab Emirates', 'Yemen','Saudi Arabia', 'United Arab Emirates', 'Oman', 'Qatar', 'Iran', 'Iraq', 'Israel', 'Jordan', 'Kuwait', 'Lebanon', 'Syria', 'Yemen', 'Bahrain', 'Palestine']
balkans = ['Albania', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Greece', 'Kosovo', 'Montenegro', 'North Macedonia', 'Romania', 'Serbia', 'Slovenia','Greece', 'Turkey', 'Bulgaria', 'Serbia', 'Croatia', 'Bosnia and Herzegovina', 'Montenegro', 'North Macedonia', 'Albania', 'Kosovo', 'Romania']
south_america = ['Barranquilla', 'Belo Horizonte', 'Jojia','Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela','Brazil', 'Colombia', 'Argentina', 'Chile', 'Peru', 'Ecuador', 'Venezuela', 'Bolivia', 'Paraguay', 'Uruguay', 'Suriname', 'Guyana']
north_america = ['Anguilla', 'Antigua', 'Aruba', 'Bahamas', 'Barbados', 'Belize', 'Bermuda', 'Cayman Islands', 'Costa Rica', 'Cuba', 'CuraÃ§ao', 'Dominica', 'El Salvador', 'Grenada', 'Guadeloupe', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Martinique', 'Mexico', 'Montserrat', 'Nicaragua', 'Panama', 'Puerto Rico', 'Saint Kitts', 'Saint Lucia', 'Saint Martin', 'Sint Maarten', 'Trinidad and Tobago', 'Turks and Caicos Islands', 'Virgin Islands','United States', 'Mexico', 'Canada', 'Guatemala', 'Panama']
eastern_europe = ['Czech', 'Macedonia','Russia', 'Ukraine', 'Belarus', 'Moldova', 'Romania', 'Hungary', 'Poland', 'Czech Republic', 'Slovakia', 'Bulgaria', 'Serbia', 'Croatia', 'Bosnia and Herzegovina', 'Montenegro', 'Albania', 'North Macedonia', 'Kosovo', 'Slovenia', 'Latvia', 'Lithuania', 'Estonia']
western_europe = ['Denmark', 'Finland', 'Iceland', 'Isle of Man', 'Jersey', 'Malta', 'Monaco', 'Norway', 'Sweden','United Kingdom', 'France', 'Germany', 'Italy', 'Spain', 'Netherlands', 'Belgium', 'Luxembourg', 'Switzerland', 'Austria', 'Portugal', 'Ireland']
east_asia = ['Bangladesh', 'Brunei', 'Cambodia', 'Indonesia', 'Laos', 'Malaysia', 'Maldives', 'Myanmar', 'Nepal', 'Philippines', 'Singapore', 'Sri Lanka', 'Thailand', 'VietNam','Korea','China', 'Japan', 'South Korea', 'North Korea', 'Taiwan', 'Mongolia', 'Hong Kong', 'Macau']
west_asia = ['Kazakhstan', 'Pakistan', 'Turkmenistan', 'Uzbekistan','Bahrain','Turkey', 'Cyprus', 'Georgia', 'Armenia', 'Azerbaijan', 'Iran', 'Iraq', 'Syria', 'Lebanon', 'Israel', 'Jordan', 'Saudi Arabia', 'Yemen', 'United Arab Emirates', 'Qatar', 'Bahrain', 'Kuwait', 'Oman']
west_africa = ['Nigeria', 'Benin', 'Ghana', 'Guinea', 'Liberia', 'Mali', 'Niger', 'Senegal', 'Sierra Leone', 'Togo', 'Burkina Faso', 'Cape Verde', 'Gambia', 'Guinea-Bissau', 'Ivory Coast']
east_africa = ['Ethiopia', 'Kenya', 'Tanzania', 'Uganda', 'Burundi', 'Rwanda', 'Somalia', 'South Sudan', 'Sudan', 'Djibouti', 'Eritrea', 'Madagascar', 'Mauritius', 'Seychelles', 'Comoros']
southern_africa = ['Swaziland','South Africa', 'Angola', 'Botswana', 'Lesotho', 'Malawi', 'Mozambique', 'Namibia', 'Eswatini', 'Zambia', 'Zimbabwe']
central_africa = ['Congo','Democratic Republic of the Congo', 'Republic of the Congo', 'Gabon', 'Equatorial Guinea', 'Sao Tome and Principe', 'Cameroon', 'Central African Republic', 'Chad']
north_africa = ['Mauritania','Gujarat','Egypt', 'Algeria', 'Libya', 'Morocco', 'Tunisia', 'Western Sahara']
oceania = ['Australia', 'Fiji', 'Guam', 'New Zealand']
origin_america = south_america+north_america
origin_europe = eastern_europe+western_europe
origin_asia = east_asia+west_asia
origin_africa = west_africa+east_africa+southern_africa+central_africa+north_africa

In [156]:
for i in middle_east:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'middle_east'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'middle_east'] = 1
for i in balkans:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'balkans'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'balkans'] = 1
for i in south_america:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'south_america'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'south_america'] = 1
for i in north_america:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'north_america'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'north_america'] = 1
for i in eastern_europe:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'eastern_europe'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'eastern_europe'] = 1
for i in western_europe:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'western_europe'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'western_europe'] = 1
for i in east_asia:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'east_asia'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'east_asia'] = 1
for i in west_asia:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'west_asia'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'west_asia'] = 1
for i in west_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'west_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'west_africa'] = 1
for i in east_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'east_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'east_africa'] = 1
for i in southern_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'southern_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'southern_africa'] = 1
for i in central_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'central_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'central_africa'] = 1
for i in north_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'north_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'north_africa'] = 1
for i in oceania:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'oceania'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'oceania'] = 1
for i in origin_america:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'origin_america'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'origin_america'] = 1
for i in origin_europe:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'origin_europe'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'origin_europe'] = 1
for i in origin_asia:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'origin_asia'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'origin_asia'] = 1
for i in origin_africa:
    df_train.loc[df_train['customer_country'].str.contains(i, case=True, na=False), 'origin_africa'] = 1
    df_test.loc[df_test['customer_country'].str.contains(i, case=True, na=False), 'origin_africa'] = 1

In [157]:
df_train[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']] = df_train[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']].fillna(0)
df_test[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']] = df_test[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']].fillna(0)

df_train[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']] = df_train[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']].astype(object)
df_test[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']] = df_test[['middle_east', 'balkans', 'south_america', 'north_america', 'eastern_europe', 'western_europe', 'east_asia', 'west_asia', 'west_africa', 'east_africa', 'southern_africa', 'central_africa', 'north_africa', 'oceania', 'origin_america', 'origin_europe', 'origin_asia', 'origin_africa']].astype(object)

***

## ***com_reg_ver_win_rate_zegob***
com_reg_ver_win_rate 제곱

In [158]:
df_train['com_reg_ver_win_rate_zegob'] = df_train['com_reg_ver_win_rate'] * df_train['com_reg_ver_win_rate']
df_test['com_reg_ver_win_rate_zegob'] = df_test['com_reg_ver_win_rate'] * df_test['com_reg_ver_win_rate']

***

## ***as_strategic_ver***
(도메인 지식) 특정 사업부(Business Unit이 AS일 때), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여

In [159]:
VL_1_lst = df_train[df_train['it_strategic_ver']!=0]['business_area'].unique().tolist()

In [160]:
df_train['as_strategic_ver'] = 0
df_test['as_strategic_ver'] = 0

In [161]:
df_train.loc[(df_train['business_area'].isin(VL_1_lst)) & (df_train['business_unit'] == 'AS'), 'as_strategic_ver'] = 1
df_test.loc[(df_test['business_area'].isin(VL_1_lst)) & (df_test['business_unit'] == 'AS'), 'as_strategic_ver'] = 1

## ***asit_strategic_ver***
as_strategic_ver이나 it_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현

In [162]:
df_train['asit_strategic_ver'] = 0
df_test['asit_strategic_ver'] = 0

In [163]:
df_train.loc[(df_train['as_strategic_ver'] == 1) | (df_train['it_strategic_ver'] == 1), 'asit_strategic_ver'] = 1
df_test.loc[(df_test['as_strategic_ver'] == 1) | (df_test['it_strategic_ver'] == 1), 'asit_strategic_ver'] = 1

## ***asid_strategic_ver***
as_strategic_ver이나 id_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현

In [164]:
df_train['asid_strategic_ver'] = 0
df_test['asid_strategic_ver'] = 0

In [165]:
df_train.loc[(df_train['as_strategic_ver'] == 1) | (df_train['id_strategic_ver'] == 1), 'asid_strategic_ver'] = 1
df_test.loc[(df_test['as_strategic_ver'] == 1) | (df_test['id_strategic_ver'] == 1), 'asid_strategic_ver'] = 1

## ***iditas_strategic_ver***
Id_strategic_ver이나 it_strategic_ver이나 as_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현

In [166]:
df_train['iditas_strategic_ver'] = 0
df_test['iditas_strategic_ver'] = 0

In [167]:
df_train.loc[(df_train['as_strategic_ver'] == 1) | (df_train['id_strategic_ver'] == 1) | (df_train['it_strategic_ver'] == 1), 'iditas_strategic_ver'] = 1
df_test.loc[(df_test['as_strategic_ver'] == 1) | (df_test['id_strategic_ver'] == 1) | (df_train['it_strategic_ver'] == 1), 'iditas_strategic_ver'] = 1

***

## ***New_Customer***
신규고객 (historical_existing_cnt 이 0인지)

In [168]:
#Historical_Existing_Cnt 신규고객 여부
df_train['New_Customer'] = 0
df_test['New_Customer'] = 0

df_train.loc[df_train['historical_existing_cnt'] == 0, 'New_Customer'] = 1
df_train.loc[df_train['historical_existing_cnt'] > 0, 'New_Customer'] = 0
df_train.loc[df_train['historical_existing_cnt'].isna(), 'New_Customer'] = -1

df_test.loc[df_test['historical_existing_cnt'] == 0, 'New_Customer'] = 1
df_test.loc[df_test['historical_existing_cnt'] > 0, 'New_Customer'] = 0
df_test.loc[df_test['historical_existing_cnt'].isna(), 'New_Customer'] = -1

In [169]:
df_train['New_Customer'] = df_train['New_Customer'].astype(object)
df_test['New_Customer'] = df_test['New_Customer'].astype(object)

***

## ***is_endcustomer***
최종 고객

In [170]:
df_train['is_endcustomer'] =0
df_test['is_endcustomer'] =0

In [171]:
df_train.loc[df_train['customer_type'] == 'End-Customer','is_endcustomer'] =1
df_test.loc[df_test['customer_type'] == 'End-Customer','is_endcustomer'] =1

***

## ***is_Partner***
유통인지

In [172]:
df_train['is_Partner'] =0
df_test['is_Partner'] =0

In [173]:
df_train.loc[df_train['customer_type'].str.contains('Partner'), 'is_Partner'] = 1
df_test.loc[df_test['customer_type'].str.contains('Partner'), 'is_Partner'] = 1

***

## ***HEC_Group***
Historical_Existing_Cnt 그룹핑 (historical_existing_cnt)

In [174]:
#Historical_Existing_Cnt 횟수별 그룹핑
df_train['HEC_Group'] = "-"
df_test['HEC_Group'] = "-"

df_train.loc[df_train['historical_existing_cnt'] >= 0, 'HEC_Group'] = "newbie"
df_train.loc[df_train['historical_existing_cnt'] >= 10, 'HEC_Group'] = "Familiar"
df_train.loc[df_train['historical_existing_cnt'] >= 100, 'HEC_Group'] = "mania"

df_test.loc[df_test['historical_existing_cnt'] >= 0, 'HEC_Group'] = "Newbie"
df_test.loc[df_test['historical_existing_cnt'] >= 10, 'HEC_Group'] = "Familiar"
df_test.loc[df_test['historical_existing_cnt'] >= 100, 'HEC_Group'] = "Mania"

***

## ***lead_desc_length***

log, sqrt 취하기

In [175]:
df_train['log_lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['log_lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [176]:
df_train['sqrt_lead_desc_length'] = np.sqrt(df_train['lead_desc_length'])
df_test['sqrt_lead_desc_length'] = np.sqrt(df_test['lead_desc_length'])

***

## ***product_category***

In [177]:
Commercial_Display = ['oled signage', 'led signage','video wall signage','interactive signage','high brightness signage',
                      'special signage', 'standard signage', 'hotel tv', 'hospital tv', 'accessories', 'software solution',
                      'signage care solution', 'webos', 'pro:centric', 'one:quick series', 'interactive digital board']

IT_PRODUCTS = ['monitor', 'laptop', 'projector', 'cloud device', 'medical display']

HVAC_ESS = ['control', 'ventilation', 'vrf', 'multi-split', 'single-split', 'chiller', 'heating','ess']

Other = ['other' , 'others', 'etc.']

In [178]:
# Commercial_Display

for i in Commercial_Display:
    df_train.loc[df_train['product_category'].str.contains(i, case=False, na=False), 'Commercial_Display'] = 1
    df_test.loc[df_test['product_category'].str.contains(i, case=False, na=False), 'Commercial_Display'] = 1
    
df_train['Commercial_Display'] = df_train['Commercial_Display'].fillna(0)
df_test['Commercial_Display'] = df_test['Commercial_Display'].fillna(0)

In [179]:
# IT_PRODUCTS

for i in IT_PRODUCTS:
    df_train.loc[df_train['product_category'].str.contains(i, case=False, na=False), 'IT_PRODUCTS'] = 1
    df_test.loc[df_test['product_category'].str.contains(i, case=False, na=False), 'IT_PRODUCTS'] = 1
    
df_train['IT_PRODUCTS'] = df_train['IT_PRODUCTS'].fillna(0)
df_test['IT_PRODUCTS'] = df_test['IT_PRODUCTS'].fillna(0)

In [180]:
# HVAC_ESS

for i in HVAC_ESS:
    df_train.loc[df_train['product_category'].str.contains(i, case=False, na=False), 'HVAC_ESS'] = 1
    df_test.loc[df_test['product_category'].str.contains(i, case=False, na=False), 'HVAC_ESS'] = 1
    
df_train['HVAC_ESS'] = df_train['HVAC_ESS'].fillna(0)
df_test['HVAC_ESS'] = df_test['HVAC_ESS'].fillna(0)

In [181]:
# other

for i in Other:
    df_train.loc[df_train['product_category'].str.contains(i, case=False, na=False), 'Other'] = 1
    df_test.loc[df_test['product_category'].str.contains(i, case=False, na=False), 'Other'] = 1
    
df_train['Other'] = df_train['Other'].fillna(0)
df_test['Other'] = df_test['Other'].fillna(0)

***

## ***Respone_Corporate_Continent***
Respone_Corporate 대륙별 그룹핑

In [182]:
#Respone_Corporate 대륙별 그룹핑
df_train['Respone_Corporate_Continent'] = "-"
df_test['Respone_Corporate_Continent'] = "-"

Asia = ['LGEIL','LGEPH','LGEVH','LGETK','LGEAP','LGESL','LGEIN','LGETH','LGEML','LGETT','LGEJP']
Middle_East = ['LGEGF','LGESJ','LGEEG','LGELF','LGEAS','LGEMC','LGEIR']
China = ['LGEHK','LGECH']
North_America = ['LGEUS','LGEMS','LGECI']
South_America = ['LGESP','LGECB','LGECL','LGEPS','LGEPR','LGEAR','LGEAG']
Europe = ['LGEUK','LGEIS','LGEDG','LGEPL','LGEES','LGEMK','LGEFS','LGEPT','LGEBN','LGEHS','LGESW','LGERO','LGEEB','LGERA','LGECZ','LGELA','LGEUR','LGEBT']
Africa = ['LGEAF','LGESA','LGEEF','LGEYK']
Combination = ['LGEKR']

df_train.loc[df_train['response_corporate'].isin(Asia), 'Respone_Corporate_Continent'] = "Asia"
df_train.loc[df_train['response_corporate'].isin(Middle_East), 'Respone_Corporate_Continent'] = "Middle_East"
df_train.loc[df_train['response_corporate'].isin(China), 'Respone_Corporate_Continent'] = "China"
df_train.loc[df_train['response_corporate'].isin(North_America), 'Respone_Corporate_Continent'] = "North_America"
df_train.loc[df_train['response_corporate'].isin(South_America), 'Respone_Corporate_Continent'] = "South_America"
df_train.loc[df_train['response_corporate'].isin(Europe), 'Respone_Corporate_Continent'] = "Europe"
df_train.loc[df_train['response_corporate'].isin(Africa), 'Respone_Corporate_Continent'] = "Africa"
df_train.loc[df_train['response_corporate'].isin(Combination), 'Respone_Corporate_Continent'] = "Combination"

df_test.loc[df_test['response_corporate'].isin(Asia), 'Respone_Corporate_Continent'] = "Asia"
df_test.loc[df_test['response_corporate'].isin(Middle_East), 'Respone_Corporate_Continent'] = "Middle_East"
df_test.loc[df_test['response_corporate'].isin(China), 'Respone_Corporate_Continent'] = "China"
df_test.loc[df_test['response_corporate'].isin(North_America), 'Respone_Corporate_Continent'] = "North_America"
df_test.loc[df_test['response_corporate'].isin(South_America), 'Respone_Corporate_Continent'] = "South_America"
df_test.loc[df_test['response_corporate'].isin(Europe), 'Respone_Corporate_Continent'] = "Europe"
df_test.loc[df_test['response_corporate'].isin(Africa), 'Respone_Corporate_Continent'] = "Africa"
df_test.loc[df_test['response_corporate'].isin(Combination), 'Respone_Corporate_Continent'] = "Combination"

***

## ***Expected_Timeline_Quantification***
Expected_Timeline별 수치화

In [183]:
#Expected_Timeline별 수치화
df_train['Expected_Timeline_Quantification'] = 0
df_test['Expected_Timeline_Quantification'] = 0

df_train.loc[df_train['expected_timeline'] == 'incomplete', 'Expected_Timeline_Quantification'] = 0
df_train.loc[df_train['expected_timeline'] == 'less than 3 months', 'Expected_Timeline_Quantification'] = 1
df_train.loc[df_train['expected_timeline'] == '3 months ~ 6 months', 'Expected_Timeline_Quantification'] = 2
df_train.loc[df_train['expected_timeline'] == '6 months ~ 9 months', 'Expected_Timeline_Quantification'] = 3
df_train.loc[df_train['expected_timeline'] == '9 months ~ 1 year', 'Expected_Timeline_Quantification'] = 4
df_train.loc[df_train['expected_timeline'] == 'more than a year', 'Expected_Timeline_Quantification'] = 5

df_test.loc[df_test['expected_timeline'] == 'incomplete', 'Expected_Timeline_Quantification'] = 0
df_test.loc[df_test['expected_timeline'] == 'less than 3 months', 'Expected_Timeline_Quantification'] = 1
df_test.loc[df_test['expected_timeline'] == '3 months ~ 6 months', 'Expected_Timeline_Quantification'] = 2
df_test.loc[df_test['expected_timeline'] == '6 months ~ 9 months', 'Expected_Timeline_Quantification'] = 3
df_test.loc[df_test['expected_timeline'] == '9 months ~ 1 year', 'Expected_Timeline_Quantification'] = 4
df_test.loc[df_test['expected_timeline'] == 'more than a year', 'Expected_Timeline_Quantification'] = 5

***

## ***Corporate_Weight***
Response_Corporate_Continent별 구매액수별 가중치

In [184]:
#Response_Corporate_Continent별 구매액수별 가중치
df_train['Corporate_Weight'] = 0.0
df_test['Corporate_Weight'] = 0.0

df_train.loc[df_train['response_corporate'].isin(Asia), 'Corporate_Weight'] = 8.8
df_train.loc[df_train['response_corporate'].isin(Middle_East), 'Corporate_Weight'] = 0.1
df_train.loc[df_train['response_corporate'].isin(China), 'Corporate_Weight'] = 5.4
df_train.loc[df_train['response_corporate'].isin(North_America), 'Corporate_Weight'] = 5.7
df_train.loc[df_train['response_corporate'].isin(South_America), 'Corporate_Weight'] = 5.7
df_train.loc[df_train['response_corporate'].isin(Europe), 'Corporate_Weight'] = 3.2
df_train.loc[df_train['response_corporate'].isin(Africa), 'Corporate_Weight'] = 0.1
df_train.loc[df_train['response_corporate'].isin(Combination), 'Corporate_Weight'] = 4.1

df_test.loc[df_test['response_corporate'].isin(Asia), 'Corporate_Weight'] = 8.8
df_test.loc[df_test['response_corporate'].isin(Middle_East), 'Corporate_Weight'] = 0.1
df_test.loc[df_test['response_corporate'].isin(China), 'Corporate_Weight'] = 5.4
df_test.loc[df_test['response_corporate'].isin(North_America), 'Corporate_Weight'] = 5.7
df_test.loc[df_test['response_corporate'].isin(South_America), 'Corporate_Weight'] = 5.7
df_test.loc[df_test['response_corporate'].isin(Europe), 'Corporate_Weight'] = 3.2
df_test.loc[df_test['response_corporate'].isin(Africa), 'Corporate_Weight'] = 0.1
df_test.loc[df_test['response_corporate'].isin(Combination), 'Corporate_Weight'] = 4.1

***

## ***Business_Area_Main***
Business_Area별 영업성공률에 기반한 그룹핑

In [185]:
#Business_Area별 영업성공률에 기반한 그룹핑
df_train['Business_Area_Main'] = 0
df_test['Business_Area_Main'] = 0

main_area = ['hospital & health care','power plant / renewable energy']

df_train.loc[df_train['business_area'].isin(main_area), 'Business_Area_Main'] = 1
df_train.loc[~df_train['business_area'].isin(main_area), 'Business_Area_Main'] = 0
df_train.loc[df_train['business_area'] == 'Not entered', 'Business_Area_Main'] = -1

df_test.loc[df_test['business_area'].isin(main_area), 'Business_Area_Main'] = 1
df_test.loc[~df_test['business_area'].isin(main_area), 'Business_Area_Main'] = 0
df_test.loc[df_test['business_area'] == 'Not entered', 'Business_Area_Main'] = -1

In [186]:
df_train['Business_Area_Main'] = df_train['Business_Area_Main'].astype(object)
df_test['Business_Area_Main'] = df_test['Business_Area_Main'].astype(object)

***

## ***Business_Subarea_Main***
Business_Subarea별 영업성공률에 기반한 그룹핑

In [187]:
#Business_Subarea별 영업성공률에 기반한 그룹핑
df_train['Business_Subarea_Main'] = 0
df_test['Business_Subarea_Main'] = 0

df_train.loc[df_train['business_subarea'] == 'Health', 'Business_Subarea_Main'] = 1
df_train.loc[df_train['business_subarea'] != 'Health', 'Business_Subarea_Main'] = 0
df_train.loc[df_train['business_subarea'] == 'Not entered', 'Business_Subarea_Main'] = -1

df_test.loc[df_test['business_subarea'] == 'Health', 'Business_Subarea_Main'] = 1
df_test.loc[df_test['business_subarea'] != 'Health', 'Business_Subarea_Main'] = 0
df_test.loc[df_test['business_subarea'] == 'Not entered', 'Business_Subarea_Main'] = -1

In [188]:
df_train['Business_Subarea_Main'] = df_train['Business_Subarea_Main'].astype(object)
df_test['Business_Subarea_Main'] = df_test['Business_Subarea_Main'].astype(object)

***

## ***🍕추가적인 피쳐 생성🍕***

## ***enterprise_country***
enterprise와 country 묶은 카테고리컬 feature

In [189]:
df_train['enterprise_country'] = df_train['customer_country'] + '_' + df_train['enterprise']
df_test['enterprise_country'] = df_test['customer_country'] + '_' + df_test['enterprise']

***

## ***country_corp***
country와 corporate 묶은 카테고리컬 feature

In [190]:
df_train['country_corp'] = df_train['customer_country'] + '_' + df_train['response_corporate']
df_test['country_corp'] = df_test['customer_country'] + '_' + df_test['response_corporate']

***

## ***high_level customer_postion***

In [191]:
df_train['high_level'] = 0
df_train.loc[df_train['customer_position']=='ceo/founder','high_level'] = 1
df_train.loc[df_train['customer_position']=='vice president','high_level'] = 1
df_train.loc[df_train['customer_position']=='manager','high_level']  = 1
df_train.loc[df_train['customer_position']=='director','high_level'] = 1

df_test['high_level'] = 0
df_test.loc[df_test['customer_position']=='ceo/founder','high_level'] = 1
df_test.loc[df_test['customer_position']=='vice president','high_level'] = 1
df_test.loc[df_test['customer_position']=='manager','high_level']  = 1
df_test.loc[df_test['customer_position']=='director','high_level'] = 1

***

## ***business_unit_area***
business_unit과 business_area 묶은 카테고리컬 feature

In [192]:
df_train['business_unit_area'] = df_train['business_unit'] + '_' + df_train['business_area']
df_test['business_unit_area'] = df_test['business_unit'] + '_' + df_test['business_area']

***

## ***corporate_unit***
response_corporate와 business_unit 묶은 카테고리컬 feature

In [193]:
df_train['corporate_unit'] = df_train['response_corporate'] + '_' + df_train['business_unit']
df_test['corporate_unit'] = df_test['response_corporate'] + '_' + df_test['business_unit']

***

## ***job_product*** (job + category)

In [194]:
df_train['job_product'] = df_train['customer_job'] +'_'+ df_train['product_category']
df_test['job_product'] = df_test['customer_job'] +'_'+ df_test['product_category']

***

## ***country_corp_status 1 2***
country와 corporate의 일치여부

In [195]:
df_train['country_corp_status'] = 0
df_test['country_corp_status'] = 0

In [196]:
df_train.loc[df_train['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status'] = None
df_test.loc[df_test['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status'] = None

In [197]:
LGEAF_lst = ['Nigeria', 'Benin', 'Ghana', 'Congo', 'Togo', 'Mauritania', 'Senegal', 'Ivory Coast', 'Mali',
       'Central African Republic', 'Angola', 'Sierra Leone', 'Cameroon', 'Burkina Faso', 'Gambia', 'Gabon', 'Guinea', 'Liberia']

In [198]:
df_train.loc[(df_train['customer_country'].isin(LGEAF_lst)) & (df_train['response_corporate'] == 'LGEAF'), 'country_corp_status'] = 1

df_train.loc[(df_train['customer_country'] == 'India') & (df_train['response_corporate'] == 'LGEIL'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Brazil') & (df_train['response_corporate'] == 'LGESP'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'United States') & (df_train['response_corporate'] == 'LGEUS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Mexico') & (df_train['response_corporate'] == 'LGEMS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Philippines') & (df_train['response_corporate'] == 'LGEPH'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'United Arab Emirates') & (df_train['response_corporate'] == 'LGEGF'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Colombia') & (df_train['response_corporate'] == 'LGECB'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'United Kingdom') & (df_train['response_corporate'] == 'LGEUK'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Saudi Arabia') & (df_train['response_corporate'] == 'LGESJ'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Chile') & (df_train['response_corporate'] == 'LGECL'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Panama') & (df_train['response_corporate'] == 'LGEPS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Italy') & (df_train['response_corporate'] == 'LGEIS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Peru') & (df_train['response_corporate'] == 'LGEPR'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Germany') & (df_train['response_corporate'] == 'LGEDG'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Poland') & (df_train['response_corporate'] == 'LGEPL'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Egypt') & (df_train['response_corporate'] == 'LGEEG'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'VietNam') & (df_train['response_corporate'] == 'LGEVH'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Spain') & (df_train['response_corporate'] == 'LGEES'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Turkey') & (df_train['response_corporate'] == 'LGETK'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Hong Kong') & (df_train['response_corporate'] == 'LGEHK'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Australia') & (df_train['response_corporate'] == 'LGEAP'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Singapore') & (df_train['response_corporate'] == 'LGESL'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Hungary') & (df_train['response_corporate'] == 'LGEMK'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'France') & (df_train['response_corporate'] == 'LGEFS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Indonesia') & (df_train['response_corporate'] == 'LGEIN'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Iraq') & (df_train['response_corporate'] == 'LGELF'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'South Africa') & (df_train['response_corporate'] == 'LGESA'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Canada') & (df_train['response_corporate'] == 'LGECI'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Thailand') & (df_train['response_corporate'] == 'LGETH'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Portugal') & (df_train['response_corporate'] == 'LGEPT'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Malaysia') & (df_train['response_corporate'] == 'LGEML'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Netherlands') & (df_train['response_corporate'] == 'LGEBN'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Israel') & (df_train['response_corporate'] == 'LGEYK'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'China') & (df_train['response_corporate'] == 'LGECH'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Greece') & (df_train['response_corporate'] == 'LGEHS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Taiwan') & (df_train['response_corporate'] == 'LGETT'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Japan') & (df_train['response_corporate'] == 'LGEJP'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Algeria') & (df_train['response_corporate'] == 'LGEAS'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Sweden') & (df_train['response_corporate'] == 'LGESW'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Morocco') & (df_train['response_corporate'] == 'LGEMC'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Romania') & (df_train['response_corporate'] == 'LGERO'), 'country_corp_status'] = 1

df_train['country_corp_status'] = np.where(
    (df_train['customer_country'] == 'Argentina') & ~(df_train['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_train['country_corp_status']
)
# df_train.loc[(df_train['customer_country'] == 'Argentina') & (df_train['response_corporate'] == 'LGEAR'), 'country_corp_status'] = 1
# df_train.loc[(df_train['customer_country'] == 'Argentina') & (df_train['response_corporate'] == 'LGEAG'), 'country_corp_status'] = 1

df_train.loc[(df_train['customer_country'] == 'Russia') & (df_train['response_corporate'] == 'LGERA'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Czech') & (df_train['response_corporate'] == 'LGECZ'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Ukraine') & (df_train['response_corporate'] == 'LGEUR'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Iran') & (df_train['response_corporate'] == 'LGEIR'), 'country_corp_status'] = 1
df_train.loc[(df_train['customer_country'] == 'Portugal') & (df_train['response_corporate'] == 'LGEBT'), 'country_corp_status'] = 1

In [199]:
df_test.loc[(df_test['customer_country'].isin(LGEAF_lst)) & (df_test['response_corporate'] == 'LGEAF'), 'country_corp_status'] = 1

df_test.loc[(df_test['customer_country'] == 'India') & (df_test['response_corporate'] == 'LGEIL'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Brazil') & (df_test['response_corporate'] == 'LGESP'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'United States') & (df_test['response_corporate'] == 'LGEUS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Mexico') & (df_test['response_corporate'] == 'LGEMS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Philippines') & (df_test['response_corporate'] == 'LGEPH'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'United Arab Emirates') & (df_test['response_corporate'] == 'LGEGF'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Colombia') & (df_test['response_corporate'] == 'LGECB'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'United Kingdom') & (df_test['response_corporate'] == 'LGEUK'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Saudi Arabia') & (df_test['response_corporate'] == 'LGESJ'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Chile') & (df_test['response_corporate'] == 'LGECL'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Panama') & (df_test['response_corporate'] == 'LGEPS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Italy') & (df_test['response_corporate'] == 'LGEIS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Peru') & (df_test['response_corporate'] == 'LGEPR'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Germany') & (df_test['response_corporate'] == 'LGEDG'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Poland') & (df_test['response_corporate'] == 'LGEPL'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Egypt') & (df_test['response_corporate'] == 'LGEEG'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'VietNam') & (df_test['response_corporate'] == 'LGEVH'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Spain') & (df_test['response_corporate'] == 'LGEES'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Turkey') & (df_test['response_corporate'] == 'LGETK'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Hong Kong') & (df_test['response_corporate'] == 'LGEHK'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Australia') & (df_test['response_corporate'] == 'LGEAP'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Singapore') & (df_test['response_corporate'] == 'LGESL'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Hungary') & (df_test['response_corporate'] == 'LGEMK'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'France') & (df_test['response_corporate'] == 'LGEFS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Indonesia') & (df_test['response_corporate'] == 'LGEIN'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Iraq') & (df_test['response_corporate'] == 'LGELF'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'South Africa') & (df_test['response_corporate'] == 'LGESA'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Canada') & (df_test['response_corporate'] == 'LGECI'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Thailand') & (df_test['response_corporate'] == 'LGETH'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Portugal') & (df_test['response_corporate'] == 'LGEPT'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Malaysia') & (df_test['response_corporate'] == 'LGEML'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Netherlands') & (df_test['response_corporate'] == 'LGEBN'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Israel') & (df_test['response_corporate'] == 'LGEYK'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'China') & (df_test['response_corporate'] == 'LGECH'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Greece') & (df_test['response_corporate'] == 'LGEHS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Taiwan') & (df_test['response_corporate'] == 'LGETT'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Japan') & (df_test['response_corporate'] == 'LGEJP'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Algeria') & (df_test['response_corporate'] == 'LGEAS'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Sweden') & (df_test['response_corporate'] == 'LGESW'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Morocco') & (df_test['response_corporate'] == 'LGEMC'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Romania') & (df_test['response_corporate'] == 'LGERO'), 'country_corp_status'] = 1

df_test['country_corp_status'] = np.where(
    (df_test['customer_country'] == 'Argentina') & ~(df_test['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_test['country_corp_status']
)
# df_test.loc[(df_test['customer_country'] == 'Argentina') & (df_test['response_corporate'] == 'LGEAR'), 'country_corp_status'] = 1
# df_test.loc[(df_test['customer_country'] == 'Argentina') & (df_test['response_corporate'] == 'LGEAG'), 'country_corp_status'] = 1

df_test.loc[(df_test['customer_country'] == 'Russia') & (df_test['response_corporate'] == 'LGERA'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Czech') & (df_test['response_corporate'] == 'LGECZ'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Ukraine') & (df_test['response_corporate'] == 'LGEUR'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Iran') & (df_test['response_corporate'] == 'LGEIR'), 'country_corp_status'] = 1
df_test.loc[(df_test['customer_country'] == 'Portugal') & (df_test['response_corporate'] == 'LGEBT'), 'country_corp_status'] = 1

##### 추가생성

In [200]:
df_train['country_corp_status_1'] = 0
df_test['country_corp_status_1'] = 0

df_train['country_corp_status_2'] = 0
df_test['country_corp_status_2'] = 0

In [201]:
df_train.loc[df_train['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status_1'] = None
df_test.loc[df_test['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status_1'] = None

df_train.loc[(~df_train['customer_country'].isin(LGEAF_lst)) & (df_train['response_corporate'] == 'LGEAF'), 'country_corp_status_1'] = 1

df_train.loc[(df_train['customer_country'] != 'India') & (df_train['response_corporate'] == 'LGEIL'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Brazil') & (df_train['response_corporate'] == 'LGESP'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'United States') & (df_train['response_corporate'] == 'LGEUS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Mexico') & (df_train['response_corporate'] == 'LGEMS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Philippines') & (df_train['response_corporate'] == 'LGEPH'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'United Arab Emirates') & (df_train['response_corporate'] == 'LGEGF'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Colombia') & (df_train['response_corporate'] == 'LGECB'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'United Kingdom') & (df_train['response_corporate'] == 'LGEUK'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Saudi Arabia') & (df_train['response_corporate'] == 'LGESJ'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Chile') & (df_train['response_corporate'] == 'LGECL'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Panama') & (df_train['response_corporate'] == 'LGEPS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Italy') & (df_train['response_corporate'] == 'LGEIS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Peru') & (df_train['response_corporate'] == 'LGEPR'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Germany') & (df_train['response_corporate'] == 'LGEDG'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Poland') & (df_train['response_corporate'] == 'LGEPL'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Egypt') & (df_train['response_corporate'] == 'LGEEG'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'VietNam') & (df_train['response_corporate'] == 'LGEVH'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Spain') & (df_train['response_corporate'] == 'LGEES'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Turkey') & (df_train['response_corporate'] == 'LGETK'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Hong Kong') & (df_train['response_corporate'] == 'LGEHK'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Australia') & (df_train['response_corporate'] == 'LGEAP'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Singapore') & (df_train['response_corporate'] == 'LGESL'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Hungary') & (df_train['response_corporate'] == 'LGEMK'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'France') & (df_train['response_corporate'] == 'LGEFS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Indonesia') & (df_train['response_corporate'] == 'LGEIN'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Iraq') & (df_train['response_corporate'] == 'LGELF'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'South Africa') & (df_train['response_corporate'] == 'LGESA'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Canada') & (df_train['response_corporate'] == 'LGECI'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Thailand') & (df_train['response_corporate'] == 'LGETH'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Portugal') & (df_train['response_corporate'] == 'LGEPT'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Malaysia') & (df_train['response_corporate'] == 'LGEML'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Netherlands') & (df_train['response_corporate'] == 'LGEBN'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Israel') & (df_train['response_corporate'] == 'LGEYK'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'China') & (df_train['response_corporate'] == 'LGECH'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Greece') & (df_train['response_corporate'] == 'LGEHS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Taiwan') & (df_train['response_corporate'] == 'LGETT'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Japan') & (df_train['response_corporate'] == 'LGEJP'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Algeria') & (df_train['response_corporate'] == 'LGEAS'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Sweden') & (df_train['response_corporate'] == 'LGESW'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Morocco') & (df_train['response_corporate'] == 'LGEMC'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Romania') & (df_train['response_corporate'] == 'LGERO'), 'country_corp_status_1'] = 1

df_train['country_corp_status_1'] = np.where(
    (df_train['customer_country'] != 'Argentina') & ~(df_train['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_train['country_corp_status_1']
)
# df_train.loc[(df_train['customer_country'] != 'Argentina') & (df_train['response_corporate'] == 'LGEAR'), 'country_corp_status_1'] = 1
# df_train.loc[(df_train['customer_country'] != 'Argentina') & (df_train['response_corporate'] == 'LGEAG'), 'country_corp_status_1'] = 1

df_train.loc[(df_train['customer_country'] != 'Russia') & (df_train['response_corporate'] == 'LGERA'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Czech') & (df_train['response_corporate'] == 'LGECZ'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Ukraine') & (df_train['response_corporate'] == 'LGEUR'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Iran') & (df_train['response_corporate'] == 'LGEIR'), 'country_corp_status_1'] = 1
df_train.loc[(df_train['customer_country'] != 'Portugal') & (df_train['response_corporate'] == 'LGEBT'), 'country_corp_status_1'] = 1

df_test.loc[(df_test['customer_country'].isin(LGEAF_lst)) & (df_test['response_corporate'] == 'LGEAF'), 'country_corp_status_1'] = 1

df_test.loc[(df_test['customer_country'] != 'India') & (df_test['response_corporate'] == 'LGEIL'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Brazil') & (df_test['response_corporate'] == 'LGESP'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'United States') & (df_test['response_corporate'] == 'LGEUS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Mexico') & (df_test['response_corporate'] == 'LGEMS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Philippines') & (df_test['response_corporate'] == 'LGEPH'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'United Arab Emirates') & (df_test['response_corporate'] == 'LGEGF'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Colombia') & (df_test['response_corporate'] == 'LGECB'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'United Kingdom') & (df_test['response_corporate'] == 'LGEUK'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Saudi Arabia') & (df_test['response_corporate'] == 'LGESJ'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Chile') & (df_test['response_corporate'] == 'LGECL'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Panama') & (df_test['response_corporate'] == 'LGEPS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Italy') & (df_test['response_corporate'] == 'LGEIS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Peru') & (df_test['response_corporate'] == 'LGEPR'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Germany') & (df_test['response_corporate'] == 'LGEDG'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Poland') & (df_test['response_corporate'] == 'LGEPL'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Egypt') & (df_test['response_corporate'] == 'LGEEG'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'VietNam') & (df_test['response_corporate'] == 'LGEVH'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Spain') & (df_test['response_corporate'] == 'LGEES'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Turkey') & (df_test['response_corporate'] == 'LGETK'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Hong Kong') & (df_test['response_corporate'] == 'LGEHK'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Australia') & (df_test['response_corporate'] == 'LGEAP'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Singapore') & (df_test['response_corporate'] == 'LGESL'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Hungary') & (df_test['response_corporate'] == 'LGEMK'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'France') & (df_test['response_corporate'] == 'LGEFS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Indonesia') & (df_test['response_corporate'] == 'LGEIN'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Iraq') & (df_test['response_corporate'] == 'LGELF'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'South Africa') & (df_test['response_corporate'] == 'LGESA'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Canada') & (df_test['response_corporate'] == 'LGECI'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Thailand') & (df_test['response_corporate'] == 'LGETH'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Portugal') & (df_test['response_corporate'] == 'LGEPT'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Malaysia') & (df_test['response_corporate'] == 'LGEML'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Netherlands') & (df_test['response_corporate'] == 'LGEBN'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Israel') & (df_test['response_corporate'] == 'LGEYK'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'China') & (df_test['response_corporate'] == 'LGECH'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Greece') & (df_test['response_corporate'] == 'LGEHS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Taiwan') & (df_test['response_corporate'] == 'LGETT'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Japan') & (df_test['response_corporate'] == 'LGEJP'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Algeria') & (df_test['response_corporate'] == 'LGEAS'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Sweden') & (df_test['response_corporate'] == 'LGESW'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Morocco') & (df_test['response_corporate'] == 'LGEMC'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Romania') & (df_test['response_corporate'] == 'LGERO'), 'country_corp_status_1'] = 1

df_test['country_corp_status_1'] = np.where(
    (df_test['customer_country'] != 'Argentina') & ~(df_test['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_test['country_corp_status_1']
)
# df_test.loc[(df_test['customer_country'] != 'Argentina') & (df_test['response_corporate'] == 'LGEAR'), 'country_corp_status_1'] = 1
# df_test.loc[(df_test['customer_country'] != 'Argentina') & (df_test['response_corporate'] == 'LGEAG'), 'country_corp_status_1'] = 1

df_test.loc[(df_test['customer_country'] != 'Russia') & (df_test['response_corporate'] == 'LGERA'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Czech') & (df_test['response_corporate'] == 'LGECZ'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Ukraine') & (df_test['response_corporate'] == 'LGEUR'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Iran') & (df_test['response_corporate'] == 'LGEIR'), 'country_corp_status_1'] = 1
df_test.loc[(df_test['customer_country'] != 'Portugal') & (df_test['response_corporate'] == 'LGEBT'), 'country_corp_status_1'] = 1

In [202]:
df_train.loc[df_train['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status_2'] = None
df_test.loc[df_test['response_corporate'].isin(['LGEKR', 'LGEAF', 'LGEEF', 'LGEEB', 'LGELA']), 'country_corp_status_2'] = None

df_train.loc[(df_train['customer_country'].isin(LGEAF_lst)) & (df_train['response_corporate'] != 'LGEAF'), 'country_corp_status_2'] = 1

df_train.loc[(df_train['customer_country'] == 'India') & (df_train['response_corporate'] != 'LGEIL'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Brazil') & (df_train['response_corporate'] != 'LGESP'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'United States') & (df_train['response_corporate'] != 'LGEUS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Mexico') & (df_train['response_corporate'] != 'LGEMS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Philippines') & (df_train['response_corporate'] != 'LGEPH'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'United Arab Emirates') & (df_train['response_corporate'] != 'LGEGF'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Colombia') & (df_train['response_corporate'] != 'LGECB'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'United Kingdom') & (df_train['response_corporate'] != 'LGEUK'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Saudi Arabia') & (df_train['response_corporate'] != 'LGESJ'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Chile') & (df_train['response_corporate'] != 'LGECL'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Panama') & (df_train['response_corporate'] != 'LGEPS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Italy') & (df_train['response_corporate'] != 'LGEIS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Peru') & (df_train['response_corporate'] != 'LGEPR'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Germany') & (df_train['response_corporate'] != 'LGEDG'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Poland') & (df_train['response_corporate'] != 'LGEPL'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Egypt') & (df_train['response_corporate'] != 'LGEEG'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'VietNam') & (df_train['response_corporate'] != 'LGEVH'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Spain') & (df_train['response_corporate'] != 'LGEES'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Turkey') & (df_train['response_corporate'] != 'LGETK'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Hong Kong') & (df_train['response_corporate'] != 'LGEHK'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Australia') & (df_train['response_corporate'] != 'LGEAP'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Singapore') & (df_train['response_corporate'] != 'LGESL'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Hungary') & (df_train['response_corporate'] != 'LGEMK'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'France') & (df_train['response_corporate'] != 'LGEFS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Indonesia') & (df_train['response_corporate'] != 'LGEIN'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Iraq') & (df_train['response_corporate'] != 'LGELF'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'South Africa') & (df_train['response_corporate'] != 'LGESA'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Canada') & (df_train['response_corporate'] != 'LGECI'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Thailand') & (df_train['response_corporate'] != 'LGETH'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Portugal') & (df_train['response_corporate'] != 'LGEPT'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Malaysia') & (df_train['response_corporate'] != 'LGEML'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Netherlands') & (df_train['response_corporate'] != 'LGEBN'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Israel') & (df_train['response_corporate'] != 'LGEYK'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'China') & (df_train['response_corporate'] != 'LGECH'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Greece') & (df_train['response_corporate'] != 'LGEHS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Taiwan') & (df_train['response_corporate'] != 'LGETT'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Japan') & (df_train['response_corporate'] != 'LGEJP'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Algeria') & (df_train['response_corporate'] != 'LGEAS'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Sweden') & (df_train['response_corporate'] != 'LGESW'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Morocco') & (df_train['response_corporate'] != 'LGEMC'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Romania') & (df_train['response_corporate'] != 'LGERO'), 'country_corp_status_2'] = 1

df_train['country_corp_status_2'] = np.where(
    (df_train['customer_country'] == 'Argentina') & ~(df_train['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_train['country_corp_status_2']
)
# df_train.loc[(df_train['customer_country'] == 'Argentina') & (df_train['response_corporate'] != 'LGEAR'), 'country_corp_status_2'] = 1
# df_train.loc[(df_train['customer_country'] == 'Argentina') & (df_train['response_corporate'] != 'LGEAG'), 'country_corp_status_2'] = 1

df_train.loc[(df_train['customer_country'] == 'Russia') & (df_train['response_corporate'] != 'LGERA'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Czech') & (df_train['response_corporate'] != 'LGECZ'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Ukraine') & (df_train['response_corporate'] != 'LGEUR'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Iran') & (df_train['response_corporate'] != 'LGEIR'), 'country_corp_status_2'] = 1
df_train.loc[(df_train['customer_country'] == 'Portugal') & (df_train['response_corporate'] != 'LGEBT'), 'country_corp_status_2'] = 1

df_test.loc[(df_test['customer_country'].isin(LGEAF_lst)) & (df_test['response_corporate'] != 'LGEAF'), 'country_corp_status_2'] = 1

df_test.loc[(df_test['customer_country'] == 'India') & (df_test['response_corporate'] != 'LGEIL'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Brazil') & (df_test['response_corporate'] != 'LGESP'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'United States') & (df_test['response_corporate'] != 'LGEUS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Mexico') & (df_test['response_corporate'] != 'LGEMS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Philippines') & (df_test['response_corporate'] != 'LGEPH'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'United Arab Emirates') & (df_test['response_corporate'] != 'LGEGF'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Colombia') & (df_test['response_corporate'] != 'LGECB'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'United Kingdom') & (df_test['response_corporate'] != 'LGEUK'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Saudi Arabia') & (df_test['response_corporate'] != 'LGESJ'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Chile') & (df_test['response_corporate'] != 'LGECL'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Panama') & (df_test['response_corporate'] != 'LGEPS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Italy') & (df_test['response_corporate'] != 'LGEIS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Peru') & (df_test['response_corporate'] != 'LGEPR'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Germany') & (df_test['response_corporate'] != 'LGEDG'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Poland') & (df_test['response_corporate'] != 'LGEPL'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Egypt') & (df_test['response_corporate'] != 'LGEEG'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'VietNam') & (df_test['response_corporate'] != 'LGEVH'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Spain') & (df_test['response_corporate'] != 'LGEES'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Turkey') & (df_test['response_corporate'] != 'LGETK'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Hong Kong') & (df_test['response_corporate'] != 'LGEHK'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Australia') & (df_test['response_corporate'] != 'LGEAP'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Singapore') & (df_test['response_corporate'] != 'LGESL'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Hungary') & (df_test['response_corporate'] != 'LGEMK'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'France') & (df_test['response_corporate'] != 'LGEFS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Indonesia') & (df_test['response_corporate'] != 'LGEIN'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Iraq') & (df_test['response_corporate'] != 'LGELF'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'South Africa') & (df_test['response_corporate'] != 'LGESA'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Canada') & (df_test['response_corporate'] != 'LGECI'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Thailand') & (df_test['response_corporate'] != 'LGETH'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Portugal') & (df_test['response_corporate'] != 'LGEPT'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Malaysia') & (df_test['response_corporate'] != 'LGEML'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Netherlands') & (df_test['response_corporate'] != 'LGEBN'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Israel') & (df_test['response_corporate'] != 'LGEYK'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'China') & (df_test['response_corporate'] != 'LGECH'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Greece') & (df_test['response_corporate'] != 'LGEHS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Taiwan') & (df_test['response_corporate'] != 'LGETT'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Japan') & (df_test['response_corporate'] != 'LGEJP'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Algeria') & (df_test['response_corporate'] != 'LGEAS'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Sweden') & (df_test['response_corporate'] != 'LGESW'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Morocco') & (df_test['response_corporate'] != 'LGEMC'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Romania') & (df_test['response_corporate'] != 'LGERO'), 'country_corp_status_2'] = 1

df_test['country_corp_status_2'] = np.where(
    (df_test['customer_country'] != 'Argentina') & ~(df_test['response_corporate'].isin(['LGEAR', 'LGEAG'])),
    1,
    df_test['country_corp_status_2']
)
# df_test.loc[(df_test['customer_country'] == 'Argentina') & (df_test['response_corporate'] != 'LGEAR'), 'country_corp_status_2'] = 1
# df_test.loc[(df_test['customer_country'] == 'Argentina') & (df_test['response_corporate'] != 'LGEAG'), 'country_corp_status_2'] = 1

df_test.loc[(df_test['customer_country'] == 'Russia') & (df_test['response_corporate'] != 'LGERA'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Czech') & (df_test['response_corporate'] != 'LGECZ'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Ukraine') & (df_test['response_corporate'] != 'LGEUR'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Iran') & (df_test['response_corporate'] != 'LGEIR'), 'country_corp_status_2'] = 1
df_test.loc[(df_test['customer_country'] == 'Portugal') & (df_test['response_corporate'] != 'LGEBT'), 'country_corp_status_2'] = 1

***

## ***ratio_rate*** (ver_ 피쳐)

In [203]:
df_train['ratio_rate'] = (df_train['ver_win_ratio_per_bu'] / df_train['ver_win_rate_x']).fillna(0)
df_test['ratio_rate'] = (df_test['ver_win_ratio_per_bu'] / df_test['ver_win_rate_x']).fillna(0)

In [204]:
df_train['log_ratio_rate'] = np.log1p(df_train['ratio_rate'])
df_test['log_ratio_rate'] = np.log1p(df_test['ratio_rate'])

***

## ***수치 Feature***

In [205]:
df_train['gob1'] = df_train['ratio_rate'] * df_train['com_reg_ver_win_rate']
df_train['gob2'] = df_train['ratio_rate'] * df_train['lead_desc_length']
df_train['gob3'] = df_train['ratio_rate'] * df_train['historical_existing_cnt']
df_train['gob4'] = df_train['lead_desc_length'] * df_train['historical_existing_cnt']
df_train['gob5'] = df_train['lead_desc_length'] * df_train['com_reg_ver_win_rate']
df_train['gob6'] = df_train['historical_existing_cnt'] * df_train['com_reg_ver_win_rate']
#df_train['gob5'] = df_train['lead_desc_length'] * df_train['com_reg_ver_win_rate']
df_train['gob6'] = df_train['historical_existing_cnt'] * df_train['com_reg_ver_win_rate']

In [206]:
df_test['gob1'] = df_test['ratio_rate'] * df_test['com_reg_ver_win_rate']
df_test['gob2'] = df_test['ratio_rate'] * df_test['lead_desc_length']
df_test['gob3'] = df_test['ratio_rate'] * df_test['historical_existing_cnt']
df_test['gob4'] = df_test['lead_desc_length'] * df_test['historical_existing_cnt']
df_test['gob5'] = df_test['lead_desc_length'] * df_test['com_reg_ver_win_rate']
df_test['gob6'] = df_test['historical_existing_cnt'] * df_test['com_reg_ver_win_rate']
#df_test['gob5'] = df_test['lead_desc_length'] * df_test['com_reg_ver_win_rate']
df_test['gob6'] = df_test['historical_existing_cnt'] * df_test['com_reg_ver_win_rate']

# ***Kmeans***

In [207]:
#결측치가 없고 Original 수치형 컬럼만 인덱싱하여 클러스터링 진행(K-means)
kmeans_cols = ['bant_submit','lead_desc_length']
kmeans_train = df_train[kmeans_cols].copy()
kmeans_test = df_test[kmeans_cols].copy()

#Elbowpoint를 기준으로 k =3으로 설정
k = 3
#그룹 수, random_state 설정
model = KMeans(n_clusters=k, init='random', n_init=10, max_iter=300, random_state=42)

#정규화된 데이터에 학습
model.fit(kmeans_train)

#클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
kmeans_train['cluster_1'] = model.predict(kmeans_train)
kmeans_test['cluster_1'] = model.predict(kmeans_test)

#클러스터 할당
df_train['cluster_1'] = kmeans_train['cluster_1'].copy()
df_test['cluster_1'] = kmeans_test['cluster_1'].copy()

In [208]:
#결측치가 없고 수치형 컬럼만 인덱싱하여 클러스터링 진행
kmeans_cols = ['bant_submit_zegob','log_lead_desc_length']
kmeans_train = df_train[kmeans_cols].copy()
kmeans_test = df_test[kmeans_cols].copy()

#Elbowpoint를 기준으로 k =3으로 설정
k = 3
#그룹 수, random_state 설정
model = KMeans(n_clusters=k, init='random', n_init=10, max_iter=300, random_state=42)

#정규화된 데이터에 학습
model.fit(kmeans_train)

#클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
kmeans_train['cluster_2'] = model.predict(kmeans_train)
kmeans_test['cluster_2'] = model.predict(kmeans_test)

#클러스터 할당
df_train['cluster_2'] = kmeans_train['cluster_2'].copy()
df_test['cluster_2'] = kmeans_test['cluster_2'].copy()

In [209]:
df_train[['business_unit','customer_type','response_corporate','expected_timeline','business_area']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55742 entries, 0 to 55741
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   business_unit       55742 non-null  object
 1   customer_type       55742 non-null  object
 2   response_corporate  55742 non-null  object
 3   expected_timeline   55742 non-null  object
 4   business_area       55742 non-null  object
dtypes: object(5)
memory usage: 2.1+ MB


In [212]:
len(df_train.columns)

94

In [213]:
len(df_test.columns)

95

In [214]:
print('중복 값 :',len(df_train[df_train.duplicated()]))
df_train = df_train.drop_duplicates().reset_index(drop=True)

중복 값 : 1842


In [215]:
ttmp = df_train.drop(columns = 'is_converted')

In [216]:
print('중복 값 :',len(ttmp[ttmp.duplicated(keep=False)]))

중복 값 : 2


In [217]:
df_train = df_train.drop(ttmp[ttmp.duplicated(keep = False)].index).reset_index(drop=True)

# MAKE XGB & LGB DATA SET

In [218]:
train_xgb = df_train.copy()
test_xgb = df_test.copy()

train_lgb = df_train.copy()
test_lgb = df_test.copy()

***
***
***

## 3. Encoding & Feature Drop

## 인코딩 

In [219]:
object_columns = list(df_train.select_dtypes(include='object').columns)
object_columns.append('customer_idx')
object_columns.append('lead_owner')
object_columns.append('country_corp_status')
object_columns.append('country_corp_status_1')
object_columns.append('country_corp_status_2')

df_train[object_columns] = df_train[object_columns].fillna('Unknown')
df_test[object_columns] = df_test[object_columns].fillna('Unknown')

In [220]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [221]:
# 레이블 인코딩할 칼럼들
label_columns = object_columns

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

In [222]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

***

## 4. Modeling

### ***😀CAT Boost with all***

In [223]:
X = df_train.drop(columns = ['is_converted'])
# X = X.drop(columns = trash_ft)
y = df_train['is_converted'].values

In [224]:
cat_features = object_columns.copy()

In [225]:
len(cat_features)

54

In [226]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import StratifiedGroupKFold

is_holdout = False

model_cat = []
f1_scores = []
n_split_list = [10]
for state in [0,5,11,42,18,19]:
    for split in n_split_list:
        fold_idx = 1
        cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=state)
        for train_index, valid_index in cv.split(X,y):
            X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
            Y_train, Y_valid = y[train_index], y[valid_index]
            print("="*50)
            pos_weight = len(Y_train)/sum(Y_train)

            model = CatBoostClassifier(iterations=1000,
                                   random_state=state,
                                   task_type="CPU",
                                   depth = 5,
                                   eval_metric="F1",
                                   #class_weights = {0: (0.1), 1: (1.1)},
                                   scale_pos_weight=pos_weight,
                                   bootstrap_type='Bayesian',  # Bayesian Bootstrap 사용
                                   random_strength = 4,
                                   cat_features=cat_features,
                                   l2_leaf_reg = 5,
                                   bagging_temperature = 0.5,
                                   one_hot_max_size=8,
                                   grow_policy='Depthwise',
                                   learning_rate=0.1
                                      )   

            model.fit(X_train, Y_train, 
                      eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                      early_stopping_rounds=200, 
                      verbose=500)
            
            pred = model.predict_proba(X_valid)[:,1]
            threshold = 0.5

            pred = np.where(pred >= threshold , True, False)
            score = f1_score(Y_valid, pred, labels=[True, False], average = 'macro')
            print(fold_idx,"Fold Validation F1 score :", score)
            f1_scores.append(score)
            model_cat.append(model)
            fold_idx += 1
            
            gc.collect()
            
            if is_holdout:
                break 
    print(state,'학습 완료')
    
print("Validation : F1 scores for each fold:", f1_scores)
print("Validation : F1:", np.mean(f1_scores))

0:	learn: 0.8510940	test: 0.8712081	test1: 0.8629698	best: 0.8629698 (0)	total: 151ms	remaining: 2m 30s
500:	learn: 0.9735358	test: 0.9861071	test1: 0.9533474	best: 0.9552804 (365)	total: 16.6s	remaining: 16.6s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9552804172
bestIteration = 365

Shrink model to first 366 iterations.
1 Fold Validation F1 score : 0.8787590433863302
0:	learn: 0.8462516	test: 0.8707790	test1: 0.8531405	best: 0.8531405 (0)	total: 80.4ms	remaining: 1m 20s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9509772486
bestIteration = 170

Shrink model to first 171 iterations.
2 Fold Validation F1 score : 0.8508631038967074
0:	learn: 0.8466484	test: 0.8704776	test1: 0.8578829	best: 0.8578829 (0)	total: 122ms	remaining: 2m 2s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9566958848
bestIteration = 245

Shrink model to first 246 iterations.
3 Fold Validation F1 score : 0.8577225389959631
0:	learn: 0.8879386	

---
### ***😀CAT Boost without idx***

In [227]:
X_without_idx = df_train.drop(columns = ['is_converted'])
X_without_idx = X_without_idx.drop(columns = ['customer_idx'])
y_without_idx = df_train['is_converted'].values

In [228]:
cat_features_without_idx = object_columns.copy()
cat_features_without_idx.remove('customer_idx')

In [229]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import StratifiedGroupKFold

is_holdout = False

model_cat_without_idx = []
f1_scores_without_idx = []
n_split_list_without_idx = [10]
for state in [0,5,11,42,18,19]:
    for split in n_split_list_without_idx:
        fold_idx = 1
        cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=state)
        for train_index, valid_index in cv.split(X_without_idx,y_without_idx):
            X_train, X_valid = X_without_idx.iloc[train_index], X_without_idx.iloc[valid_index]
            Y_train, Y_valid = y_without_idx[train_index], y_without_idx[valid_index]
            print("="*50)
            pos_weight = len(Y_train)/sum(Y_train)

            model = CatBoostClassifier(iterations=1000,
                                   random_state=state,
                                   task_type="CPU",
                                   depth = 5,
                                   eval_metric="F1",
                                   #class_weights = {0: (0.1), 1: (1.1)},
                                   scale_pos_weight=pos_weight,
                                   bootstrap_type='Bayesian',  # Bayesian Bootstrap 사용
                                   random_strength = 4,
                                   cat_features=cat_features_without_idx,
                                   l2_leaf_reg = 5,
                                   bagging_temperature = 0.5,
                                   one_hot_max_size=8,
                                   grow_policy='Depthwise',
                                   learning_rate=0.1
                                      )   

            model.fit(X_train, Y_train, 
                      eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                      early_stopping_rounds=200, 
                      verbose=500)
            
            pred = model.predict_proba(X_valid)[:,1]
            threshold = 0.5

            pred = np.where(pred >= threshold , True, False)
            score = f1_score(Y_valid, pred, labels=[True, False], average = 'macro')
            print(fold_idx,"Fold Validation F1 score :", score)
            f1_scores_without_idx.append(score)
            model_cat_without_idx.append(model)
            fold_idx += 1
            
            gc.collect()
            
            if is_holdout:
                break 
    print(state,'학습 완료')
    
print("Validation : F1 scores for each fold:", f1_scores_without_idx)
print("Validation : F1:", np.mean(f1_scores_without_idx))

0:	learn: 0.8489494	test: 0.8689760	test1: 0.8606766	best: 0.8606766 (0)	total: 79.2ms	remaining: 1m 19s
500:	learn: 0.9508635	test: 0.9578114	test1: 0.9189014	best: 0.9230642 (436)	total: 21.5s	remaining: 21.4s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.923064232
bestIteration = 436

Shrink model to first 437 iterations.
1 Fold Validation F1 score : 0.8022055133849476
0:	learn: 0.8506243	test: 0.8726700	test1: 0.8508978	best: 0.8508978 (0)	total: 77.3ms	remaining: 1m 17s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9213304998
bestIteration = 278

Shrink model to first 279 iterations.
2 Fold Validation F1 score : 0.7750656153144391
0:	learn: 0.8509645	test: 0.8706734	test1: 0.8613500	best: 0.8613500 (0)	total: 76.7ms	remaining: 1m 16s
500:	learn: 0.9528062	test: 0.9563433	test1: 0.9023854	best: 0.9140136 (356)	total: 17.2s	remaining: 17.1s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9140136463
bestIteration = 35

---
### ***😀CAT Boost without lead***

In [230]:
X_without_lead = df_train.drop(columns = ['is_converted'])
X_without_lead = X_without_lead.drop(columns = ['lead_owner'])
y_without_lead = df_train['is_converted'].values

In [231]:
cat_features_without_lead = object_columns.copy()
cat_features_without_lead.remove('lead_owner')

In [232]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import StratifiedGroupKFold

is_holdout = False

model_cat_without_lead = []
f1_scores_without_lead = []
n_split_list_without_lead = [10]
for state in [0,5,11,42,18,19]:
    for split in n_split_list_without_lead:
        fold_idx = 1
        cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=state)
        for train_index, valid_index in cv.split(X_without_lead,y_without_lead):
            X_train, X_valid = X_without_lead.iloc[train_index], X_without_lead.iloc[valid_index]
            Y_train, Y_valid = y_without_lead[train_index], y_without_lead[valid_index]
            print("="*50)
            pos_weight = len(Y_train)/sum(Y_train)

            model = CatBoostClassifier(iterations=1000,
                                   random_state=state,
                                   task_type="CPU",
                                   depth = 5,
                                   eval_metric="F1",
                                   #class_weights = {0: (0.1), 1: (1.1)},
                                   scale_pos_weight=pos_weight,
                                   bootstrap_type='Bayesian',  # Bayesian Bootstrap 사용
                                   random_strength = 4,
                                   cat_features=cat_features_without_lead,
                                   l2_leaf_reg = 5,
                                   bagging_temperature = 0.5,
                                   one_hot_max_size=8,
                                   grow_policy='Depthwise',
                                   learning_rate=0.1
                                      )   

            model.fit(X_train, Y_train, 
                      eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                      early_stopping_rounds=200, 
                      verbose=500)
            
            pred = model.predict_proba(X_valid)[:,1]
            threshold = 0.5

            pred = np.where(pred >= threshold , True, False)
            score = f1_score(Y_valid, pred, labels=[True, False], average = 'macro')
            print(fold_idx,"Fold Validation F1 score :", score)
            f1_scores_without_lead.append(score)
            model_cat_without_lead.append(model)
            fold_idx += 1
            
            gc.collect()
            
            if is_holdout:
                break 
    print(state,'학습 완료')        
        
print("Validation : F1 scores for each fold:", f1_scores_without_lead)
print("Validation : F1:", np.mean(f1_scores_without_lead))

0:	learn: 0.8623842	test: 0.8961467	test1: 0.8489936	best: 0.8489936 (0)	total: 81.9ms	remaining: 1m 21s
500:	learn: 0.9582387	test: 0.9833326	test1: 0.9050564	best: 0.9090292 (377)	total: 16.9s	remaining: 16.8s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9090291905
bestIteration = 377

Shrink model to first 378 iterations.
1 Fold Validation F1 score : 0.8181786463489626
0:	learn: 0.7298601	test: 0.7461465	test1: 0.7388935	best: 0.7388935 (0)	total: 84.6ms	remaining: 1m 24s
500:	learn: 0.9581451	test: 0.9812403	test1: 0.9119314	best: 0.9183737 (353)	total: 16.7s	remaining: 16.6s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9183737044
bestIteration = 353

Shrink model to first 354 iterations.
2 Fold Validation F1 score : 0.8169463706148248
0:	learn: 0.8481638	test: 0.8922496	test1: 0.8506043	best: 0.8506043 (0)	total: 78.5ms	remaining: 1m 18s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9193796502
bestIteration = 1

---
### ***😀CAT Boost without all***

In [233]:
X_without_all = df_train.drop(columns = ['is_converted'])
X_without_all = X_without_all.drop(columns = ['customer_idx'])
X_without_all = X_without_all.drop(columns = ['lead_owner'])
y_without_all = df_train['is_converted'].values

In [234]:
cat_features_without_all = object_columns.copy()
cat_features_without_all.remove('customer_idx')
cat_features_without_all.remove('lead_owner')

In [235]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import StratifiedGroupKFold

is_holdout = False

model_cat_without_all = []
f1_scores_without_all = []
n_split_list_without_all = [10]
for state in [0,5,11,42,18,19]:
    for split in n_split_list_without_all:
        fold_idx = 1
        cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=state)
        for train_index, valid_index in cv.split(X_without_all,y_without_all):
            X_train, X_valid = X_without_all.iloc[train_index], X_without_all.iloc[valid_index]
            Y_train, Y_valid = y_without_all[train_index], y_without_all[valid_index]
            print("="*50)
            pos_weight = len(Y_train)/sum(Y_train)

            model = CatBoostClassifier(iterations=1000,
                                   random_state=state,
                                   task_type="CPU",
                                   depth = 5,
                                   eval_metric="F1",
                                   #class_weights = {0: (0.1), 1: (1.1)},
                                   scale_pos_weight=pos_weight,
                                   bootstrap_type='Bayesian',  # Bayesian Bootstrap 사용
                                   random_strength = 4,
                                   cat_features=cat_features_without_all,
                                   l2_leaf_reg = 5,
                                   bagging_temperature = 0.5,
                                   one_hot_max_size=8,
                                   grow_policy='Depthwise',
                                   learning_rate=0.1
                                      )   

            model.fit(X_train, Y_train, 
                      eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
                      early_stopping_rounds=200, 
                      verbose=500)
            
            pred = model.predict_proba(X_valid)[:,1]
            threshold = 0.5

            pred = np.where(pred >= threshold , True, False)
            score = f1_score(Y_valid, pred, labels=[True, False], average = 'macro')
            print(fold_idx,"Fold Validation F1 score :", score)
            f1_scores_without_all.append(score)
            model_cat_without_all.append(model)
            fold_idx += 1
            
            gc.collect()
            
            if is_holdout:
                break 
    print(state,'학습 완료')
    
                
print("Validation : F1 scores for each fold:", f1_scores_without_all)
print("Validation : F1:", np.mean(f1_scores_without_all))

0:	learn: 0.7928277	test: 0.7925809	test1: 0.7915416	best: 0.7915416 (0)	total: 75.4ms	remaining: 1m 15s
500:	learn: 0.9252787	test: 0.9278288	test1: 0.8730024	best: 0.8741581 (430)	total: 16.2s	remaining: 16.1s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8750933821
bestIteration = 525

Shrink model to first 526 iterations.
1 Fold Validation F1 score : 0.7321229351961964
0:	learn: 0.7904106	test: 0.7933373	test1: 0.8011683	best: 0.8011683 (0)	total: 77.2ms	remaining: 1m 17s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8796811006
bestIteration = 202

Shrink model to first 203 iterations.
2 Fold Validation F1 score : 0.6983451536643026
0:	learn: 0.7911910	test: 0.7922049	test1: 0.7992456	best: 0.7992456 (0)	total: 76.8ms	remaining: 1m 16s
500:	learn: 0.9273089	test: 0.9262743	test1: 0.8715573	best: 0.8851425 (384)	total: 16.4s	remaining: 16.3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8851424664
bestIteration = 3

---
### ***CATBOOST PREDICT***

In [236]:
X_test = df_test.drop(columns = ['id','is_converted'])

In [237]:
with_all_idx = X_test.loc[(X_test['customer_idx'].isin(df_train['customer_idx'].unique())) & (X_test['lead_owner'].isin(df_train['lead_owner'].unique()))].index
without_idx_ldx = X_test.loc[(~X_test['customer_idx'].isin(df_train['customer_idx'].unique())) & (X_test['lead_owner'].isin(df_train['lead_owner'].unique()))].index
without_lead_ldx = X_test.loc[(X_test['customer_idx'].isin(df_train['customer_idx'].unique())) & (~X_test['lead_owner'].isin(df_train['lead_owner'].unique()))].index
without_all_ldx = X_test.loc[(~X_test['customer_idx'].isin(df_train['customer_idx'].unique())) & (~X_test['lead_owner'].isin(df_train['lead_owner'].unique()))].index

In [238]:
print('둘다 겹치는 것 : ', len(with_all_idx), '\n'\
     'lead_owner만 겹치는 것 : ', len(without_idx_ldx), '\n'\
     'customer_idx만 겹치는 것 : ', len(without_lead_ldx), '\n'\
     '둘다 안 겹치는 것 : ', len(without_all_ldx))

둘다 겹치는 것 :  1114 
lead_owner만 겹치는 것 :  3711 
customer_idx만 겹치는 것 :  84 
둘다 안 겹치는 것 :  362


In [239]:
test_pred_with_all_idx = []
test_with_all_idx = X_test.loc[with_all_idx]
for i in range(len(model_cat)):
    pred = model_cat[i].predict_proba(test_with_all_idx)[:,1]
    test_pred_with_all_idx.append(pred)

    
test_pred_with_all_idx = np.array(test_pred_with_all_idx)

sorted_probs = np.sort(test_pred_with_all_idx, axis=0)
mean_pred_probs = np.mean(sorted_probs[3:-3], axis=0)

test_pred_final_with_all_idx = mean_pred_probs >= 0.5
    
print(sum(test_pred_final_with_all_idx) , '/' , len(test_with_all_idx))

408 / 1114


In [240]:
test_pred_without_idx = []
X_test_without_idx = X_test.drop(columns=['customer_idx'])
test_without_idx = X_test_without_idx.loc[without_idx_ldx]
for i in range(len(model_cat_without_idx)):
    pred = model_cat_without_idx[i].predict_proba(test_without_idx)[:,1]
    test_pred_without_idx.append(pred)

    
test_pred_without_idx = np.array(test_pred_without_idx)

sorted_probs = np.sort(test_pred_without_idx, axis=0)
mean_pred_probs = np.mean(sorted_probs[3:-3], axis=0)

test_pred_final_without_idx = mean_pred_probs >= 0.5
    
print(sum(test_pred_final_without_idx) , '/' , len(test_without_idx))

1183 / 3711


In [241]:
test_pred_without_lead = []
X_test_without_lead = X_test.drop(columns=['lead_owner'])
test_without_lead = X_test_without_lead.loc[without_lead_ldx]
for i in range(len(model_cat_without_lead)):
    pred = model_cat_without_lead[i].predict_proba(test_without_lead)[:,1]
    test_pred_without_lead.append(pred)

    
test_pred_without_lead = np.array(test_pred_without_lead)

sorted_probs = np.sort(test_pred_without_lead, axis=0)
mean_pred_probs = np.mean(sorted_probs[3:-3], axis=0)

test_pred_final_without_lead = mean_pred_probs >= 0.5
    
print(sum(test_pred_final_without_lead) , '/' , len(test_without_lead))

33 / 84


In [242]:
test_pred_without_all = []
X_test_without_all = X_test.drop(columns=['lead_owner'])
X_test_without_all = X_test_without_all.drop(columns=['customer_idx'])
test_without_all = X_test_without_all.loc[without_all_ldx]
for i in range(len(model_cat_without_all)):
    pred = model_cat_without_all[i].predict_proba(test_without_all)[:,1]
    test_pred_without_all.append(pred)

    
test_pred_without_all = np.array(test_pred_without_all)

sorted_probs = np.sort(test_pred_without_all, axis=0)
mean_pred_probs = np.mean(sorted_probs[3:-3], axis=0)

test_pred_final_without_all = mean_pred_probs >= 0.5
    
print(sum(test_pred_final_without_all) , '/' , len(test_without_all))

122 / 362


***

In [243]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub_cat = pd.read_csv("submission.csv")

In [244]:
df_sub_cat.loc[with_all_idx,'is_converted'] = test_pred_final_with_all_idx
df_sub_cat.loc[without_idx_ldx,'is_converted'] = test_pred_final_without_idx
df_sub_cat.loc[without_lead_ldx,'is_converted'] = test_pred_final_without_lead
df_sub_cat.loc[without_all_ldx,'is_converted'] = test_pred_final_without_all

In [245]:
df_sub_cat['is_converted'].value_counts()

is_converted
False    3525
True     1746
Name: count, dtype: int64

***

## ***제출***

In [253]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")

In [254]:
df_sub['is_converted'] = df_sub_cat['is_converted'].copy()

In [255]:
df_sub['is_converted'].value_counts()

is_converted
False    3525
True     1746
Name: count, dtype: int64

In [256]:
df_sub.to_csv("submission.csv", index=False)

***